####  The code in this script can be used to replicate the publication figures from the decode pipeline output files (provided in https://drive.google.com/open?id=15YoTBTZh4MdtAqHbibkYieEqyLyFi5hb&usp=drive_fs)  and the Supplemental Data Files.

Most analysis required for replicating the figures is computed here. Data compiled elsewhere (e.g. abundances of tRNA ligases) and required for plots, is provided as a table in the google drive folder 2024_Tsour/pipeline_output/analysis_dependencies/.

A subset of analyses and figures that are not present here were generated in an independent script and can be accessed with other scripts in the repository, found in decode_analysis. These include Figure 2j; Figure 3b,c,e; Figure 4c,d,e; Figure 5a,b,c,d; Extndede Data Figure 3k; Extended Data Figure 4; Extended Data Figure 5a,c,f; Extended Data Figure 7; Extended Data Figure 8; Extended Data Figure 9b,c,d; Extended Data Figure 10a,b,c.

There is a lot of code in this script. It is organized and labeled by references to figures in the manuscript. It is recommended to find and run the analyses of interest, rather than running the entire script at once. E.g. search page for "Figure 3a" to find code to replicate that data.

In [4]:
# read in packages needed

import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
from collections import Counter
import scipy as sp
from copy import deepcopy
from statsmodels.stats.multitest import multipletests
from adjustText import adjust_text
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
from matplotlib.colors import LogNorm, Normalize
from matplotlib.ticker import MaxNLocator
import os
import ast
from scipy.odr import Model, RealData, ODR
from scipy.stats import gaussian_kde
from matplotlib import gridspec
import milkviz as mv
from matplotlib_venn import venn3
import zipfile
from glob import glob
import gzip
import tarfile
from matplotlib.lines import Line2D
from matplotlib.ticker import AutoMinorLocator, FixedLocator
import statsmodels.api as sm
from statsmodels.formula.api import ols

#### Setting directories and reading in data.

All of these directories should be updated by the user to reflect where their output data from the decode pipeline st stored.


In [ ]:
proj_dir = '/Users/shiri/Google Drive/My Drive/MS/SuppData/2024_Tsour/'

ccrcc_proj_dir = proj_dir+'pipeline_output/CCRCC/'
ccrcc_aa_subs_dir = ccrcc_proj_dir+'AA_subs_pipeline/'# aa_subs_dir is the output folder
ccrcc_sample_map = pd.read_excel(ccrcc_aa_subs_dir+'sample_map.xlsx')
ccrcc_samples = ['S'+str(i) for i in list(set(ccrcc_sample_map['TMT plex']))]

ucec_proj_dir = proj_dir+'pipeline_output/UCEC/'
ucec_aa_subs_dir = ucec_proj_dir+'AA_subs_pipeline/'# aa_subs_dir is the output folder
ucec_sample_map = pd.read_excel(ucec_aa_subs_dir+'sample_map.xlsx')
ucec_samples = ['S'+str(i) for i in list(set(ucec_sample_map['TMT plex']))]

brca_proj_dir = proj_dir+'pipeline_output/BRCA/'
brca_aa_subs_dir = brca_proj_dir+'AA_subs_pipeline/'# aa_subs_dir is the output folder
brca_sample_map = pd.read_excel(brca_aa_subs_dir+'sample_map.xlsx')
brca_samples = ['S'+str(i) for i in list(set(brca_sample_map['TMT plex']))]

luad_proj_dir = proj_dir+'pipeline_output/LUAD/'
luad_aa_subs_dir = luad_proj_dir+'AA_subs_pipeline/'# aa_subs_dir is the output folder
luad_sample_map = pd.read_excel(luad_aa_subs_dir+'sample_map.xlsx')
luad_samples = ['S'+str(i) for i in list(set(luad_sample_map['TMT plex']))]

pdac_proj_dir = proj_dir+'pipeline_output/PDAC/'
pdac_aa_subs_dir = pdac_proj_dir+'AA_subs_pipeline/'# aa_subs_dir is the output folder
pdac_sample_map = pd.read_excel(pdac_aa_subs_dir+'sample_map.xlsx')
pdac_samples = ['S'+str(i) for i in list(set(pdac_sample_map['TMT plex']))]

lscc_proj_dir = proj_dir+'pipeline_output/LSCC/'
lscc_aa_subs_dir = lscc_proj_dir+'AA_subs_pipeline/'# aa_subs_dir is the output folder
lscc_sample_map = pd.read_excel(lscc_aa_subs_dir+'sample_map.xlsx')
lscc_samples = ['S'+str(i) for i in list(set(lscc_sample_map['TMT plex']))]

wang_proj_dir = proj_dir+'pipeline_output/HealthyTissues/'
wang_data_dir = wang_proj_dir+'AA_subs_pipeline/'# aa_subs_dir is the output folder
wang_samples = open(wang_data_dir+'proteomic_tissue_list.txt', 'r').read().split('\n')
wang_samples = [x for x in wang_samples if x!='rectum']

# lists used to quickly index dataset-specific sample maps (e.g.) from just the dataset name
datasets = ['CCRCC', 'UCEC', 'BRCA', 'LUAD', 'PDAC', 'LSCC', 'Healthy']
data_dir_list = [ccrcc_aa_subs_dir, ucec_aa_subs_dir, brca_aa_subs_dir, luad_aa_subs_dir, pdac_aa_subs_dir, lscc_aa_subs_dir, wang_data_dir]
samples_list = [ccrcc_samples, ucec_samples, brca_samples, luad_samples, pdac_samples, lscc_samples, wang_samples]
sample_map_list = [ccrcc_sample_map, ucec_sample_map, brca_sample_map, luad_sample_map, pdac_sample_map, lscc_sample_map]
proj_dir_list = [ccrcc_proj_dir, ucec_proj_dir, brca_proj_dir, luad_proj_dir, pdac_proj_dir, lscc_proj_dir, wang_proj_dir]

outdir = 'figures/' # output directory where to save figures

In [ ]:
class GridShader():
    """
    function used to create alternating vertical gray and white background in plots
    """
    def __init__(self, ax, first=True, **kwargs):
        self.spans = []
        self.sf = first
        self.ax = ax
        self.kw = kwargs
        self.ax.autoscale(False, axis="x")
        self.cid = self.ax.callbacks.connect('xlim_changed', self.shade)
        self.shade()
    def clear(self):
        for span in self.spans:
            try:
                span.remove()
            except:
                pass
    def shade(self, evt=None):
        self.clear()
        xticks = self.ax.get_xticks()
        xlim = self.ax.get_xlim()
        xticks = xticks[(xticks > xlim[0]) & (xticks < xlim[-1])]
        locs = np.concatenate(([[xlim[0]], xticks, [xlim[-1]]]))

        start = [x-0.5 for x in locs[1-int(self.sf)::2]]
        end = [x-0.5 for x in locs[2-int(self.sf)::2]]

        for s, e in zip(start, end):
            self.spans.append(self.ax.axvspan(s, e, zorder=0, **self.kw))


def bihist(y1, y2, nbins=10, h=None):
    '''
    Function used to create violin plots as bihistograms with no smoothing.
    h is an axis handle. If not present, a new figure is created.
    '''
    if h is None: h = plt.figure().add_subplot(111)
    ymin = np.floor(np.minimum(min(y1), min(y2)))
    ymax = np.ceil(np.maximum(max(y1), max(y2)))
    bins = np.linspace(ymin, ymax, nbins)
    n1, bins1, patch1 = h.hist(y1, bins, orientation='horizontal', color='#AAAAAA', edgecolor=None, linewidth=0,rwidth=1)
    n2, bins2, patch2 = h.hist(y2, bins, orientation='horizontal', color='#AAAAAA', edgecolor=None, linewidth=0,rwidth=1)
    # set xmax:
    xmax = 0
    for i in patch1:
        i.set_edgecolor(None)
        width = i.get_width()
        if width > xmax: xmax = width
    # invert second histogram and set xmin:
    xmin = 0
    for i in patch2:
        i.set_edgecolor(None)
        width = i.get_width()
        width = -width
        i.set_width(width)
        if width < xmin: xmin = width
    h.set_xlim(xmin*1.1, xmax*1.1)          
    h.figure.canvas.draw()

# Figure 1. SAAP detection

#### Incl. Extended Data Figure 1

### Fig1a. N peptide IDs through filtering steps, from all DP to validated SAAP. Data used in generating Fig1a in biorender.

In [ ]:
# get number of samples in each dataset
n_samples_data = []
total = 0
for ds in datasets:
    if ds!='Healthy':
        sample_map = sample_map_list[datasets.index(ds)]
        samples = sample_map['sample_name'].values
    else:
        samples = samples_list[datasets.index(ds)] 
    n_samples = len(samples)
    total += n_samples
    n_samples_data.append([ds, n_samples])
    print(ds, n_samples)
print(total)
n_samples_df = pd.DataFrame(n_samples_data, columns=['Dataset', 'N samples'])
n_samples_df.to_excel(outdir+'N_samples_in_datasets.xlsx')

In [ ]:
def get_filter_df(data_dir, samples):
    """ function that reads in output from decode pipeline to get number of peptides in each category
    Input: directory with output files for dataset, dataset samples/TMT set names
    output: dataframe with the number of peptides in each category used to create fig 1a, c
    """
    dp_ev_dict = pickle.load(open(data_dir+'DP_search_evidence_dict.p', 'rb'))
    dp_dict = pickle.load(open(data_dir+'DP_dict.p', 'rb'))
    mtp_dict = pickle.load(open(data_dir+'MTP_dict.p', 'rb'))
    ptm_dict = pickle.load(open(data_dir+'PTM_dict.p', 'rb'))
    hc_mtp_dict = pickle.load(open(data_dir+'qMTP_dict.p', 'rb'))
    substr_dict = pickle.load(open(data_dir+'genome_substr_dict.p', 'rb'))
    val_mtp_dict = pickle.load(open(data_dir+'Ion_validated_MTP_dict.p','rb'))
    
    rows = []
    for s in samples:
        seqs = [[x for y in list(mtp_dict[s]['mistranslated sequence'].values()) for x in y] for s in samples]
        seqs = [x for y in seqs for x in y]
        homolog_seqs = substr_dict['all_6frame_seqs']
        seqs_hom = [x for x in seqs if x in homolog_seqs]
        
        n_main = len(dp_ev_dict[s]['Raw file'])
        n_dp = len(dp_dict[s]['Raw file'])
        n_ptm = len(ptm_dict[s]['Raw file'])
        n_aas = len(mtp_dict[s]['Raw file'])
        n_nohom = len([i for i,x in mtp_dict[s]['mistranslated sequence'].items() if all(y not in seqs_hom for y in x)])
        n_hc = len([i for i,x in hc_mtp_dict[s]['mistranslated sequence'].items() if all(y not in seqs_hom for y in x)])
        n_val = len(val_mtp_dict[s]['Raw file'])

        rows.append([s, n_main, n_dp, n_ptm, n_aas, n_nohom, n_hc, n_val])
    df = pd.DataFrame(rows, columns=['TMT set', 'Main peptides', 'DP', 'PTM', 'AAS', 'Non-homologous', 'High-confidence', 'Validated'])

    return(df)

In [ ]:
# create a dictionary containing the dataframes generated with the above function for each dataset
filter_dict = {}
for ds in datasets:
    print(ds)
    data_dir = data_dir_list[datasets.index(ds)]
    samples = samples_list[datasets.index(ds)]
    filter_dict[ds] = get_filter_df(data_dir, samples)
pickle.dump(filter_dict, open(outdir+'Modified_peptide_filter_dict_DP2valSAAP.p', 'wb'))
#filter_dict = pickle.load(open(outdir+'Modified_peptide_filter_dict_DP2valSAAP.p', 'rb'))

#### print numbers needed for figure 1A - all datasets
1. Total N DP 
2. Total N match to AAS
3. Total N match to AAS and not genome
4. Total N match to AAS, not genome and high confidence
5. Total N validated

In [ ]:
all_df = pd.concat([filter_dict[ds] for ds in datasets])

sum_dict = {}
for col in all_df.columns:
    if col not in ['TMT set', 'Dataset', 'Tissue']:
        sum_dict[col] = np.nansum(all_df[col].values)
        
print(sum_dict) # these values are used in 1a

### Figure 1c. Filtering modified peptides to candidate SAAP

In [ ]:
boxplot_rows = []
for i,row in all_df.iterrows():
    boxplot_rows.append([row['Dataset'], row['Main peptides'], 'Main peptides', row['TMT set']])
    boxplot_rows.append([row['Dataset'], row['DP'], 'Dependent peptides', row['TMT set']])
    boxplot_rows.append([row['Dataset'], row['PTM'], 'Peptides with PTM', row['TMT set']])
    #boxplot_rows.append([row['Dataset'], row['AAS'], 'Peptides with AAS', row['TMT set']])
    #boxplot_rows.append([row['Dataset'], row['Non-homologous'], 'Non-homologous AAS', row['TMT set']])
    boxplot_rows.append([row['Dataset'], row['High-confidence'], 'High-confidence AAS', row['TMT set']])
    
boxplot_df = pd.DataFrame(boxplot_rows, columns=['Dataset','N peptide IDs', 'Peptide type', 'TMT set'])

fig,ax = plt.subplots(figsize=(4,3.5))
sns.set_style('whitegrid')
plt.grid(True,which="major",c='gray')  
plt.grid(True,which="minor",ls="--",c='gray', linewidth=0.3)  

sns.stripplot(data= boxplot_df, x='Dataset', y='N peptide IDs', hue='Peptide type',dodge=False,s=3, linewidth=0.4, edgecolor='k')
sns.boxplot(data= boxplot_df, x='Dataset', y='N peptide IDs', hue='Peptide type', dodge=False, linewidth=0.5, fliersize=0.7, saturation=1)
plt.yscale('log')
plt.ylim([100,10**6])
plt.xticks(rotation=45, fontsize=13)
plt.yticks(fontsize=13)
plt.ylabel('# peptides$/$sample', fontsize=15)
plt.xlabel('')
handles, labels = ax.get_legend_handles_labels()
custom_labels = ['Main peptides', 'Modified peptides', 'Peptides with PTM','Candidate SAAP']
plt.legend(handles=handles[0:], labels=custom_labels, bbox_to_anchor=(0.5,1.1), fontsize=11, ncol=2, labelspacing=0.3, handletextpad=0, frameon=False, columnspacing=0.3, loc='center');
plt.savefig(outdir+'Dataset_filtering_boxplot_wmainpeptides.pdf', bbox_inches='tight')

### Fig 1D. PTMs

In [ ]:
def get_ptm_df(ptm_dict):
"""
function to get a dataframe of PTMS x datasets from the datasets' PTM_dict.p
"""
    master_ptm_list = []
    for s,v in ptm_dict.items():
        s_ptm_dict = v['PTM']
        for ptm_list in s_ptm_dict.values():
            new_ptm_list = [x for x in ptm_list if x not in master_ptm_list]
            new_ptm_list = list(set(new_ptm_list))
            master_ptm_list = master_ptm_list + new_ptm_list

    heatmap_df = pd.DataFrame(index=master_ptm_list, columns=list(range(1,24)))
    for s, v in ptm_dict.items():
        s_int = int(s[1:])
        ptm_list = list(v['PTM'].values())
        ptm_list = [x for y in ptm_list for x in y]
        ptm_count = Counter(ptm_list)
        for ptm, count in ptm_count.items():
            heatmap_df.loc[ptm, s_int] = count
            #heatmap_df.loc[ptm, s_int] = np.log10(count)
    heatmap_df.fillna(0, inplace=True)
    return(heatmap_df)


In [ ]:
# get PTM dataframe for each dataset
for i in range(len(datasets)):
    print(datasets[i])
    data_dir =  data_dir_list[i]
    ptm_dict = pickle.load(open(data_dir+'PTM_dict.p', 'rb'))
    heatmap_df = get_ptm_df(ptm_dict)
    heatmap_df.to_excel(data_dir+'PTM_heatmap_df.xlsx')
    
# create a dictionary with the PTM dataframes for each dataset
ptm_heatmap_dict = {}
for i,ds in enumerate(datasets):
    data_dir = data_dir_list[i]
    ptm_df = pd.read_excel(data_dir + 'PTM_heatmap_df.xlsx', index_col=0)
    ptm_df['avg'] = [np.mean(row.values) for i,row in ptm_df.iterrows()]
    ptm_df.sort_values('avg', ascending=False, inplace=True) # sort by frequency to extract top PTMs for plot
    
    ptm_df.index = [x[0].upper()+x[1:] for x in ptm_df.index]
    ptm_heatmap_dict[ds] = ptm_df
    
pickle.dump(ptm_heatmap_dict, open(outdir+'PTM_heatmap_dict.p', 'wb'))
#ptm_heatmap_dict = pickle.load(open(nofilter_outdir+'PTM_heatmap_dict.p', 'rb'))


# extract the top PTMs by frequency into a dataframe for plotting
top20 = []
for ds, ds_df in ptm_heatmap_dict.items():
    if len(top20)==0:
        top20 = ds_df.index.values[0:40]
    else:
        top20new = ds_df.index.values[0:40]
        top20 = [x for x in top20 if x in top20new]
plot_df = pd.DataFrame(index=top20, columns=datasets)
for ptm in top20:
    for ds in datasets:
        heatmap_df = ptm_heatmap_dict[ds]
        plot_df.loc[ptm,ds] = heatmap_df.loc[ptm,'avg']
plot_df = plot_df.astype(float)

# need number of peptides IDd to normalize N each PTM per 1000 peptides
ds_metrics= pd.read_excel(outdir+'Dataset_metrics.xlsx', index_col=0) # dataframe created externally with number of peptides identified in the main seach of each dataset
n_peptides_list = []
for ds in datasets:
    n_peptides = ds_metrics.loc[ds_metrics['Dataset']==ds, 'Peptides (evidence)'].values[0]
    n_peptides_list.append(n_peptides)

# normalize PTM plot df by per thousand peptides
scaled_plot_df = deepcopy(plot_df)
for i,ds in enumerate(datasets):
    scaled_plot_df[ds] = [x/(n_peptides_list[i]/1000) for x in scaled_plot_df[ds]]
scaled_plot_df.to_excel(outdir+'PTM_heatmap_data.xlsx')

In [ ]:
# plot PTM heatmap
cg = sns.clustermap(data=scaled_plot_df, yticklabels=True,norm=LogNorm(), cbar_kws={'orientation':'horizontal', 'ticks':[0.01,0.1,1,10]},
                    figsize=(5,5.5), cmap=sns.color_palette('rocket', as_cmap=True), method='ward', col_cluster=False, row_cluster=True)
cg.ax_cbar.set_title(r'PTMs per thousand peptides')
plt.setp(cg.ax_heatmap.xaxis.get_majorticklabels(), rotation=90, fontsize=12)
plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), fontsize=11)
x0, y0, w, h = cg.cbar_pos
cg.ax_cbar.set_position([0.5, 0.04, 0.4, 0.04])
cg.ax_row_dendrogram.set_visible(False)
plt.savefig(outdir+'PTM_heatmap_top20.pdf', bbox_inches='tight')

### Exteded data figure 1e.  Types of modifications identified (PTM, chemical, etc.)

In [ ]:
all_PTMs = []
for ds, ptm_df in ptm_heatmap_dict.items():
    ds_ptms = list(ptm_df.index)
    all_PTMs = all_PTMs + ds_ptms
set_PTMs = list(set(all_PTMs))

# create a dictionary with the classification of each modification type
mod_class_dict = {}
mod_list =  pickle.load(open(outdir+'Modifications_dict.p', 'rb')) # derived from Unimod database
for mod_dict in mod_list:
    mod_name = mod_dict['Description']
    class_keys = [x for x in mod_dict if 'classification' in x]
    classes = [mod_dict[k] for k in class_keys if mod_dict[k]!=['Multiple']]
    classes = list(set([x if 'glycosylation' not in x else 'Post-translational' for x in classes]))
    mod_class_dict[mod_name.lower()] = classes

# get the number of modifications pertaining to each class
ptm_ptms = []
chemical_ptms = []
artifact_ptms = []
ptm_or_chemical_ptms = []
chem_or_artifact_ptms = []
ptm_or_artifact_ptms = []
alltypes_ptms = []

for i,ptm in enumerate(set_PTMs):
    if 'TMT' not in ptm:
        ptm = ptm.lower()
        ptm_class = mod_class_dict[ptm]
        if 'Post-translational' in ptm_class:
            if len(ptm_class)==1:
                ptm_ptms.append(ptm)
            elif 'Chemical derivative' in ptm_class and 'Artefact' in ptm_class:
                alltypes_ptms.append(ptm)
            elif 'Chemical derivative' in ptm_class:
                ptm_or_chemical_ptms.append(ptm)
            elif 'Artefact' in ptm_class:
                ptm_or_artifact_ptms.append(ptm)

        elif 'Chemical derivative' in ptm_class:
            if len(ptm_class)==1:
                chemical_ptms.append(ptm)
            elif 'Artefact' in ptm_class:
                chem_or_artifact_ptms.append(ptm)
        elif 'Artefact' in ptm_class:
            artifact_ptms.append(ptm)
    else:
        chemical_ptms.append(ptm)

n_ptms = len(ptm_ptms)
n_chemical_dervs = len(chemical_ptms)
n_artifacts = len(artifact_ptms)
n_ptm_chem = len(ptm_or_chemical_ptms)
n_chem_artifact = len(chem_or_artifact_ptms)
n_ptm_artifact = len(ptm_or_artifact_ptms)
n_any = len(alltypes_ptms)

# plot the Venn diagram
fig,ax = plt.subplots(figsize=(3,2))
v = venn3(subsets = (n_ptms, n_chemical_dervs, n_ptm_chem, n_artifacts, n_ptm_artifact, n_chem_artifact, n_any),
         set_labels = ('Post-\ntranslational', 'Chemical\nderivative', 'Artifact'), ax=ax)
plt.savefig(outdir+'PTM_source.pdf', bbox_inches='tight')


### Extended Data Figure 1a,b. N samples per dataset

In [ ]:
# get the number of samples in each CPTAC dataset and plot

plot_rows = []
plot_cols = ['Dataset', 'N samples', 'Sample type']
for ds in datasets[:-1]:
    sample_map = sample_map_list[datasets.index(ds)]
    n_t_samples = len(sample_map.loc[sample_map['Group']=='Tumor'])
    n_n_samples = len(sample_map.loc[sample_map['Group']=='Normal'])
    plot_rows.append([ds, n_t_samples, 'Tumor'])
    plot_rows.append([ds, n_n_samples, 'Normal adjacent tissue'])

plot_df = pd.DataFrame(plot_rows, columns=plot_cols)
sns.set_style('whitegrid')
fig,ax = plt.subplots(figsize=(4.5,3))
sns.barplot(data=plot_df, x='Dataset', y='N samples', hue='Sample type', palette=sns.color_palette(palette='Greys', n_colors=2))
ax.tick_params('both', labelsize=12)
plt.xlabel('')
plt.ylabel('# samples', fontsize=14)
plt.ylim([0,155])
plt.yticks([0,25,50,75,100,125,150])
plt.bar_label(plt.gca().containers[0])
plt.bar_label(plt.gca().containers[1])
plt.legend(ncol=2, bbox_to_anchor=(1.04,1.15), frameon=False, fontsize=13, handletextpad=0.1)
plt.savefig(outdir+'CPTAC_N_samples_per_dataset.pdf', bbox_inches='tight')

In [ ]:
# get the number of samples in each healthy tissue and plot

plot_rows = []
plot_cols = ['Tissue', 'N samples', 'Protease']
for sample in wang_samples:
    n = 1
    protease = 'trypsin'
    if sample == 'liver':
        n = 4
    elif sample == 'colon' or sample == 'placenta' or sample == 'lung' or sample == 'stomach' :
        n = 2
    plot_rows.append([sample, n, protease])
    plot_rows.append(['tonsil', 5, 'trypsin'])
    plot_rows.append(['tonsil', 2, 'chymotrypsin'])
    plot_rows.append(['tonsil', 1, 'LysN/ArgC/GluC'])
    #plot_rows.append(['tonsil', 1, 'GluC'])
    #plot_rows.append(['tonsil', 1, 'ArgC'])

plot_df = pd.DataFrame(plot_rows, columns=plot_cols)

sns.set_style('whitegrid')
fig,ax = plt.subplots(figsize=(6,2))
#fig,axes = plt.subplots(1,2, figsize=(8,3), sharey=True, gridspec_kw={'width_ratios':[4,1]})
plt.subplots_adjust(wspace=0.03)

sns.barplot(data=plot_df, x='Tissue', y='N samples', hue='Protease', dodge=False, palette=sns.color_palette(palette='Greys', n_colors=4)[1:])
ax.tick_params('both', labelsize=12)
ax.tick_params('x', rotation=90)
ax.set_xlabel('')
ax.set_ylabel('# samples', fontsize=14)
ax.set_ylim([0,5])
plt.legend(ncol=3, bbox_to_anchor=(0.5,1.05), frameon=False, fontsize=13, loc='center', handletextpad=0.1)

    
plt.savefig(outdir+'Healthy_N_samples_per_dataset.pdf', bbox_inches='tight')

### Extended Data Figure 1i. Mass errors of SAAP and main peptides

In [ ]:
# generate a dictionary with mass error data
mass_err_dict = {ds:{'Main peptide mass error':[], 'SAAP mass error':[]} for ds in datasets}
for ds in datasets:
    print(ds)
    data_dir = data_dir_list[datasets.index(ds)]
    mainpep_dict = pickle.load(open(data_dir+'DP_search_evidence_dict.p','rb'))
    samples = samples_list[datasets.index(ds)]
    for s in samples:
        print(s)
        errs = mainpep_dict[s]['Mass error [ppm]']
        errs = [x for x in errs if np.abs(x)<10000]
        mass_err_dict[ds]['Main peptide mass error'] = mass_err_dict[ds]['Main peptide mass error']+errs

pickle.dump(mass_err_dict, open(outdir+'Main_peptide_mass_error_ppm.p','wb'))


In [ ]:
#mass_err_dict = pickle.load(open(outdir+'Main_peptide_mass_error_ppm.p','rb'))

In [ ]:
# plot mass error boxplots
plot_df_list = []
for ds in datasets:
    print(ds)
    ds_me_dict = mass_err_dict[ds]
    main_pep_errs = ds_me_dict['Main peptide mass error']
    #if ds!='UCEC':
    saap_errs = [np.abs(x) for x in ds_me_dict['SAAP mass error']]
    saap_errs = saap_errs + [-x for x in saap_errs]
    ds_list = [ds]*len(main_pep_errs+saap_errs)
    main_str_list = ['Main peptide']*len(main_pep_errs)
    saap_str_list = ['SAAP']*len(saap_errs)
    
    ds_plot_df = pd.DataFrame(zip(ds_list, main_pep_errs+saap_errs, main_str_list+saap_str_list), columns=['Dataset', 'Mass error (ppm)', 'Peptide type'])
    plot_df_list.append(ds_plot_df)
plot_df = pd.concat(plot_df_list)

fig,ax = plt.subplots(figsize=(3,4))
sns.boxplot(data=plot_df, y='Dataset', hue='Peptide type', x='Mass error (ppm)', linewidth=1, fliersize=0, palette='Greys')
plt.xlabel('Mass error (ppm)', fontsize=15)
ax.tick_params('both', labelsize=14)
plt.ylabel('')
plt.xlim([-2.5,2.5])
plt.legend(loc='lower left', fontsize=14, bbox_to_anchor=(-0.2,0.97), frameon=False, ncol=2, handletextpad=0.1, columnspacing=0.8)
plt.savefig(outdir+'Mass_error_boxplots.pdf', bbox_inches='tight')


### Figure 1f. Number of fragments covering site of substitution

In [ ]:
def n_frags_over_MTP(frag_match, mtp, sub_idx):
    """
    Input: fragment matches for peptide from msms.txt (MQ output file), peptide sequence, index of AAS on sequence
    Output: number of fragment ions covering site of AAS
    """
    count = 0
    for f, frag in enumerate(frag_match):
        mtp_frag=0
        if ('NH3' not in frag) and ('H2O' not in frag) and ('(' not in frag) and ('a' not in frag):
            if 'b' in frag:
                frag_start = 0
                frag_end = int(frag[1:])
                frag_seq = mtp[frag_start:frag_end]
                if frag_end>sub_idx:
                    mtp_frag = 1
            elif 'y' in frag:
                frag_start = -int(frag[1:])
                frag_seq = mtp[frag_start:]
                if len(mtp)+frag_start <= sub_idx:
                    mtp_frag=1
            count+=mtp_frag
        
    return(count)


In [ ]:
# add fragment evidence data to Validation_MTP_dict.p for each dataset

for ds in datasets:
    print(ds)
    data_dir = data_dir_list[datasets.index(ds)]
    mtp_dict = pickle.load(open(data_dir+'Validated_MTP_dict.p','rb'))
    val_evidence_dict = pickle.load(open(data_dir+'Validation_search_evidence_dict.p', 'rb'))
    MQ_dir = data_dir+'MQ_output/Validation_search'
    samples = samples_list[datasets.index(ds)]

    for s in samples:
        print(s)
       # if s not in done:
        ev = val_evidence_dict[s]
        with open(MQ_dir+'/'+s+'/txt/msms.txt') as msms_file:
            msms = pd.read_csv(msms_file, '\t', low_memory=False)
            msms_vals = msms.values

            mtp_dict[s]['fragment_evidence'] = {}
            for k,v in mtp_dict[s]['aa subs'].items():
                seq = mtp_dict[s]['mistranslated sequence'][k]
                bp = mtp_dict[s]['DP Base Sequence'][k]
                sub_idx = [i for i,x in enumerate(bp) if seq[i]!=x][0]

                ev_idx = mtp_dict[s]['idx_val_evidence'][k]
                mtp_dict[s]['fragment_evidence'][k] = 0
                for idx in ev_idx:
                    row = ev.iloc[idx,:]
                    raw_file = row['Raw file']
                    scan = row['MS/MS scan number']
                    scan_row_idx = [i for i in range(len(msms_vals)) if (msms_vals[i][0]==raw_file) and (msms_vals[i][1]==scan)]
                    #scan_row = msms.loc[(msms['Raw file']==raw_file) & (msms['Scan number']==scan),:]
                    if len(scan_row_idx)>0:
                        scan_row = msms.iloc[scan_row_idx[0]]
                        frag_match = scan_row['Matches'].split(';')
                        count_frags = n_frags_over_MTP(frag_match, seq, sub_idx)
                        if count_frags>mtp_dict[s]['fragment_evidence'][k]:
                            mtp_dict[s]['fragment_evidence'][k] = count_frags
        msms_file.close()
    pickle.dump(mtp_dict, open(data_dir+'Validated_MTP_dict.p','wb'))

In [ ]:
# generate dataframe with fragment ion data
cats = ['0', '1','2-5','6-10','>10']
scan_counts = [0,0,0,0,0]
pep_counts = [0,0,0,0,0]

for ds in datasets:
    print(ds)
    data_dir = data_dir_list[datasets.index(ds)]
    samples = samples_list[datasets.index(ds)]
    mtp_dict = pickle.load(open(data_dir+'Validated_MTP_dict.p','rb'))

    for s in samples:
        s_dict = mtp_dict[s]
        for i,n in s_dict['fragment_evidence'].items():
            if n ==0:
                scan_counts[0]+=1
            elif n ==1:
                scan_counts[1]+=1
            elif (n >1) and (n <=5):
                scan_counts[2]+=1
            elif (n>5) and (n <=10):
                scan_counts[3]+=1
            else:
                scan_counts[4]+=1

plot_df = pd.DataFrame(zip(cats,scan_counts), columns=['Bin', 'Count'])
plot_df['Used'] = ['Yes' if row['Bin'] not in ['0','1'] else 'No' for i,row in plot_df.iterrows()]
plot_df.to_excel(outdir+'by_fragments_per_saap_4barplot_allDS.xlsx')

In [ ]:
#plot_df = pd.read_excel(outdir+'by_fragments_per_saap_4barplot_allDS.xlsx', index_col=0)

In [ ]:
# plot n fragments barplots

sns.set_style('whitegrid')
fig,ax = plt.subplots(figsize=(4,4))
sns.barplot(data = plot_df, x='Bin', y='Count', hue='Used',dodge=False, palette=['#aaaaaa',colors[0]])   
ax.tick_params('both',labelsize=14)

plt.ylabel('# peptides', fontsize=15)
handles, labels = ax.get_legend_handles_labels()
plt.legend(handles=handles,loc='upper left', labels=['Excluded', 'Included'], fontsize=13)
plt.xlabel('# peptide fragments supporting\nsubstitution site', fontsize=15)
plt.savefig(outdir+'fragment_ion_evidence_barplot_allDS.pdf', bbox_inches='tight')

### Fig 1e. % genome homology/% validated

In [ ]:
def get_n_unq_seqs(data_dir, s):
    """ input: dataset directory, sample
        output: number of unique candidate substituted peptides
    """
    hc_mtp_dict = pickle.load(open(data_dir+'qMTP_dict.p', 'rb'))
    s_seqs = [x for y in hc_mtp_dict[s]['mistranslated sequence'].values() for x in y]
    s_seqs = list(set(s_seqs))
    n_seqs = len(s_seqs)
    return(n_seqs)

def get_n_unq_val(data_dir, s):
    """ input: dataset directory, sample
        output: number of unique validated substituted peptides
    """
    val_mtp_dict = pickle.load(open(data_dir+'Ion_validated_MTP_dict.p', 'rb'))
    s_seqs = list(val_mtp_dict[s]['mistranslated sequence'].values())
    s_seqs = list(set(s_seqs))
    n_seqs = len(s_seqs)
    return(n_seqs)

In [ ]:
# get dataframe with number of unique substituted peptides, candidate and validated
rows = []
cols = ['Dataset', 'N SAAP sequences', 'SAAP sequence type', 'TMT set']

for i,ds in enumerate(datasets):
    print(ds)
    samples = samples_list[i]
    data_dir = data_dir_list[i]
    for s in samples:
        n_seqs = get_n_unq_seqs(data_dir, s)
        n_val_seqs = get_n_unq_val(data_dir, s)
        rows.append([ds, n_seqs, 'Candidate', s])
        rows.append([ds, n_val_seqs, 'Validated', s])
plt_df = pd.DataFrame(rows, columns=cols)

# get dataframe with percentage of peptides validated and percentage of peptides with no genome homology (from ds_filter_dict.p generated above)
pcnt_rows = []
for i,ds in enumerate(datasets):
    print(ds)
    data_dir = data_dir_list[i]
    samples = samples_list[i]
    ds_plt_df = plt_df.loc[plt_df['Dataset']==ds,:]
    ds_filter_dict = filter_dict[ds]
    for j,s in enumerate(samples):
        s_df = ds_plt_df.loc[ds_plt_df['TMT set']==s,:]
        n_cand = s_df.loc[s_df['AASP sequence type']=='Candidate', 'N AASP sequences'].values[0]
        n_val = s_df.loc[s_df['AASP sequence type']=='Validated', 'N AASP sequences'].values[0]
        val_pcnt = 100*(n_val/n_cand)
        n_aas = ds_filter_dict['AAS'][j]
        n_nonhom = ds_filter_dict['Non-homologous'][j]
        hom_pcnt = 100*(n_aas - n_nonhom)/n_aas
        pcnt_rows.append([ds,s, val_pcnt, 'Validated SAAP'])
        pcnt_rows.append([ds,s, hom_pcnt, '6-frame + UTR translation'])
pcnt_df = pd.DataFrame(pcnt_rows, columns=['Dataset', 'TMT set', '% peptides', '% type'])
nonrectum_row = [i for i,row in pcnt_df.iterrows() if (row['TMT set']!='rectum') and (row['TMT set']!='bonemarrow')] # too little data, removed these tissues from analysis
pcnt_df = pcnt_df.loc[nonrectum_row]
pcnt_df.to_excel(outdir+'percent_validated_genomehomol_SAAP.xlsx')
#pcnt_df = pd.read_excel(outdir+'percent_validated_genomehomol_SAAP.xlsx', index_col=0)

# plot figure 1e
sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize=(3,4.5))
sns.boxplot(data=pcnt_df, x='Dataset', y='% peptides', linewidth=0.8, hue='% type', fliersize=0.8, dodge=False)
plt.ylim([-0.5,52])
plt.yticks(fontsize=14)
plt.xticks(rotation=45, fontsize=14)
plt.xlabel('')
plt.ylabel('% candidate SAAP', fontsize=15)
plt.legend(bbox_to_anchor=(-0.2,0.98), frameon=False, loc='lower left', fontsize=13, markerscale=0.5, handletextpad=0.2)
plt.savefig(outdir+'percent_validated_genomehom_allDS_boxplot.pdf', bbox_inches='tight')



### Fig 1g. Prosit rescoring

In [ ]:
# read in results from prosit rescoring
rescoring_data = pd.read_csv(rescoring_data_dir+'Rescoring.csv')

# plot rescoring data
MTP_only = rescoring_data
sns.set(style="whitegrid")
vect1 = MTP_only['log10PEP_A'].values
vect2 = MTP_only['log10PEP_P'].values
kde = sp.stats.gaussian_kde(np.vstack([vect1, vect2]))
densities = kde(np.vstack([vect1, vect2]))

plt.figure(figsize=(4,4))
plt.xlabel('-log10(PEP) Andromeda')
plt.ylabel('-log10(PEP) Prosit')

plt.scatter(
    x=MTP_only['log10PEP_A'],
    y=MTP_only['log10PEP_P'],
    c=densities,  
    cmap='viridis', s = 25, alpha=0.75, edgecolor='w', linewidth=0.3)

plt.plot(
    [0, 15], [0, 15],  
    color='red',           
    linewidth=2 
)

plt.xlim(0, 15)
plt.ylim(0, 15)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xticks([0,2,4,6,8,10,12,14])
plt.yticks([0,2,4,6,8,10,12,14])
plt.xlabel('-log$_{10}$(error prob.), identification', fontsize=15)
plt.ylabel('-log$_{10}$(error prob.), rescoring', fontsize=15)

plt.savefig(outdir+'Prosit_Rescoring.png', dpi=300, bbox_inches='tight')

### Figure 1h. Detection of SAAP in multiple digests

Identify the same substitutions in different protease digest from the tonsil data (part of healthy tissues dataset).

Two new dictionaries generated here, 'Tonsil_SAAP_dict.p' and 'Tonsil_BP_dict.p' are provided in the shared google drive folder. 

In [ ]:
# set directories and read in files needed to analyze the tonsil data from multiple digests

aa_subs_dir = wang_data_dir + 'AA_subs_pipeline/'
pub_dir = wang_data_dir+'Publication/'
enzymes = ['Trypsin', 'LysC','ArgC', 'GluC','Chymotrypsin']

# need the fasta for protein sequences - to match the substitutions across different peptides 
fasta = open(wang_data_dir+'databases/tonsil_MTP.fasta', 'r').readlines()
fasta = [x for x in fasta if x!='\n' and len(x)>0]
fasta_headers = [x for x in fasta if x[0]=='>']
fasta_header_lines = [i for i,x in enumerate(fasta) if x[0]=='>']
fasta_seqs = [fasta[header_idx+1] for header_idx in fasta_header_lines]


mtp_quant_dict = pickle.load(open(aa_subs_dir+'tonsil_MTP_quant_dict_newvalsearch.p', 'rb'))
val_mtp_dict = pickle.load(aa_subs_dir+ open('tonsil_Ion_validated_MTP_dict_newvalsearch.p', 'rb'))
val_ev_dict = pickle.load(open('tonsil_Validation_search_evidence_dict.p', 'rb'))
trypsin_dict = val_mtp_dict['Trypsin']

In [ ]:
def get_idx_range_of_pep_in_prot(pep_seq, prot_seq):
    """
    function to get the positions of a peptide sequence in a protein sequence
    """
    start_idx = [x.start() for x in re.finditer(pep_seq, prot_seq)][0]
    end_idx = [x.end() for x in re.finditer(pep_seq, prot_seq)][0]
    
    idx_range = list(range(start_idx,end_idx))
    return(idx_range)

def get_aa_idx_in_prot(pep_range, aa_idx):
    """
    function to get the position of an AAS in a protein, given the peptide sequence and the index of the AAS in the peptide
    """
    saap_idx = pep_range[0]+aa_idx
    return(saap_idx)

In [ ]:
# create dictionaries for SAAP and BP with protein index position of the AAS annotated tonsil_saap_dict = {}

tonsil_saap_dict = {}
for s, s_dict in mtp_quant_dict.items():
    saap = s_dict['MTP_seq']
    bp = s_dict['BP_seq']
    aas = s_dict['aa_sub']
    aas_idx = s_dict['sub_index'][0]
    saap_abund_dict = {k:v for k,v in s_dict['MTP_PrecInt'].items() if ~np.isnan(s_dict['Prec_ratio'][k])}
    bp_abund_dict = {k:v for k,v in s_dict['BP_PrecInt'].items() if ~np.isnan(s_dict['Prec_ratio'][k])}
    raas_dict = {k:v for k,v in s_dict['Prec_ratio'].items() if ~np.isnan(s_dict['Prec_ratio'][k])}
    digests = list(raas_dict.keys())
    
    tonsil_saap_dict[s] = {'SAAP':saap, 'BP':bp, 'AAS':aas, 'AAS_index':aas_idx, 
                           'SAAP abundance':saap_abund_dict, 'BP abundance':bp_abund_dict, 'RAAS':raas_dict,
                          'Digests':digests, 'Peptide IDs':{}}

    prots_w_saap = [i for i,seq in enumerate(fasta_seqs) if re.search(saap, seq)]
    for i,prot_idx in enumerate(prots_w_saap):
        seq = fasta_seqs[prot_idx]
        saap_idx_range = get_idx_range_of_pep_in_prot(saap, seq)
        saap_prot_idx = get_aa_idx_in_prot(saap_idx_range, aas_idx)
        tonsil_saap_dict[s]['Peptide IDs'][i] = {'saap_prot_fasta_idx':prot_idx, 'saap_prot_range':saap_idx_range, 'saap_prot_idx':saap_prot_idx}

pickle.dump(tonsil_saap_dict, open('Tonsil_SAAP_dict.p','wb'))


tonsil_bp_dict = {}
for s, s_dict in mtp_quant_dict.items():
    if s%100==0:
        print(s)
    bp = s_dict['BP_seq']
    rand_idx = random.sample(list(range(len(bp))), 1)[0]
    bp_abund_dict = {k:v for k,v in s_dict['BP_PrecInt'].items() if ~np.isnan(s_dict['Prec_ratio'][k])}
    digests = list(bp_abund_dict.keys())
        
    tonsil_bp_dict[s] = {'BP':bp, 'random_index':rand_idx, 'BP abundance':bp_abund_dict, 
                         'Digests':digests, 'Peptide IDs':{}}
    
    prots_w_bp = [i for i,seq in enumerate(fasta_seqs) if re.search(bp, seq)]
    
    for i,prot_idx in enumerate(prots_w_bp):
        seq = fasta_seqs[prot_idx]
        bp_idx_range = get_idx_range_of_pep_in_prot(bp, seq)
        rand_prot_idx = get_aa_idx_in_prot(bp_idx_range, rand_idx)
        
        tonsil_bp_dict[s]['Peptide IDs'][i] = {'prot_fasta_idx':prot_idx, 'bp_prot_range':bp_idx_range, 'random_prot_idx':rand_prot_idx}
pickle.dump(tonsil_bp_dict, open('Tonsil_BP_dict.p','wb'))

#### get proteins and peptide range for all main peptides
This chunk was computationally intensive and was run on HPC for each enzyme separately and then combined. But the code is here if want to run locally. The output is also shared in the google drive. 

In [ ]:
all_enz_mainpep_prot_idx = {}

for enz in enzymes:
    print(enz)
    all_enz_mainpep_prot_idx[enz] = {}
    ev_data = val_ev_dict[enz]
    ev_data = ev_data.loc[ev_data['PEP']<=0.01]
    set_peps = list(set(ev_data['Sequence'].values))
    print(len(set_peps))

    for p,pep in enumerate(set_peps):
        if p%1000==0:
            print(p)

        all_enz_mainpep_prot_idx[enz][p] = {'Peptide':pep, 'Peptide IDs':{}}    
        prots_w_pep = [i for i,seq in enumerate(fasta_seqs) if re.search(pep, seq)]    

        for i,prot_idx in enumerate(prots_w_pep):
            seq = fasta_seqs[prot_idx]
            pep_idx_range = get_idx_range_of_pep_in_prot(pep, seq)
            rand_prot_idx = random.sample(pep_idx_range,1)

            all_enz_mainpep_prot_idx[enz][p]['Peptide IDs'][i] = {'prot_fasta_idx':prot_idx, 'pep_prot_range':pep_idx_range, 'random_prot_idx':rand_prot_idx}

pickle.dump(all_enz_mainpep_prot_idx, open(aa_subs_dir+'all_tonsil_mainpeps_in_prots_newval_search.p','wb'))

In [ ]:
# annotate tonsil_saap_dict and tonsil_bp_dict 
# with information about if the same substitution site (or random position for bp) is found in other digests

tonsil_same_saap_mult_dig = {k:v for k,v in tonsil_saap_dict.items() if len(v['Digests'])>1}
for s, sdict in tonsil_saap_dict.items():
    if s%100==0:
        print(s)
    s_prot_dict = sdict['Peptide IDs']
    saap = sdict['SAAP']
    saap_idx = sdict['AAS_index'] # index of AAS in peptide

    digests = sdict['Digests']
    all_digests = copy.deepcopy(digests) 
    alt_peps = {}
    for enz in enzymes:
        if enz not in digests:
            enz_dict = all_enz_mainpep_prot_idx[enz]
            for p,prot_dict in s_prot_dict.items():
                prot_idx = prot_dict['saap_prot_fasta_idx']
                aas_idx = prot_dict['saap_prot_idx'] # index of AAS in protein
                
                main_peps = []
                for k,v in enz_dict.items():
                    pep = v['Peptide']
                    pep_ids = v['Peptide IDs']
                    for i in pep_ids:
                        pep_prot_idx = pep_ids[i]['prot_fasta_idx']
                        pep_range = pep_ids[i]['pep_prot_range']
                        if (pep_prot_idx==prot_idx) and (aas_idx in pep_range):
                            if pep[pep_range.index(aas_idx)]==saap[saap_idx]:
                                main_peps.append(k)
                                alt_peps[enz] = pep
                                break
                
                if len(main_peps)>0:
                    all_digests.append(enz)
                    break
    sdict['Same_seq_digests'] = digests
    sdict['All_digests'] = all_digests
    sdict['Alternative_peptides'] = alt_peps
pickle.dump(tonsil_saap_dict, open(aa_subs_dir+'Tonsil_SAAP_dict.p','wb'))


tonsil_same_bp_mult_dig = {k:v for k,v in tonsil_bp_dict.items() if len(v['Digests'])>1}
for s, sdict in tonsil_bp_dict.items():
    if s%100==0:
        print(s)
    s_prot_dict = sdict['Peptide IDs']
    bp = sdict['BP']
    bp_rand_idx = sdict['random_index'] # index of random position on BP
    digests = sdict['Digests']
    all_digests = copy.deepcopy(digests)

    for enz in enzymes:
        if enz not in digests:
            enz_dict = all_enz_mainpep_prot_idx[enz]

            for p,prot_dict in s_prot_dict.items():
                prot_idx = prot_dict['prot_fasta_idx']
                rand_idx = prot_dict['random_prot_idx'] # index of random position on BP in protein

                main_peps = []
                for k,v in enz_dict.items():
                    pep = v['Peptide']
                    pep_ids = v['Peptide IDs']
                    for i in pep_ids:
                        pep_prot_idx = pep_ids[i]['prot_fasta_idx']
                        pep_range = pep_ids[i]['pep_prot_range']
                        if (pep_prot_idx==prot_idx) and (rand_idx in pep_range):
                            if pep[pep_range.index(rand_idx)]==bp[bp_rand_idx]:
                                main_peps.append(k)
                                break
                if len(main_peps)>0:
                    all_digests.append(enz)
                    break
    sdict['Same_seq_digests'] = digests
    sdict['All_digests'] = all_digests
pickle.dump(tonsil_bp_dict, open(aa_subs_dir+'Tonsil_BP_dict_Dec2023_newvalsearch.p','wb'))

In [ ]:
# extract metrics for plot from tonsil_saap_dict and tonsil_bp_dict

saap_1dig = {k:v for k,v in tonsil_saap_dict.items() if len(v['All_digests'])==1}
saap_2dig = {k:v for k,v in tonsil_saap_dict.items() if len(v['All_digests'])==2}
saap_2dig_sameseq = {k:v for k,v in tonsil_saap_dict.items() if len(v['All_digests'])==2 and len(v['Digests'])==2}
saap_2dig_diffseq = {k:v for k,v in tonsil_saap_dict.items() if len(v['All_digests'])==2 and len(v['Digests'])!=2}
saap_3dig = {k:v for k,v in tonsil_saap_dict.items() if len(v['All_digests'])==3}
saap_4dig = {k:v for k,v in tonsil_saap_dict.items() if len(v['All_digests'])==4}
saap_5dig = {k:v for k,v in tonsil_saap_dict.items() if len(v['All_digests'])==5}
saap_gr3dig = {k:v for k,v in tonsil_saap_dict.items() if len(v['All_digests'])>=3}
total = len(tonsil_saap_dict)

saap_thresh = 1e9
e9_saap_1dig = {k:v for k,v in tonsil_saap_dict.items() if len(v['All_digests'])==1 and np.nanmean(list(v['SAAP abundance'].values()))>=saap_thresh}
e9_saap_2dig_sameseq = {k:v for k,v in tonsil_saap_dict.items() if len(v['All_digests'])==2 and len(v['Digests'])==2 and np.nanmean(list(v['SAAP abundance'].values()))>=saap_thresh}
e9_saap_2dig_diffseq = {k:v for k,v in tonsil_saap_dict.items() if len(v['All_digests'])==2 and len(v['Digests'])!=2 and np.nanmean(list(v['SAAP abundance'].values()))>=saap_thresh}
e9_saap_gr3dig = {k:v for k,v in tonsil_saap_dict.items() if len(v['All_digests'])>=3 and np.nanmean(list(v['SAAP abundance'].values()))>=saap_thresh}
e9_total = len({k:v for k,v in tonsil_saap_dict.items() if np.nanmean(list(v['SAAP abundance'].values()))>=saap_thresh})

saap_thresh = 1e8
e8_saap_1dig = {k:v for k,v in tonsil_saap_dict.items() if len(v['All_digests'])==1 and np.nanmean(list(v['SAAP abundance'].values()))>=saap_thresh}
e8_saap_2dig_sameseq = {k:v for k,v in tonsil_saap_dict.items() if len(v['All_digests'])==2 and len(v['Digests'])==2 and np.nanmean(list(v['SAAP abundance'].values()))>=saap_thresh}
e8_saap_2dig_diffseq = {k:v for k,v in tonsil_saap_dict.items() if len(v['All_digests'])==2 and len(v['Digests'])!=2 and np.nanmean(list(v['SAAP abundance'].values()))>=saap_thresh}
e8_saap_gr3dig = {k:v for k,v in tonsil_saap_dict.items() if len(v['All_digests'])>=3 and np.nanmean(list(v['SAAP abundance'].values()))>=saap_thresh}
e8_total = len({k:v for k,v in tonsil_saap_dict.items() if np.nanmean(list(v['SAAP abundance'].values()))>=saap_thresh})

saap_thresh = 1e7
e7_saap_1dig = {k:v for k,v in tonsil_saap_dict.items() if len(v['All_digests'])==1 and np.nanmean(list(v['SAAP abundance'].values()))>=saap_thresh}
e7_saap_2dig_sameseq = {k:v for k,v in tonsil_saap_dict.items() if len(v['All_digests'])==2 and len(v['Digests'])==2 and np.nanmean(list(v['SAAP abundance'].values()))>=saap_thresh}
e7_saap_2dig_diffseq = {k:v for k,v in tonsil_saap_dict.items() if len(v['All_digests'])==2 and len(v['Digests'])!=2 and np.nanmean(list(v['SAAP abundance'].values()))>=saap_thresh}
e7_saap_gr3dig = {k:v for k,v in tonsil_saap_dict.items() if len(v['All_digests'])>=3 and np.nanmean(list(v['SAAP abundance'].values()))>=saap_thresh}
e7_total = len({k:v for k,v in tonsil_saap_dict.items() if np.nanmean(list(v['SAAP abundance'].values()))>=saap_thresh})

bp_1dig = {k:v for k,v in tonsil_bp_dict.items() if len(v['All_digests'])==1}
bp_2dig_sameseq = {k:v for k,v in tonsil_bp_dict.items() if len(v['All_digests'])==2 and len(v['Digests'])==2}
bp_2dig_diffseq = {k:v for k,v in tonsil_bp_dict.items() if len(v['All_digests'])==2 and len(v['Digests'])!=2}
bp_gr3dig = {k:v for k,v in tonsil_bp_dict.items() if len(v['All_digests'])>=3}
bp_total = len(tonsil_bp_dict)

bp_thresh = 1e9
e9_bp_1dig = {k:v for k,v in tonsil_bp_dict.items() if len(v['All_digests'])==1 and np.nanmean(list(v['BP abundance'].values()))<=bp_thresh}
e9_bp_2dig_sameseq = {k:v for k,v in tonsil_bp_dict.items() if len(v['All_digests'])==2 and len(v['Digests'])==2 and np.nanmean(list(v['BP abundance'].values()))<=bp_thresh}
e9_bp_2dig_diffseq = {k:v for k,v in tonsil_bp_dict.items() if len(v['All_digests'])==2 and len(v['Digests'])!=2 and np.nanmean(list(v['BP abundance'].values()))<=bp_thresh}
e9_bp_gr3dig = {k:v for k,v in tonsil_bp_dict.items() if len(v['All_digests'])>=3 and np.nanmean(list(v['BP abundance'].values()))<=bp_thresh}
e9_bp_total = len({k:v for k,v in tonsil_bp_dict.items() if np.nanmean(list(v['BP abundance'].values()))<=bp_thresh})

all_saap_pcnt_1dig = 100*len(saap_1dig)/total 
all_saap_pcnt_2dig_sameseq = 100*len(saap_2dig_sameseq)/total
all_saap_pcnt_2dig_diffseq = 100*len(saap_2dig_diffseq)/total
all_saap_pcnt_3dig = 100*len(saap_gr3dig)/total

e9_saap_pcnt_1dig = 100*len(e9_saap_1dig)/e9_total 
e9_saap_pcnt_2dig_sameseq = 100*len(e9_saap_2dig_sameseq)/e9_total
e9_saap_pcnt_2dig_diffseq = 100*len(e9_saap_2dig_diffseq)/e9_total
e9_saap_pcnt_3dig = 100*len(e9_saap_gr3dig)/e9_total

e8_saap_pcnt_1dig = 100*len(e8_saap_1dig)/e8_total 
e8_saap_pcnt_2dig_sameseq = 100*len(e8_saap_2dig_sameseq)/e8_total
e8_saap_pcnt_2dig_diffseq = 100*len(e8_saap_2dig_diffseq)/e8_total
e8_saap_pcnt_3dig = 100*len(e8_saap_gr3dig)/e8_total

e7_saap_pcnt_1dig = 100*len(e7_saap_1dig)/e7_total 
e7_saap_pcnt_2dig_sameseq = 100*len(e7_saap_2dig_sameseq)/e7_total
e7_saap_pcnt_2dig_diffseq = 100*len(e7_saap_2dig_diffseq)/e7_total
e7_saap_pcnt_3dig = 100*len(e7_saap_gr3dig)/e7_total

e9_bp_pcnt_1dig = 100*len(e9_bp_1dig)/e9_bp_total 
e9_bp_pcnt_2dig_sameseq = 100*len(e9_bp_2dig_sameseq)/e9_bp_total
e9_bp_pcnt_2dig_diffseq = 100*len(e9_bp_2dig_diffseq)/e9_bp_total
e9_bp_pcnt_3dig = 100*len(e9_bp_gr3dig)/e9_bp_total

bp_pcnt_1dig = 100*len(bp_1dig)/bp_total 
bp_pcnt_2dig_sameseq = 100*len(bp_2dig_sameseq)/bp_total
bp_pcnt_2dig_diffseq = 100*len(bp_2dig_diffseq)/bp_total
bp_pcnt_3dig = 100*len(bp_gr3dig)/bp_total

In [ ]:
# create stacked barplot showing the % of SAAP in multiple digests

bar_df = pd.DataFrame({'1 digest':[bp_pcnt_1dig, e9_bp_pcnt_1dig, e9_saap_pcnt_1dig, e8_saap_pcnt_1dig, e7_saap_pcnt_1dig, all_saap_pcnt_1dig],
                     '2 digests, same sequence':[bp_pcnt_2dig_sameseq, e9_bp_pcnt_2dig_sameseq, e9_saap_pcnt_2dig_sameseq, e8_saap_pcnt_2dig_sameseq, e7_saap_pcnt_2dig_sameseq, all_saap_pcnt_2dig_sameseq],
                     '2 digests, different sequence':[bp_pcnt_2dig_diffseq, e9_bp_pcnt_2dig_diffseq, e9_saap_pcnt_2dig_diffseq, e8_saap_pcnt_2dig_diffseq, e7_saap_pcnt_2dig_diffseq, all_saap_pcnt_2dig_diffseq],
                     '3+ digests':[bp_pcnt_3dig, e9_bp_pcnt_3dig, e9_saap_pcnt_3dig, e8_saap_pcnt_3dig, e7_saap_pcnt_3dig, all_saap_pcnt_3dig]},
                      index=['BP', 'BP$\leq$10$^{9}$','SAAP>10$^{9}$','SAAP>10$^{8}$', 'SAAP>10$^{7}$', 'All SAAP'])

saap_bar_df = bar_df.loc[[i for i in bar_df.index if 'SAAP' in i]]
bp_bar_df = bar_df.loc[[i for i in bar_df.index if 'BP' in i]]

sns.set_style('whitegrid')
fig,ax = plt.subplots(figsize=(4,6), sharex=True)
palette = sns.color_palette('rocket_r', n_colors=10)
palette = [colors[0], palette[0], palette[3], palette[6], palette[9]]

bar_df.plot(kind='barh',stacked=True, figsize=(4,3), color=palette, linewidth=0.1, ax=ax)
ax.set_yticks([])
ax.legend().remove()
ax.set_xlabel('% peptides', fontsize=15)
ax.tick_params('x', labelsize=13)
plt.legend(bbox_to_anchor=(-0.5,1), ncol=2, frameon=False, loc='lower left', handletextpad=0.1, fontsize=11)

plt.savefig(outdir+'BP_SAAP_%multdigests_bar_binnedSAAP_horz_newvalsearch_bpthresh.pdf', bbox_inches='tight')

In [ ]:
# create ridgeplot with the abundance distributions of the peptides in each bar of the barplot 

ridge_rows = []
ridge_cols = ['log$_{10}$(Abundance)','Group']

for s,sdict in tonsil_saap_dict.items():
    saap_abund = np.log10(np.mean(list(sdict['SAAP abundance'].values())))
    bp_abund = np.log10(np.mean(list(sdict['BP abundance'].values())))
    
    if saap_abund!=0 and bp_abund!=0:
        if saap_abund<=7:
            ridge_rows.append([saap_abund, 'All SAAP'])
        elif saap_abund<=8:
            ridge_rows.append([saap_abund, 'All SAAP'])
            ridge_rows.append([saap_abund, 'Log$_{10}$SAAP>7'])
        elif saap_abund<=9:
            ridge_rows.append([saap_abund, 'All SAAP'])
            ridge_rows.append([saap_abund, 'Log$_{10}$SAAP>7'])
            ridge_rows.append([saap_abund, 'Log$_{10}$SAAP>8'])
            ridge_rows.append([saap_abund, 'Log$_{10}$SAAP>9'])
        ridge_rows.append([bp_abund, 'BP'])
        if bp_abund<=9:
            ridge_rows.append([bp_abund, 'Log$_{10}$BP<9'])

ridge_df = pd.DataFrame(ridge_rows, columns=ridge_cols)
ridge_df.replace(-np.inf, np.nan, inplace=True)
ridge_df.replace(np.inf, np.nan, inplace=True)
ridge_df.dropna(how='any', axis=0, inplace=True)

bp_ridge_df = ridge_df.loc[ridge_df['Group']=='BP']
saap_ridge_df = ridge_df.loc[ridge_df['Group']!='BP']

sns.set(font_scale=1.2)
sns.set_style('whitegrid')
g = sns.FacetGrid(ridge_df, row='Group', aspect=3, height=0.6, sharex=True,sharey=False, gridspec_kws={'hspace':-0.05},
                 row_order=['All SAAP', 'Log$_{10}$SAAP>7', 'Log$_{10}$SAAP>8', 'Log$_{10}$SAAP>9', 'Log$_{10}$BP<9', 'BP'])
g.set_titles("")
g.set(yticks=[])
g.set(xticks=[7,9,11])
g.map_dataframe(sns.kdeplot, x='log$_{10}$(Abundance)', fill=True, color='#AAAAAA', alpha=0.5)
g.savefig(outdir+'BP_SAAP_abund_ridgeplot_nevalsearch.pdf', bbox_inches='tight')

### Extended Data Figure 1g. PEP post-FDR correction

In [ ]:
# plot the p-values determined by MaxQuant and the q-values determined from only the substituted peptides

plot_rows = []
plot_cols = ['Dataset', 'TMT/Tissue', 'q-value', 'PEP']
for ds in datasets:
    data_dir = data_dir_list[datasets.index(ds)]
    samples = samples_list[datasets.index(ds)]
    qmtp_dict = pickle.load(open(data_dir+'qMTP_dict.p', 'rb'))
    for s in samples:
        s_dict = qmtp_dict[s]
        for k,v in s_dict['Posterior subs probability'].items():
            plot_rows.append([ds, s, 1-v[0], s_dict['q-value'][k][0]])
plot_df = pd.DataFrame(plot_rows, columns=plot_cols)

plot_df['-log q'] = plot_df.apply(lambda x: -np.log10(x['q-value']),axis=1)
plot_df['-log PEP'] = plot_df.apply(lambda x: -np.log10(x['PEP']),axis=1)

fig,ax = plt.subplots(figsize=(2.5,2.5))
sns.ecdfplot(plot_df, x='-log q', label='log$_{10}$(q)')
sns.ecdfplot(plot_df, x='-log PEP', label='log$_{10}$(PEP)')
plt.ylabel('Cumulative density', fontsize=14)
plt.xlabel('-log$_{10}$(FDR)', fontsize=14)
ax.tick_params('both', labelsize=13)
plt.plot((2,2), plt.ylim(), '--r')
plt.legend(loc='lower right', fontsize=12, handletextpad=0.1)
plt.savefig(outdir+'qvalue_CumDensity_plot.pdf', bbox_inches='tight')

### Extended Data Figure 1 c,d. Trancript coverage stats

In [ ]:
def find_all_matches(file_path, regex_pattern):
    """Finds all instances of a regular expression in a text file.

    Args:
    file_path: Path to the text file.
    regex_pattern: The regular expression pattern to search for.

    Returns:
    A list of all matching strings.
    """

    matches = []
    with open(file_path, 'r') as file:
        for line in file:
            for match in re.finditer(regex_pattern, line):
                matches.append(match.group())
    return(matches)


def get_transcrip_coverage(tmap_path):
    """
    Returns the fraction of a transcriptome sequence that was contained a read.
    Input is the path to a .gft.tmap file, generated by the custom protein database pipeline
    Output is the a list of the fraction covered of each sequence in the file provided
    
    """
    
    lines = zip_ref.open(tmap_path, 'r').readlines()
    lines = [line.decode('utf-8') for line in lines]
    lines = [x.split('\n')[0] for x in lines]
    cols = [x for x in lines[0].split('\t') if len(x)>0]
    mtx = [x.split('\t') for x in lines[1:]]
    df = pd.DataFrame(mtx, columns=cols)
    t_len = df['len'].astype(int).to_list()
    match_len = df['ref_match_len'].to_list()
    match_len = [int(x) if x!='-' else 0 for x in match_len]
    frac_cov = [match_len[i]/t_len[i] for i in range(len(match_len))]
    frac_cov = [x for x in frac_cov if x<=1]
    return(frac_cov)

In [ ]:
# get a dictionary with each sample in each dataset and a list of the transcript sequence covereage in each samples database
seq_cov_frac = {ds:{} for ds in datasets}
for ds in datasets:
    print(ds)
    ds_dir = proj_dir_list[datasets.index(ds)]
    rnaseq_dir = ds_dir+'RNAseq_data/'
    zip_files = glob(rnaseq_dir+'*files2transfer.zip')

    print(len(zip_files))
    for i,zipdir in enumerate(zip_files):
        sample = zipdir.split('/')[-1].split('.')[0]
        file2read = 'files2transfer/'+sample+'.gffcmp.'+sample+'.stringtie.gtf.tmap'
        try:
            with zipfile.ZipFile(zipdir, 'r') as zip_ref:
                try:
                    frac_cov = get_transcrip_coverage(file2read)
                    seq_cov_frac[ds][sample] = frac_cov
                except:
                    print('file not found')
        except:
            print('notzipfile')

In [ ]:
# extract and plot overall distribution of the transcript sequence coverage per sample         
frac_cov = [list(seq_cov_frac[ds].values()) for ds in datasets]
frac_cov = [x for y in frac_cov for x in y]
frac_cov = [100*x for y in frac_cov]

n_samples = np.sum([len(seq_cov_frac[ds].keys()) for ds in datasets])
weights=[1/n_samples]*len(frac_cov)
plot_df = pd.DataFrame(zip(frac_cov, weights), columns=['Transcript coverage', 'Weight'])

sns.set_style('whitegrid')
fig,ax = plt.subplots(figsize=(3,3))
sns.histplot(data=plot_df, x='Transcript coverage', weights='Weight', color='#AAAAAA', linewidth=0, )
mean = np.mean(frac_cov)
plt.plot((mean, mean), ax.get_ylim(), '--r')
ax.annotate('mean$=$'+str(np.round(mean,2))+'%', xy=(40,5e4), fontsize=12)
plt.xlabel('Transcript coverage (%)', fontsize=14)
plt.ylabel('# transcripts / sample', fontsize=14)
ax.tick_params('both', labelsize=13)
plt.yscale('log')
plt.savefig(outdir+'transcript_coverage_percent_persample_all_hist.png', dpi=300, bbox_inches='tight')

In [ ]:
# extract and plot a distribution of the number of transcripts with 100% sequence coverage

plot_rows = []
plot_cols = ['Sample', 'N transcripts 100%']
for ds in datasets:
    ds_dict = seq_cov_frac[ds]
    for k,v in ds_dict.items():
        plot_rows.append([k, len([x for x in v if x==1])])
plot_df = pd.DataFrame(plot_rows, columns=plot_cols)
plot_df  = plot_df.loc[plot_df['N transcripts 100%']!=0]

fig,ax = plt.subplots(figsize=(3,3))
sns.histplot(data=plot_df, x='N transcripts 100%', color='#AAAAAA', linewidth=0, )
median = np.median(plot_df['N transcripts 100%'].to_list())
plt.plot((median, median), ax.get_ylim(), '--r')
ax.annotate('median$=$\n70,956', xy=(35000,125), fontsize=12)
plt.xlabel('# transcripts with\n100% sequence coverage', fontsize=14)
plt.ylabel('# samples', fontsize=14)
ax.tick_params('both', labelsize=13)

plt.savefig(outdir+'n_transctipts_100pcnt_coverage_all_hist.png', dpi=300, bbox_inches='tight')

### Extended Data Figure 1h: K/R AAS

In [ ]:
# the file used here is the same as Supplemental_Data_2.SAAP_protein.xlsx but include SAAP with substitutions of K or R which were then filtered out and only has a subset of the columns (no protein data) 
all_saap_protein_df = pd.read_excel(proj_dir+'pipeline_output/analysis_dependencies/All_SAAP_protein_df_w_KR_AAS.xlsx')

# get index of SAAP-BP pairs with substitution of K or R
all_aas = all_saap_protein_df['AAS'].values
kr_idx = [i for i,aas in enumerate(all_aas) if 'K' in aas or 'R' in aas]

# subset of K|R substitutions that are K->R or R->K
k2r_aas = [i for i in kr_idx if 'K' in all_aas[i] and 'R' in all_aas[i]]

# subset of K|R substitutions that are missed cleavages
all_kr_saaps = all_saap_protein_df.loc[kr_idx, 'SAAP'].values
all_kr_saaps_trimmed = [x[:-1] for x in all_kr_saaps]
all_missed_cleavages = [x for x in all_kr_saaps_trimmed if 'K' in x or 'R' in x]
missed_cleavage_idx = [i for i in kr_idx if 'R' in  all_saap_protein_df.loc[i,'SAAP'][:-1] or 'K' in  all_saap_protein_df.loc[i,'SAAP'][:-1]]

# subsets of K|R substitutions for pie chart
other_kr_idx = [i for i in kr_idx if i not in k2r_aas and i not in missed_cleavage_idx]
k2r_aas_and_missed = [i for i in k2r_aas if i in missed_cleavage_idx]
miss_cleavage_only = [i for i in missed_cleavage_idx if i not in k2r_aas]
k2r_aas_only = [i for i in k2r_aas if i not in missed_cleavage_idx]

# plot pie chart
labels = 'Other K|R\nAAS', 'Missed\ncleavage', 'K<>R', 'K<>R +\nmissed cleavage'
sizes = [len(other_kr_idx), len(miss_cleavage_only), len(k2r_aas_only), len(k2r_aas_and_missed)]
fig,ax=plt.subplots(figsize=(2,2))
ax.pie(sizes, labels=labels, labeldistance=1.1, autopct='%1.f%%')
plt.savefig(outdir+'KR_missed_clevage_pie.pdf', bbox_inches='tight')

# Figure 2 - RAAS

#### Incl. Extended Data Figures 2, 3.

Read in dataframes generated for Supplemental Data Figures.
All data in these dataframes was taken from the dictionaries output from the decode pipeline

In [ ]:
filt_saap_df = pd.read_excel(proj_dir+'Supplemental_Data_2.SAAP_proteins.xlsx', index_col=0)
filt_prec_quant_df = pd.read_excel(proj_dir+'Supplemental_Data_3.SAAP_precursor_quant.xlsx', index_col=0)
filt_reporter_quant_df = pd.read_excel(proj_dir+'Supplemental_Data_4.SAAP_reporter_quant.xlsx', index_col=0)

filt_saap_df_list = [filt_saap_df.loc[filt_saap_df['Dataset']==ds] for ds in datasets]
reporter_quant_df_list = [filt_reporter_quant_df.loc[filt_reporter_quant_df['Dataset']==ds] for ds in datasets[:-1]]
prec_quant_df_list = [filt_prec_quant_df.loc[filt_prec_quant_df['Dataset']==ds] for ds in datasets]

### Figure 2b. sample-level RAAS distribution 

In [ ]:
# sample/reporter ion level RAAS distributions for each dataset 

fig,axes = plt.subplots(1,len(datasets),figsize=(5,3), sharey=True)
plt.subplots_adjust(wspace=0.05)

sns.set_style('whitegrid')
for ax in axes:
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.tick_params('x', bottom=False,labelbottom=False)
    if ax!=axes[0]:    
        ax.spines['left'].set_visible(False)
        ax.tick_params(left=False)

medians = [np.nanmedian(reporter_quant_df_list[i]['RAAS'].values) for i in range(len(datasets)-1)]
med_sort_idx = list(np.argsort(medians))
med_sort_idx.append(max(med_sort_idx)+1)

for i,ds in enumerate(datasets):
    ax_idx = med_sort_idx.index(i)
    axes[ax_idx].set_xlabel(ds, fontsize=12)
    if ds!='Healthy':
        ratio_data = [x for x in reporter_quant_df_list[i]['RAAS'].values if ~np.isnan(x) and ~np.isinf(x)]
    else:
        ratio_data = [x for x in prec_quant_df_list[i]['RAAS'].values if ~np.isnan(x) and ~np.isinf(x)]
    bihist(ratio_data, ratio_data, nbins=100,h=axes[ax_idx])
    if ax_idx==0:
        axes[ax_idx].annotate('N= ', (-0.1,1.01), xycoords='axes fraction', fontsize=13)
        axes[ax_idx].annotate(str(np.round(len(ratio_data), -3))[0:2]+'k', (0.32,1.01), xycoords='axes fraction', fontsize=13)
        
    else:
        axes[ax_idx].annotate(str(np.round(len(ratio_data), -3))[0:2]+'k', (0.25,1.01), xycoords='axes fraction', fontsize=13)
        
    axes[ax_idx].plot(axes[ax_idx].get_xlim(), (np.median(ratio_data), np.median(ratio_data)), '--r',linewidth=0.7)

axes[0].set_ylabel(r'log$_{10}$(RAAS)', fontsize=15)
axes[0].tick_params('y',labelsize=13)
axes[0].set_yticks([-6,-4,-2,0,2,4]);

plt.savefig(outdir+'Sample_level_RAAS_allDS.pdf', bbox_inches='tight')

In [ ]:
# fig 2b. sample-level RAAS distribution - all datasets in one distribution

sns.set_style('whitegrid')
fig,ax = plt.subplots(figsize=(3,3))

ratios = []
for i,ds in enumerate(datasets):
    if ds!='Healthy':
        ratio_data = [x for x in reporter_quant_df_list[i]['RAAS'].values if ~np.isnan(x) and ~np.isinf(x)]
    else:
        ratio_data = [x for x in prec_quant_df_list[i]['RAAS'].values if ~np.isnan(x) and ~np.isinf(x)]
    ratios = ratios + ratio_data
    
sns.histplot(ratios, color='#AAAAAA', linewidth=0)
plt.plot((np.median(ratios), np.median(ratios)),(plt.ylim()), '--r', linewidth=1)
ax.annotate('Median =\n$-$'+str(np.abs(np.round(np.median(ratios), 2))), (-1,2000), fontsize=12)
ax.set_ylabel(r'# SAAP', fontsize=14)
ax.set_xlabel(r'log$_{10}$(RAAS)', fontsize=14)
ax.tick_params('both',labelsize=13)
ax.set_xticks([-6,-4,-2,0,2,4]);

plt.savefig(outdir+'Sample_level_RAAS_allDS_onehistplot.pdf', bbox_inches='tight')

### Extended Data Figure 3a. TMT-level RAAS distribution

In [ ]:
fig,axes = plt.subplots(1,len(datasets),figsize=(len(datasets),3), sharey=True)

sns.set_style('whitegrid')
for ax in axes:
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.tick_params('x', bottom=False,labelbottom=False)
    if ax!=axes[0]:    
        ax.spines['left'].set_visible(False)
        ax.tick_params(left=False)

medians = [np.nanmedian(prec_quant_df_list[i]['RAAS'].values) for i in range(len(datasets)-1)]
med_sort_idx = list(np.argsort(medians))
med_sort_idx.append(max(med_sort_idx)+1)

for i,ds in enumerate(datasets):
    ax_idx = med_sort_idx.index(i)
    axes[ax_idx].set_xlabel(ds, fontsize=14)
    ratio_data = [x for x in prec_quant_df_list[i]['RAAS'].values if ~np.isnan(x) and ~np.isinf(x)]
    bihist(ratio_data, ratio_data, nbins=50,h=axes[ax_idx])
    if ax_idx==0:
        axes[ax_idx].annotate('N= ', (-0.2,1.01), xycoords='axes fraction', fontsize=13)
    axes[ax_idx].annotate('{:,}'.format((len(ratio_data))), (0.2,1.01), xycoords='axes fraction', fontsize=13)
    axes[ax_idx].plot(axes[ax_idx].get_xlim(), (np.median(ratio_data), np.median(ratio_data)), '--r',linewidth=0.7)

axes[0].set_ylabel(r'log$_{10}$(RAAS)', fontsize=15)
axes[0].tick_params('y',labelsize=13)

plt.savefig(outdir+'TMT_level_RAAS_allDS_nogrid.pdf', bbox_inches='tight')

### Extended Data Figure 3c. Sample median RAAS distributions

In [ ]:
fig,axes = plt.subplots(1,len(datasets),figsize=(len(datasets),3), sharey=True)
#plt.ylim([-3.5,0])
sns.set_style('whitegrid')
for ax in axes:
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.tick_params('x', bottom=False,labelbottom=False)
    if ax!=axes[0]:    
        ax.spines['left'].set_visible(False)
        ax.tick_params(left=False)

sample_median_values_list = []
for i,ds in enumerate(datasets):
    if ds!='Healthy':
        reporter_quant_df = reporter_quant_df_list[i]
        samples = list(set(reporter_quant_df['Sample name'].values))
        sample_medians = [np.nanmedian(reporter_quant_df.loc[reporter_quant_df['Sample name']==s, 'RAAS'].values) for s in samples]
        sample_median_values_list.append(sample_medians)
    else:
        prec_quant_df = prec_quant_df_list[i]
        samples = list(set(prec_quant_df['TMT/Tissue'].values))
        sample_medians = [np.nanmedian(prec_quant_df.loc[prec_quant_df['TMT/Tissue']==s, 'RAAS'].values) for s in samples]
        sample_median_values_list.append(sample_medians)
        
medians = [np.nanmedian(sample_median_values_list[i]) for i in range(len(datasets)-1)]
med_sort_idx = list(np.argsort(medians))
med_sort_idx.append(max(med_sort_idx)+1)

for i,ds in enumerate(datasets):
    ax_idx = med_sort_idx.index(i)
    axes[ax_idx].set_xlabel(ds, fontsize=14)
    ratio_data = [x for x in sample_median_values_list[i] if ~np.isnan(x) and ~np.isinf(x)]
    bihist(ratio_data, ratio_data, nbins=15,h=axes[ax_idx])
    if ax_idx==0:
        axes[ax_idx].annotate('N= ', (-0.15,1.01), xycoords='axes fraction', fontsize=13)
    axes[ax_idx].annotate('{:,}'.format((len(ratio_data))), (0.25,1.01), xycoords='axes fraction', fontsize=13)
    axes[ax_idx].plot(axes[ax_idx].get_xlim(), (np.median(ratio_data), np.median(ratio_data)), '--r',linewidth=0.7)

axes[0].set_ylabel(r'log$_{10}$(RAAS)', fontsize=15)
axes[0].tick_params('y',labelsize=13)

#plt.annotate('Cancer and normal adjacent tissue', (0.14,0.97), xycoords='figure fraction')
#plt.show()
plt.savefig(outdir+'Sample_median_RAAS_allDS.pdf', bbox_inches='tight')

### Extended data figure 3b. Median RAAS for all unique SAAP

In [ ]:
fig,axes = plt.subplots(1,len(datasets),figsize=(len(datasets),3), sharey=True)
sns.set_style('whitegrid')
for ax in axes:
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.tick_params('x', bottom=False,labelbottom=False)
    if ax!=axes[0]:    
        ax.spines['left'].set_visible(False)
        ax.tick_params(left=False)

saap_median_values_list = []
for i,ds in enumerate(datasets):
    if ds!='Healthy':
        reporter_quant_df = reporter_quant_df_list[i]
        saap = list(set(reporter_quant_df['SAAP'].values))
        saap_medians = [np.nanmedian(reporter_quant_df.loc[reporter_quant_df['SAAP']==s, 'RAAS'].values) for s in saap]
        saap_median_values_list.append(saap_medians)
    else:
        prec_quant_df = prec_quant_df_list[i]
        saap = list(set(prec_quant_df['SAAP'].values))
        saap_medians = [np.nanmedian(prec_quant_df.loc[prec_quant_df['SAAP']==s, 'RAAS'].values) for s in saap]
        saap_median_values_list.append(saap_medians)
        
medians = [np.nanmedian(saap_median_values_list[i]) for i in range(len(datasets)-1)]
med_sort_idx = list(np.argsort(medians))
med_sort_idx.append(max(med_sort_idx)+1)

for i,ds in enumerate(datasets):
    ax_idx = med_sort_idx.index(i)
    axes[ax_idx].set_xlabel(ds, fontsize=14)
    ratio_data = [x for x in saap_median_values_list[i] if ~np.isnan(x) and ~np.isinf(x)]
    bihist(ratio_data, ratio_data, nbins=15,h=axes[ax_idx])
    if ax_idx==0:
        axes[ax_idx].annotate('N= ', (-0.15,1.01), xycoords='axes fraction', fontsize=13)
    axes[ax_idx].annotate('{:,}'.format((len(ratio_data))), (0.25,1.01), xycoords='axes fraction', fontsize=13)
    axes[ax_idx].plot(axes[ax_idx].get_xlim(), (np.median(ratio_data), np.median(ratio_data)), '--r',linewidth=0.7)

axes[0].set_ylabel(r'log$_{10}$(RAAS)', fontsize=15)
axes[0].tick_params('y',labelsize=13)

plt.savefig(outdir+'SAAP_median_RAAS_allDS.pdf', bbox_inches='tight')

### Extended data figure 3d. Medians of extended 3a-c.

In [ ]:
all_medians = [np.nanmedian(reporter_quant_df_list[i]['RAAS'].values) for i in range(len(datasets)-1)]
all_medians.append(np.nanmedian(prec_quant_df_list[-1]['RAAS'].values))
sample_median_values_list = []
for i,ds in enumerate(datasets):
    if ds!='Healthy':
        reporter_quant_df = reporter_quant_df_list[i]
        samples = list(set(reporter_quant_df['Sample name'].values))
        sample_medians = [np.nanmedian(reporter_quant_df.loc[reporter_quant_df['Sample name']==s, 'RAAS'].values) for s in samples]
        sample_median_values_list.append(sample_medians)
    else:
        prec_quant_df = prec_quant_df_list[i]
        samples = list(set(prec_quant_df['TMT/Tissue'].values))
        sample_medians = [np.nanmedian(prec_quant_df.loc[prec_quant_df['TMT/Tissue']==s, 'RAAS'].values) for s in samples]
        sample_median_values_list.append(sample_medians)
sample_medians = [np.nanmedian(sample_median_values_list[i]) for i in range(len(datasets))]

saap_median_values_list = []
for i,ds in enumerate(datasets):
    if ds!='Healthy':
        reporter_quant_df = reporter_quant_df_list[i]
        saap = list(set(reporter_quant_df['SAAP'].values))
        saap_medians = [np.nanmedian(reporter_quant_df.loc[reporter_quant_df['SAAP']==s, 'RAAS'].values) for s in saap]
        saap_median_values_list.append(saap_medians)
    else:
        prec_quant_df = prec_quant_df_list[i]
        saap = list(set(prec_quant_df['SAAP'].values))
        saap_medians = [np.nanmedian(prec_quant_df.loc[prec_quant_df['SAAP']==s, 'RAAS'].values) for s in saap]
        saap_median_values_list.append(saap_medians)
        
saap_medians = [np.nanmedian(saap_median_values_list[i]) for i in range(len(datasets))]

plot_rows = []
plot_cols = ['Dataset', 'Median RAAS', 'Median type']
for m, median in enumerate(all_medians):
    plot_rows.append([datasets[m], median, 'All data median'])
for m, median in enumerate(sample_medians):
    plot_rows.append([datasets[m], median, 'Sample median'])
for m, median in enumerate(saap_medians):
    plot_rows.append([datasets[m], median, 'SAAP median'])
    
plot_df = pd.DataFrame(plot_rows, columns=plot_cols)    

fig,ax = plt.subplots(figsize=(6,3))
sns.swarmplot(data=plot_df, x='Dataset', y='Median RAAS', hue='Median type', s=8,
              order=['LUAD','LSCC','CCRCC','BRCA','UCEC','PDAC','Healthy'])
plt.legend(title='', fontsize=13)
ax.tick_params('both', labelsize=13)
plt.ylabel('Median log$_{10}$(RAAS)', fontsize=14)
plt.xlabel('')
plt.savefig(outdir+'RAAS_distribution_median_comparison_swarmplot.pdf', bbox_inches='tight')

### Figure 2g. Proteins with high RAAS

In [ ]:
# this dataframe was generated separately using the results from the protein set enrichment analysis 

ref_prot_df = pd.read_excel(proj_dir+'pipeline_output/analysis_dependencies/Reference_proteins_ranked_Median_RAAS.xlsx', index_col=0)

In [ ]:
sns.set_style('ticks')
fig,ax = plt.subplots(figsize=(9,4))

hue_order=['Protein transport and degradation','DNA, RNA processing and metabolism','Signaling and enzymatic reactions', 'Immune response', 'Cytokeletal and stuctural', 'Small molecule metabolism', 'Oxidative stress response', 'Transcription factor']
nonpath_rows = [i for i,row in ref_prot_df.iterrows() if row['Median RAAS']>-1 and row['Pathway group'] not in hue_order]

sns.scatterplot(data=ref_prot_df.loc[nonpath_rows], x='Rank', y='Median RAAS', color='#aaaaaa', s=80, linewidth=0, alpha=1)
sns.scatterplot(data=ref_prot_df.loc[(ref_prot_df['Median RAAS']>-1) & (ref_prot_df['Pathway group']!='')], x='Rank', y='Median RAAS', s=80, hue='Pathway group', linewidth=0.2,alpha=0.8,
                palette=colors[0:7]+['black'], hue_order=hue_order)

ax.tick_params('both', labelsize=15)
plt.xlabel('Protein rank', fontsize=17)
plt.ylabel('Median log$_{10}$(RAAS)', fontsize=17)
plt.legend(title='', fontsize=11)

ranks2plot = [0,495,4,34,171, 158, 6]
sns.scatterplot(data=ref_prot_df.loc[ref_prot_df['Rank'].isin(ranks2plot)], x='Rank', y='Median RAAS', s=120, hue='Pathway group', linewidth=0.8, alpha=1,
                palette=colors[0:7]+['black'], hue_order=hue_order)
handles, labels = ax.get_legend_handles_labels()
plt.legend(handles=handles[:8], labels=labels[:8], fontsize=11, bbox_to_anchor=(1,1))

texts2plot = ['LYST: Lysosomal trafficking regulator', 'PSMA1: Proteasome\n'+r'subunit $\alpha$ type-1',
             'FES: Tyrosine-protein kinase Fes/Fps', 'DDX17: Probable ATP-dependent\nRNA helicase isoform 3',
             'ACTN1: Alpha-actinin-1', 'PSMA4: Proteasome\n'+r'subunit $\alpha$ type-4', 
             'ZNF845: Zinc finger protein 845']
colors2plot = [colors[0], colors[0], colors[2], colors[1], colors[4], colors[0], 'black']

for i,rank in enumerate(ranks2plot):
    txt = texts2plot[i]
    y = ref_prot_df.loc[ref_prot_df['Rank']==rank,'Median RAAS']
    if 'Proteasome' not in txt and 'actin' not in txt and 'DDX' not in txt:
        ax.annotate(txt, xy=(rank,y), xytext=(rank+50,y+0.1), xycoords='data', textcoords='data', va='top',
                arrowprops=dict(facecolor='gray', headwidth=5,width=2, linewidth=0), color=colors2plot[i], fontsize=11)
    elif 'DDX' in txt:
        ax.annotate(txt, xy=(rank,y), xytext=(rank+50,y+0.3), xycoords='data', textcoords='data', va='center',
        arrowprops=dict(facecolor='gray', headwidth=5,width=2, linewidth=0), color=colors2plot[i], fontsize=11)
    elif 'type-4' in txt:
        ax.annotate(txt, xy=(rank,y), xytext=(rank-180,y-0.6), xycoords='data', textcoords='data', va='bottom',
        arrowprops=dict(facecolor='gray', headwidth=5,width=2, linewidth=0), color=colors2plot[i], fontsize=11)
    elif 'type-1' in txt:
        ax.annotate(txt, xy=(rank,y), xytext=(rank+45,y+0.4), xycoords='data', textcoords='data', va='bottom',
        arrowprops=dict(facecolor='gray', headwidth=5,width=2, linewidth=0), color=colors2plot[i], fontsize=11)
    else:
        ax.annotate(txt, xy=(rank,y), xytext=(rank-160,y-0.8), xycoords='data', textcoords='data', va='center',
        arrowprops=dict(facecolor='gray', headwidth=5,width=2, linewidth=0), color=colors2plot[i], fontsize=11)
plt.savefig(outdir+'Protein_group_RAAS_rank_plot.pdf', bbox_inches='tight')


### Figure 2d. Peptide precursor ion abundance relative to shared peptide 


In [ ]:
def raas2bin(raas):
    """ function to bin RAAS values for plotting"""
    if raas<-3:
        raas_bin = '[-$\infty$,-3]'
    elif raas<-2:
        raas_bin = '(-3,-2]'
    elif raas<-1:
        raas_bin = '(-2,-1]'
    elif raas<0:
        raas_bin = '(-1,0]'
    elif raas<1:
        raas_bin = '(0,1]'
    else:
        raas_bin = '(1,$\infty$)'
    return(raas_bin)


# add RAAS bin and ratios of SAAP and BP to shared peptide abundance
filt_prec_quant_df['RAAS_bin'] = filt_prec_quant_df['RAAS'].apply(raas2bin)
filt_prec_quant_df['MTP/SP'] = np.nan
filt_prec_quant_df['BP/SP'] = np.nan

for i,row in filt_prec_quant_df.iterrows():
    mtp = row['SAAP abundance']
    bp = row['BP abundance']
    sp = row['Mean shared peptide precursor intensity']
    filt_prec_quant_df.loc[i,'MTP/SP'] = mtp/sp
    filt_prec_quant_df.loc[i, 'BP/SP'] = bp/sp

raas_bins = ['[-$\infty$,-3]', '(-3,-2]','(-2,-1]','(-1,0]','(0,1]','(1,$\infty$)']

In [ ]:
# plot distributions of abundance of SAAP/BP to shared peptide abundance as a function of RAAS

fig,axi = plt.subplots(2, len(raas_bins), figsize=(5,4), sharey=True)#, sharex=True)
plt.subplots_adjust(wspace=0, hspace=0.1)
sns.set_style('whitegrid')
for j, axes in enumerate(axi):
    for i,rbin in enumerate(raas_bins):
        ax = axes[i]
        if j==1:
            ax.set_xlabel(rbin, fontsize=14);
        ax.set_xticks([])
        if j==0:
            rbin_data = [np.log10(x) for x in filt_prec_quant_df.loc[filt_prec_quant_df['RAAS_bin']==rbin, 'MTP/SP'].values]
            peptype = 'SAAP'
        else:
            rbin_data = [np.log10(x) for x in filt_prec_quant_df.loc[filt_prec_quant_df['RAAS_bin']==rbin, 'BP/SP'].values]
            peptype = 'BP'

        rbin_data = [x for x in rbin_data if (~np.isnan(x)) and (~np.isinf(x))]
        bihist(rbin_data, rbin_data, nbins=20, h=ax)
        if i>0:
            ax.spines['left'].set_visible(False)
           # ax.set_yticks([])
        ax.spines['top'].set_visible(False)

        median_abund = np.nanmedian(rbin_data)
        print(rbin, median_abund)
        ax.plot(ax.get_xlim(), (median_abund, median_abund), '--r')

        if ax==axes[-1]:
            ax.spines['right'].set_visible(False)
        ax.set_ylim([-4.5,4])

    axes[0].tick_params('y', labelsize=14)
    axes[0].set_yticks([-4,-2,0,2,4], labels=[-4,-2,0,2,'']);
    axes[0].annotate(peptype, (0.2, 0.85), xycoords='axes fraction', fontsize=13)   
axi[1][0].annotate('log$_{10}$(ratio to shared peptides)', (-0.9,1.1), xycoords='axes fraction', fontsize=15, 
                   rotation='vertical', va='center')
    
plt.savefig(outdir+'SAAP_BP_SP_ratio_bihist_RAASbins_allDS.pdf', bbox_inches='tight')    

### Extended data figure 2i: Correlations of SAAP|BP to SP across all samples

In [ ]:
# get a dataframe of the correlations of SAAP/BP abundances to shared peptide abundance 
# at the reporter ion level, across all sampls

corr_dfs = []
for d,ds in enumerate(datasets[:-1]):
    print(ds)
    ds_df = filt_reporter_quant_df.loc[filt_reporter_quant_df['Dataset']==ds]
    set_peps = list(set(ds_df['SAAP']))

    corr_rows = []
    corr_cols = ['Dataset','SAAP','BP', 'N samples', 'RAAS list', 'Median RAAS', 'SAAP corr to mean SP', 'BP corr to mean SP']
    for i,pep in enumerate(set_peps):
        pep_df = ds_df.loc[ds_df['SAAP']==pep]
        set_bps = list(set(pep_df['BP']))
        for bp in set_bps:
            pep_bp_df = pep_df.loc[pep_df['BP']==bp]
            pep_bp_df = pep_bp_df.loc[~np.isnan(pep_bp_df['Mean shared peptide reporter intensity'])]
            if len(pep_bp_df)>=10:
                SAAP_corr = sp.stats.pearsonr(pep_bp_df['SAAP abundance'], pep_bp_df['Mean shared peptide reporter intensity'])[0]
                bp_corr = sp.stats.pearsonr(pep_bp_df['BP abundance'], pep_bp_df['Mean shared peptide reporter intensity'])[0]
                raas_list = list(pep_bp_df['RAAS'].values)
                n_samples = len(raas_list)
                median_raas = np.nanmedian(raas_list)
                corr_rows.append([ds, pep, bp, n_samples, raas_list, median_raas, SAAP_corr, bp_corr])
    
    corr_df = pd.DataFrame(corr_rows, columns=corr_cols)
    corr_df['RAAS bin'] = corr_df['Median RAAS'].apply(raas2bin)
    corr_df['SAAP - BP corr'] = [row['SAAP corr to mean SP']-row['BP corr to mean SP'] for i,row in corr_df.iterrows()]
    corr_df.dropna(how='any', axis=0, inplace=True)
    corr_dfs.append(corr_df)
    
all_corr_df = pd.concat(corr_dfs)
all_corr_df.reset_index(inplace=True)

In [ ]:
# plot violin plots of correlation distributions as function of RAAS for SAAP (middle panel)

fig,axes = plt.subplots(1, len(raas_bins), figsize=(6,3), sharey=True)
plt.subplots_adjust(wspace=0)
plt.ylim([-1,1.2])
sns.set_style('whitegrid')
for i,rbin in enumerate(raas_bins):
    ax = axes[i]
    #ax.set_ylim([-0.5,1])
    ax.set_xlabel(rbin, fontsize=13)
    ax.set_xticks([])
    rbin_data = all_corr_df.loc[all_corr_df['RAAS bin']==rbin, 'SAAP corr to mean SP'].values
    print(len(rbin_data))
    
    bihist(rbin_data, rbin_data, nbins=20, h=ax)
    if i>0:
        ax.spines['left'].set_visible(False)
       # ax.set_yticks([])
    ax.spines['top'].set_visible(False)

    median_corr = np.nanmedian(rbin_data)
    print(rbin, median_corr)
    ax.plot(ax.get_xlim(), (median_corr, median_corr), '--r')
     
    if ax==axes[-1]:
        ax.spines['right'].set_visible(False)
        ax.plot(ax.get_xlim(), (median_corr, median_corr), '--r', linewidth=0.5)
axes[0].set_ylabel('corr(SAAP, shared peptides)', fontsize=14)
axes[0].set_yticks([-1, -0.5,0,0.5,1])
axes[0].set_yticklabels([-1, -0.5,0,0.5,1], fontsize=13);

plt.annotate(r'log$_{10}$(RAAS)',(0.45,0.02), xycoords=('figure fraction'), fontsize=15)
plt.savefig(outdir+'SAAP_corr_to_median_SP.png', dpi=300, bbox_inches='tight')    

In [ ]:
# plot violin plots of correlation distributions as function of RAAS for SAAP (top panel)

fig,axes = plt.subplots(1, len(raas_bins), figsize=(6,3), sharey=True)
plt.subplots_adjust(wspace=0)
plt.ylim([-1,1.2])
sns.set_style('whitegrid')
for i,rbin in enumerate(raas_bins):
    ax = axes[i]
    #ax.set_ylim([-0.5,1])
    ax.set_xlabel(rbin, fontsize=13)
    ax.set_xticks([])
    rbin_data = all_corr_df.loc[all_corr_df['RAAS bin']==rbin, 'BP corr to mean SP'].values
    
    bihist(rbin_data, rbin_data, nbins=20, h=ax)
    if i>0:
        ax.spines['left'].set_visible(False)
       # ax.set_yticks([])
    ax.spines['top'].set_visible(False)

    median_corr = np.nanmedian(rbin_data)
    print(rbin, median_corr)
    ax.plot(ax.get_xlim(), (median_corr, median_corr), '--r')
     
    if ax==axes[-1]:
        ax.spines['right'].set_visible(False)
        ax.plot(ax.get_xlim(), (median_corr, median_corr), '--r', linewidth=0.5)
axes[0].set_ylabel('corr(BP, shared peptides)', fontsize=14)
axes[0].set_yticks([-1, -0.5,0,0.5,1])
axes[0].set_yticklabels([-1, -0.5,0,0.5,1], fontsize=13);

plt.annotate(r'log$_{10}$(RAAS)',(0.45,0.02), xycoords=('figure fraction'), fontsize=15)
plt.savefig(outdir+'BP_corr_to_median_SP.png', dpi=300, bbox_inches='tight')    

In [ ]:
# plot violin plots of difference in correlation between SAAP and BP (lower panel)

fig,axes = plt.subplots(1, len(raas_bins), figsize=(6,3), sharey=True)
plt.subplots_adjust(wspace=0)
sns.set_style('whitegrid')
for i,rbin in enumerate(raas_bins):
    ax = axes[i]
    ax.set_xlabel(rbin, fontsize=13)
    ax.set_xticks([])
    rbin_data = [x for x in all_corr_df.loc[all_corr_df['RAAS bin']==rbin, 'SAAP - BP corr'].values if (~np.isnan(x) and (~np.isinf(x)))]
    
    bihist(rbin_data, rbin_data, nbins=20, h=ax)
    if i>0:
        ax.spines['left'].set_visible(False)
    ax.spines['top'].set_visible(False)

    median_corr = np.nanmedian(rbin_data)
    ax.plot(ax.get_xlim(), (median_corr, median_corr), '--r')

axes[0].set_ylabel('Correlation difference', fontsize=14)
axes[0].set_yticks([-2,-1,0,1,2])
axes[0].set_yticklabels([-2,-1,0,1,2], fontsize=13);


axes[2].annotate('log$_{10}$(RAAS)',(0,-0.25), xycoords=('axes fraction'), fontsize=16)

plt.savefig(outdir+'SAAP_minus_BP_corr_to_SP.png', dpi=300, bbox_inches='tight')    

### Figure 2i and extended data figure 3i: codon mismatch

In [ ]:
codon2aa = {
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',                
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
        'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
        'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W'
}

AAs = 'ACDEFGHIKLMNPQRSTVWY'
subs_list = [i+' to '+j for i in AAs for j in AAs if i!=j]
subs_list = [x for x in subs_list if x[-1]!='L']
subs_list = [x if x[-1]!='I' else x[:-1]+'I/L' for x in subs_list]

# generate a dictionary with substitution types and their RAAS values for each dataset
aas_ratio_dict = {}
for i,ds in enumerate(datasets):
    print(ds)
    subs_dict = {x:[] for x in subs_list}
    mtp_quant = pickle.load(open(data_dir_list[i]+'MTP_quant_dict.p', 'rb'))
    for k,v in subs_dict.items():
        sub_peps = {i:x for i,x in mtp_quant.items() if x['aa_sub']==k}
        if len(sub_peps)>0:
            sub_ratios = [l for L in [[y for y in x['Prec_ratio'].values() if (~np.isnan(y)) and (~np.isinf(y))] for x in sub_peps.values()] for l in L]
            subs_dict[k] = v + sub_ratios
    aas_ratio_dict[ds] = subs_dict

In [ ]:
# generate dataframe for plot

plot_rows = []
cols = ['Dataset', 'AAS', 'Minimum base error', 'Log substitution ratio', 'Bases changed']

for ds in datasets:
    subs_dict = aas_ratio_dict[ds]
    for k,v in subs_dict.items():
        base_codons = [i for i,x in codon2aa.items() if x==k[0]]
        dest_codons =  [i for i,x in codon2aa.items() if x==k[-1]]
        if base_codons!=dest_codons: # case of I/L to I/L
            min_change = 3
            min_changes = [1,2,3]
            for b in base_codons:
                for d in dest_codons:
                    change = len([x for i,x in enumerate(b) if d[i]!=x])
                    if change<min_change:
                        min_change=change
                        min_changes = [i+1 for i,x in enumerate(b) if d[i]!=x]
                    elif change==min_change:
                        new_changes = [i+1 for i,x in enumerate(b) if d[i]!=x]
                        if new_changes[0]>min_changes[0]:
                            min_changes=new_changes

            for aasr in v:
                aasr = np.log10(2**aasr)
                plot_rows.append([ds, k, min_change, aasr, min_changes])

plot_df = pd.DataFrame(plot_rows, columns=cols)
plot_df.to_excel(outdir+'Codon_change_df.xlsx')

In [ ]:
#plot_df = pd.read_excel(outdir+'Codon_change_df.xlsx', index_col=0)

In [ ]:
# plot for all dataset separately (extended 3i)

sns.set_style('whitegrid')
fig,ax = plt.subplots(figsize=(6,1.6))
sns.boxplot(data=plot_df, x='Dataset', y='Log substitution ratio', hue='Minimum base error', dodge=True, fliersize=0.5, linewidth=0.8)
plt.ylabel(r'log$_{10}$(RAAS)', fontsize=15);
plt.xlabel('')
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
handles,labels = ax.get_legend_handles_labels()
ph = [plt.plot([],marker="", ls="")[0]]
plt.legend(handles=ph+handles, labels=['# codon mismatches:']+labels ,bbox_to_anchor=(0.45,1.05), fontsize=13, ncol=4, frameon=False, 
           columnspacing=0.7, handletextpad=0.1, loc='center')
plt.savefig(outdir+'RAAS_vs_nCodonMismatches.pdf', bbox_inches='tight')

In [ ]:
# plot for all dataset together (figure 2i)

sns.set_style('whitegrid')
fig,ax = plt.subplots(figsize=(1.7,3))
sns.boxplot(data=plot_df, x='Minimum base error', y='Log substitution ratio', fliersize=0.5, linewidth=0.8, color='#AAAAAA')
plt.ylabel(r'log$_{10}$(RAAS)', fontsize=14);
plt.xlabel('# codon mismatches', fontsize=14)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.savefig(outdir+'allDS_RAAS_vs_nCodonMismatches.pdf', bbox_inches='tight')

### Extended data figure 3j. T and V tRNA ligase abundances 

In [ ]:
 aa_name_letter_dict = {'cysteine': 'C', 'aspartate': 'D', 'serine': 'S', 'glutamine': 'Q', 'lysine': 'K',
     'isoleucine': 'I', 'proline': 'P', 'threonine': 'T', 'phenylalanine': 'F', 'asparagine': 'N', 
     'glycine': 'G', 'histidine': 'H', 'leucine': 'L', 'arginine': 'R', 'tryptophan': 'W', 
     'alanine': 'A', 'valine':'V', 'glutamate': 'E', 'tyrosine': 'Y', 'methionine': 'M'}

aa_letter_name_dict = {v:k for k,v in aa_name_letter_dict.items()}

# this dictionary was generated in a separate script using the MaxQuant protein output data
aars_abund_dict = pickle.load(open(proj_dir+'pipeline_output/analysis_dependencies/refnorm_AARS_abund_dict.p', 'rb'))

In [ ]:
def get_boxplot_data(aas):
    """ function to get data for plotting tRNA ligase abundances of 2 amino acids, given a subsitution type"""
    aa1 = aa_letter_name_dict[aas[0]]
    aa2 = aa_letter_name_dict[aas[-1]]
    
    boxplot_rows = []
    cols=['Reporter-distributed\nprecursor intensity', 'tRNA ligase', 'Dataset']
    
    for ds in datasets[:-1]:
        aars_df = aars_abund_dict[ds]
        #aa1_aars_vals = [x for x in aars_df.loc[[i for i,row in aars_df.iterrows() if (aa1 in row['Unnamed: 0']) and ('mitochondria' not in row['Unnamed: 0'])],:].values[0][1:-1] if ~np.isnan(x)]
        #aa2_aars_vals = [x for x in aars_df.loc[[i for i,row in aars_df.iterrows() if (aa2 in row['Unnamed: 0']) and ('mitochondria' not in row['Unnamed: 0'])],:].values[0][1:-1] if ~np.isnan(x)]
        aa1_aars_vals = [x for x in aars_df.loc[[i for i,row in aars_df.iterrows() if (aa1 in i) and ('mitochondria' not in i)],:].values.flatten() if ~np.isnan(x)]
        aa2_aars_vals = [x for x in aars_df.loc[[i for i,row in aars_df.iterrows() if (aa2 in i) and ('mitochondria' not in i)],:].values.flatten() if ~np.isnan(x)]

        [boxplot_rows.append([val, aa1+'-tRNA ligase', ds]) for val in aa1_aars_vals]
        [boxplot_rows.append([val, aa2+'-tRNA ligase', ds]) for val in aa2_aars_vals]
    boxplot_df = pd.DataFrame(boxplot_rows, columns=cols)
    
    return(boxplot_df)

In [ ]:
# generate boxplot for substitution type

aas = 'T to V'
boxplot_df = get_boxplot_data(aas)

fig,ax = plt.subplots(figsize=(3,2))
sns.boxplot(data=boxplot_df, y='Reporter-distributed\nprecursor intensity', x='Dataset', hue='tRNA ligase', linewidth=0.8, fliersize=0)
#plt.ylim([0,0.8])
ax.tick_params('y', labelsize=13)
plt.xlabel('')
plt.ylim([0,6])
ax.legend(bbox_to_anchor=(0.5,1.15), fontsize=11, title='', frameon=False, loc='center')
ax.tick_params('x', rotation=45, labelsize=12)
plt.ylabel('Protein levels', fontsize=14)
plt.savefig(outdir+'T2V_AAligase_abund.pdf', bbox_inches='tight')

### Figures 2k,l and Extended Data Figure 3l,m,n and Supplemental Figure 1: Protein turnover analysis with metabolic pulse SILAC data



In [ ]:
savitski_dir = proj_dir+'Savitski/'
aa_subs_dir = proj_dir+'AA_subs_pipeline/'
val_dir = proj_dir+'MSFragger_validation/'

cell_types = ['Hepatocytes','Bcells','Monocytes','NKcells']

# read in dataset metadata and create dictionary with values 
metadata = pd.read_csv(val_dir+'Meta.csv')
meta_dict = {}
for i,row in metadata.iterrows():
    meta_dict[row['raw file']] = {'celltype':row['cell type'], 'timepoint':row['time point'], 

In [ ]:
# read in supplemental data file generated from output of running decode pipeline on the savitski data
# here there is one line per SAAP-BP per sample (replicates on separate lines)

quant_df = pd.read_excel(proj_dir+'Supplemental_Data_5.SAAP_SILAC_quant.xlsx', index_col=0) 

In [ ]:
# plot of Hepatocyte RAAS, extended data figure 3l

fig,ax = plt.subplots(figsize=(3,3))
sns.histplot(data=quant_df, x='Light+Heavy RAAS', color='#AAAAAA', linewidth=0)
plt.xlabel('log$_{10}$RAAS', fontsize=14)
plt.ylabel('# SAAP', fontsize=14)
median = np.nanmedian(quant_df['Light+Heavy RAAS'])
plt.plot((median,median), ax.get_ylim(), '--r')
plt.annotate('Median$=$\n'+'$-$'+str(np.round(np.abs(median),2)), xy=(-1,1500), fontsize=12)
plt.xticks([-4,-2,0,2])
ax.tick_params('both', labelsize=13)
plt.savefig(outdir+'Savistki_allhepatocyte_RAAS_distribution.pdf', bbox_inches='tight')

In [ ]:
# reformat the data in quant_df so that there is one line per SAAP-BP pair per cell type
# this is more convenient for computing degradation rates 
# this file is also provided in the google drive folder 

import warnings
warnings.filterwarnings("ignore")

rep_data_df_list = []
quant_data = quant_df

celltype = 'Hepatocytes'
for celltype in cell_types:
    print(celltype)
    cell_data = quant_data.loc[quant_data['Cell type']==celltype]
    saap_bp = list(set([row['SAAP']+';'+row['BP']+';'+row['AAS'] for i,row in cell_data.iterrows()]))
    print(len(saap_bp))
    timepoints = sorted(list(set(cell_data['Time'].to_list())))
    reps = list(set(cell_data['Replicate'].to_list()))

    cols = ['Cell type', 'SAAP', 'BP', 'AAS', 'CPTAC SAAP', 
            'BP_H_time1_rep1', 'BP_H_time1_rep2', 'BP_H_time2_rep1', 'BP_H_time2_rep2', 'BP_H_time3_rep1', 'BP_H_time3_rep2', 'BP_H_time4_rep1', 'BP_H_time4_rep2',
            'BP_L_time1_rep1', 'BP_L_time1_rep2', 'BP_L_time2_rep1', 'BP_L_time2_rep2', 'BP_L_time3_rep1', 'BP_L_time3_rep2', 'BP_L_time4_rep1', 'BP_L_time4_rep2',
            'BP_time1_rep1', 'BP_time1_rep2', 'BP_time2_rep1', 'BP_time2_rep2', 'BP_time3_rep1', 'BP_time3_rep2', 'BP_time4_rep1', 'BP_time4_rep2',
            'SAAP_H_time1_rep1', 'SAAP_H_time1_rep2', 'SAAP_H_time2_rep1', 'SAAP_H_time2_rep2', 'SAAP_H_time3_rep1', 'SAAP_H_time3_rep2', 'SAAP_H_time4_rep1', 'SAAP_H_time4_rep2',
            'SAAP_L_time1_rep1', 'SAAP_L_time1_rep2', 'SAAP_L_time2_rep1', 'SAAP_L_time2_rep2', 'SAAP_L_time3_rep1', 'SAAP_L_time3_rep2', 'SAAP_L_time4_rep1', 'SAAP_L_time4_rep2',
            'SAAP_time1_rep1', 'SAAP_time1_rep2', 'SAAP_time2_rep1', 'SAAP_time2_rep2', 'SAAP_time3_rep1', 'SAAP_time3_rep2', 'SAAP_time4_rep1', 'SAAP_time4_rep2', 
            'BP_HL_time1_rep1', 'BP_HL_time1_rep2', 'BP_HL_time2_rep1', 'BP_HL_time2_rep2', 'BP_HL_time3_rep1', 'BP_HL_time3_rep2', 'BP_HL_time4_rep1', 'BP_HL_time4_rep2',
            'SAAP_HL_time1_rep1', 'SAAP_HL_time1_rep2', 'SAAP_HL_time2_rep1', 'SAAP_HL_time2_rep2', 'SAAP_HL_time3_rep1', 'SAAP_HL_time3_rep2', 'SAAP_HL_time4_rep1', 'SAAP_HL_time4_rep2', 
            'RAAS_time1_rep1', 'RAAS_time1_rep2', 'RAAS_time2_rep1', 'RAAS_time2_rep2', 'RAAS_time3_rep1', 'RAAS_time3_rep2', 'RAAS_time4_rep1', 'RAAS_time4_rep2',
            'BP_H_mean_time1', 'BP_H_mean_time2', 'BP_H_mean_time3', 'BP_H_mean_time4',
            'BP_L_mean_time1', 'BP_L_mean_time2', 'BP_L_mean_time3', 'BP_L_mean_time4',
            'BP_mean_time1', 'BP_mean_time2', 'BP_mean_time3', 'BP_mean_time4', 
            'SAAP_H_mean_time1', 'SAAP_H_mean_time2', 'SAAP_H_mean_time3', 'SAAP_H_mean_time4',
            'SAAP_L_mean_time1', 'SAAP_L_mean_time2', 'SAAP_L_mean_time3', 'SAAP_L_mean_time4',
            'SAAP_mean_time1', 'SAAP_mean_time2', 'SAAP_mean_time3','SAAP_mean_time4', 
            'BP_HL_mean_time1', 'BP_HL_mean_time2', 'BP_HL_mean_time3', 'BP_HL_mean_time4', 
            'SAAP_HL_mean_time1', 'SAAP_HL_mean_time2', 'SAAP_HL_mean_time3', 'SAAP_HL_mean_time4',
            'RAAS_mean_time1', 'RAAS_mean_time2', 'RAAS_mean_time3', 'RAAS_mean_time4', 'N_timepoints',
            'BP_H_mean', 'BP_H_std','BP_H_CV', 'BP_L_mean', 'BP_L_std','BP_L_CV', 'BP_mean', 'BP_std','BP_CV',
            'SAAP_H_mean', 'SAAP_H_std','SAAP_H_CV', 'SAAP_L_mean', 'SAAP_L_std','SAAP_L_CV','SAAP_mean','SAAP_std','SAAP_CV',
            'BP_HL_mean','BP_HL_std','BP_HL_CV','SAAP_HL_mean','SAAP_HL_std','SAAP_HL_CV','RAAS_mean','RAAS_std','RAAS_CV']
    rep_data_df = pd.DataFrame(index=list(range(len(saap_bp))), columns=cols)

    for i, sba in enumerate(saap_bp):
        if i%1000==0:
            print(i)
        sba_split = sba.split(';')
        saap = sba_split[0]
        bp = sba_split[1]
        aas = sba_split[2]

        rep_data_df.loc[i, 'Cell type'] = celltype
        rep_data_df.loc[i, 'SAAP'] = saap
        rep_data_df.loc[i, 'BP'] = bp
        rep_data_df.loc[i, 'AAS'] = aas

        sba_df = cell_data.loc[(cell_data['SAAP']==saap) & (cell_data['BP']==bp)]
        cptac = sba_df['CPTAC SAAP'].values[0]
        rep_data_df.loc[i,'CPTAC SAAP'] = cptac
        #sba_df

        for j,tp in enumerate(timepoints):
            rep1_df = sba_df.loc[(sba_df['Time']==tp) & (sba_df['Replicate']==1)]
            rep2_df = sba_df.loc[(sba_df['Time']==tp) & (sba_df['Replicate']==2)]

            bp_h_1 = rep1_df['Heavy BP intensity'].values[0]
            bp_h_2 = rep2_df['Heavy BP intensity'].values[0]
            bp_l_1 = rep1_df['Light BP intensity'].values[0]
            bp_l_2 = rep2_df['Light BP intensity'].values[0]
            bp_1 = rep1_df['Light+Heavy BP intensity'].values[0]
            bp_2 = rep2_df['Light+Heavy BP intensity'].values[0]
            bp_hl_1 = rep1_df['H/L BP intensity'].values[0]
            bp_hl_2 = rep2_df['H/L BP intensity'].values[0]
            saap_h_1 = rep1_df['Heavy SAAP intensity'].values[0]
            saap_h_2 = rep2_df['Heavy SAAP intensity'].values[0]
            saap_l_1 = rep1_df['Light SAAP intensity'].values[0]
            saap_l_2 = rep2_df['Light SAAP intensity'].values[0]
            saap_1 = rep1_df['Light+Heavy SAAP intensity'].values[0]
            saap_2 = rep2_df['Light+Heavy SAAP intensity'].values[0]
            saap_hl_1 = rep1_df['H/L SAAP intensity'].values[0]
            saap_hl_2 = rep2_df['H/L SAAP intensity'].values[0]
            raas_1 = rep1_df['Light+Heavy RAAS'].values[0]
            raas_2 = rep2_df['Light+Heavy RAAS'].values[0]

            rep_data_df.loc[i, 'BP_H_time'+str(j+1)+'_rep1'] = bp_h_1
            rep_data_df.loc[i, 'BP_H_time'+str(j+1)+'_rep2'] = bp_h_2
            rep_data_df.loc[i, 'BP_L_time'+str(j+1)+'_rep1'] = bp_l_1
            rep_data_df.loc[i, 'BP_L_time'+str(j+1)+'_rep2'] = bp_l_2
            rep_data_df.loc[i, 'BP_time'+str(j+1)+'_rep1'] = bp_1
            rep_data_df.loc[i, 'BP_time'+str(j+1)+'_rep2'] = bp_2
            rep_data_df.loc[i, 'SAAP_H_time'+str(j+1)+'_rep1'] = saap_h_1
            rep_data_df.loc[i, 'SAAP_H_time'+str(j+1)+'_rep2'] = saap_h_2
            rep_data_df.loc[i, 'SAAP_L_time'+str(j+1)+'_rep1'] = saap_l_1
            rep_data_df.loc[i, 'SAAP_L_time'+str(j+1)+'_rep2'] = saap_l_2
            rep_data_df.loc[i, 'SAAP_time'+str(j+1)+'_rep1'] = saap_1
            rep_data_df.loc[i, 'SAAP_time'+str(j+1)+'_rep2'] = saap_2
            rep_data_df.loc[i, 'BP_HL_time'+str(j+1)+'_rep1'] = bp_hl_1
            rep_data_df.loc[i, 'BP_HL_time'+str(j+1)+'_rep2'] = bp_hl_2
            rep_data_df.loc[i, 'SAAP_HL_time'+str(j+1)+'_rep1'] = saap_hl_1
            rep_data_df.loc[i, 'SAAP_HL_time'+str(j+1)+'_rep2'] = saap_hl_2
            rep_data_df.loc[i, 'RAAS_time'+str(j+1)+'_rep1'] = raas_1
            rep_data_df.loc[i, 'RAAS_time'+str(j+1)+'_rep2'] = raas_2

            rep_data_df.loc[i, 'BP_H_mean_time'+str(j+1)] = np.nanmean([bp_h_1,bp_h_2])
            rep_data_df.loc[i, 'BP_L_mean_time'+str(j+1)] = np.nanmean([bp_l_1,bp_l_2])
            rep_data_df.loc[i, 'BP_mean_time'+str(j+1)] = np.nanmean([bp_1,bp_2])
            rep_data_df.loc[i, 'SAAP_H_mean_time'+str(j+1)] = np.nanmean([saap_h_1,saap_h_2])
            rep_data_df.loc[i, 'SAAP_L_mean_time'+str(j+1)] = np.nanmean([saap_l_1,saap_l_2])
            rep_data_df.loc[i, 'SAAP_mean_time'+str(j+1)] = np.nanmean([saap_1,saap_2])
            rep_data_df.loc[i, 'BP_HL_mean_time'+str(j+1)] = np.nanmean([bp_hl_1,bp_hl_2])
            rep_data_df.loc[i, 'SAAP_HL_mean_time'+str(j+1)] = np.nanmean([saap_hl_1,saap_hl_2])
            rep_data_df.loc[i, 'RAAS_mean_time'+str(j+1)] = np.nanmean([raas_1,raas_2])

        rep_data_df.loc[i, 'N_timepoints'] = len([x for x in rep_data_df.loc[i,[c for c in rep_data_df.columns if 'RAAS_mean_time' in c]] if ~np.isnan(x)])

        bp_h_vals = rep_data_df.loc[i,[x for x in rep_data_df.columns if 'BP_H_mean_time' in x]].to_list()
        rep_data_df.loc[i, 'BP_H_mean'] = np.nanmean(bp_h_vals)
        rep_data_df.loc[i, 'BP_H_std'] = np.nanstd(bp_h_vals)
        rep_data_df.loc[i, 'BP_H_CV'] = np.nanstd(bp_h_vals)/np.nanmean(bp_h_vals)
        bp_l_vals = rep_data_df.loc[i,[x for x in rep_data_df.columns if 'BP_L_mean_time' in x]].to_list()
        rep_data_df.loc[i, 'BP_L_mean'] = np.nanmean(bp_l_vals)
        rep_data_df.loc[i, 'BP_L_std'] = np.nanstd(bp_l_vals)
        rep_data_df.loc[i, 'BP_L_CV'] = np.nanstd(bp_l_vals)/np.nanmean(bp_l_vals)
        bp_vals = rep_data_df.loc[i,[x for x in rep_data_df.columns if 'BP_mean_time' in x]].to_list()
        rep_data_df.loc[i, 'BP_mean'] = np.nanmean(bp_vals)
        rep_data_df.loc[i, 'BP_std'] = np.nanstd(bp_vals)
        rep_data_df.loc[i, 'BP_CV'] = np.nanstd(bp_vals)/np.nanmean(bp_vals)
        saap_h_vals = rep_data_df.loc[i,[x for x in rep_data_df.columns if 'SAAP_H_mean_time' in x]].to_list()
        rep_data_df.loc[i, 'SAAP_H_mean'] = np.nanmean(saap_h_vals)
        rep_data_df.loc[i, 'SAAP_H_std'] = np.nanstd(saap_h_vals)
        rep_data_df.loc[i, 'SAAP_H_CV'] = np.nanstd(saap_h_vals)/np.nanmean(saap_h_vals)
        saap_l_vals = rep_data_df.loc[i,[x for x in rep_data_df.columns if 'SAAP_L_mean_time' in x]].to_list()
        rep_data_df.loc[i, 'SAAP_L_mean'] = np.nanmean(saap_l_vals)
        rep_data_df.loc[i, 'SAAP_L_std'] = np.nanstd(saap_l_vals)
        rep_data_df.loc[i, 'SAAP_L_CV'] = np.nanstd(saap_l_vals)/np.nanmean(saap_l_vals)
        saap_vals = rep_data_df.loc[i,[x for x in rep_data_df.columns if 'SAAP_mean_time' in x]].to_list()
        rep_data_df.loc[i, 'SAAP_mean'] = np.nanmean(saap_vals)
        rep_data_df.loc[i, 'SAAP_std'] = np.nanstd(saap_vals)
        rep_data_df.loc[i, 'SAAP_CV'] = np.nanstd(saap_vals)/np.nanmean(saap_vals)
        bp_hl_vals = rep_data_df.loc[i,[x for x in rep_data_df.columns if 'BP_HL_mean_time' in x]].to_list()
        rep_data_df.loc[i, 'BP_HL_mean'] = np.nanmean(bp_hl_vals)
        rep_data_df.loc[i, 'BP_HL_std'] = np.nanstd(bp_hl_vals)
        rep_data_df.loc[i, 'BP_HL_CV'] = np.nanstd(bp_hl_vals)/np.nanmean(bp_hl_vals)
        saap_hl_vals = rep_data_df.loc[i,[x for x in rep_data_df.columns if 'SAAP_HL_mean_time' in x]].to_list()
        rep_data_df.loc[i, 'SAAP_HL_mean'] = np.nanmean(saap_hl_vals)
        rep_data_df.loc[i, 'SAAP_HL_std'] = np.nanstd(saap_hl_vals)
        rep_data_df.loc[i, 'SAAP_HL_CV'] = np.nanstd(saap_hl_vals)/np.nanmean(saap_hl_vals)
        raas_vals = [10**x for x in rep_data_df.loc[i,[x for x in rep_data_df.columns if 'RAAS_mean_time' in x]].to_list()]
        rep_data_df.loc[i, 'RAAS_mean'] = np.log10(np.nanmean(raas_vals))
        rep_data_df.loc[i, 'RAAS_std'] = np.nanstd(raas_vals)
        rep_data_df.loc[i, 'RAAS_CV'] = np.nanstd(raas_vals)/np.nanmean(raas_vals)

    rep_data_df_list.append(rep_data_df)

In [ ]:
# add columns with log of the abundance values for all heavy and light peptides and save file

new_rep_data_df_list = []
for i,rep_data_df in enumerate(rep_data_df_list):
    celltype = rep_data_df['Cell type'].values[0]
    print(celltype)
    cell_meta = metadata.loc[metadata['cell type']==celltype]
    timepoints = list(set(cell_meta['time point'].to_list()))
    timepoints = [str(y)+'h' for y in sorted([int(x.split('h')[0]) for x in timepoints])]
    for j,tp in enumerate(timepoints): 
        print(tp)
        rep_data_df['BP_H_time'+str(j+1)+'_rep1_log'] = [np.log10(x) for x in rep_data_df['BP_H_time'+str(j+1)+'_rep1'].to_list()]
        rep_data_df['BP_H_time'+str(j+1)+'_rep2_log'] = [np.log10(x) for x in rep_data_df['BP_H_time'+str(j+1)+'_rep2'].to_list()]
        rep_data_df['BP_L_time'+str(j+1)+'_rep1_log'] = [np.log10(x) for x in rep_data_df['BP_L_time'+str(j+1)+'_rep1'].to_list()]
        rep_data_df['BP_L_time'+str(j+1)+'_rep2_log'] = [np.log10(x) for x in rep_data_df['BP_L_time'+str(j+1)+'_rep2'].to_list()]
        rep_data_df['BP_time'+str(j+1)+'_rep1_log'] = [np.log10(x) for x in rep_data_df['BP_time'+str(j+1)+'_rep1'].to_list()]
        rep_data_df['BP_time'+str(j+1)+'_rep2_log'] = [np.log10(x) for x in rep_data_df['BP_time'+str(j+1)+'_rep2'].to_list()]
        rep_data_df['SAAP_H_time'+str(j+1)+'_rep1_log'] = [np.log10(x) for x in rep_data_df['SAAP_H_time'+str(j+1)+'_rep1'].to_list()]
        rep_data_df['SAAP_H_time'+str(j+1)+'_rep2_log'] = [np.log10(x) for x in rep_data_df['SAAP_H_time'+str(j+1)+'_rep2'].to_list()]
        rep_data_df['SAAP_L_time'+str(j+1)+'_rep1_log'] = [np.log10(x) for x in rep_data_df['SAAP_L_time'+str(j+1)+'_rep1'].to_list()]
        rep_data_df['SAAP_L_time'+str(j+1)+'_rep2_log'] = [np.log10(x) for x in rep_data_df['SAAP_L_time'+str(j+1)+'_rep2'].to_list()]
        rep_data_df['SAAP_time'+str(j+1)+'_rep1_log'] = [np.log10(x) for x in rep_data_df['SAAP_time'+str(j+1)+'_rep1'].to_list()]
        rep_data_df['SAAP_time'+str(j+1)+'_rep2_log'] = [np.log10(x) for x in rep_data_df['SAAP_time'+str(j+1)+'_rep2'].to_list()]
        rep_data_df['BP_H_time'+str(j+1)+'_mean_log'] = [np.log10(np.nanmean([row['BP_H_time'+str(j+1)+'_rep1'], row['BP_H_time'+str(j+1)+'_rep2']])) for i,row in rep_data_df.iterrows()]
        rep_data_df['BP_L_time'+str(j+1)+'_mean_log'] = [np.log10(np.nanmean([row['BP_L_time'+str(j+1)+'_rep1'], row['BP_L_time'+str(j+1)+'_rep2']])) for i,row in rep_data_df.iterrows()]
        rep_data_df['BP_time'+str(j+1)+'_mean_log'] = [np.log10(np.nanmean([row['BP_time'+str(j+1)+'_rep1'], row['BP_time'+str(j+1)+'_rep2']])) for i,row in rep_data_df.iterrows()]
        rep_data_df['SAAP_H_time'+str(j+1)+'_mean_log'] = [np.log10(np.nanmean([row['SAAP_H_time'+str(j+1)+'_rep1'], row['SAAP_H_time'+str(j+1)+'_rep2']])) for i,row in rep_data_df.iterrows()]
        rep_data_df['SAAP_L_time'+str(j+1)+'_mean_log'] = [np.log10(np.nanmean([row['SAAP_L_time'+str(j+1)+'_rep1'], row['SAAP_L_time'+str(j+1)+'_rep2']])) for i,row in rep_data_df.iterrows()]
        rep_data_df['SAAP_time'+str(j+1)+'_mean_log'] = [np.log10(np.nanmean([row['SAAP_time'+str(j+1)+'_rep1'], row['SAAP_time'+str(j+1)+'_rep2']])) for i,row in rep_data_df.iterrows()]

        rep_data_df['BP_HL_time'+str(j+1)+'_rep1_log'] = [np.log10(x) for x in rep_data_df['BP_HL_time'+str(j+1)+'_rep1'].to_list()]
        rep_data_df['BP_HL_time'+str(j+1)+'_rep2_log'] = [np.log10(x) for x in rep_data_df['BP_HL_time'+str(j+1)+'_rep2'].to_list()]
        rep_data_df['SAAP_HL_time'+str(j+1)+'_rep1_log'] = [np.log10(x) for x in rep_data_df['SAAP_HL_time'+str(j+1)+'_rep1'].to_list()]
        rep_data_df['SAAP_HL_time'+str(j+1)+'_rep2_log'] = [np.log10(x) for x in rep_data_df['SAAP_HL_time'+str(j+1)+'_rep2'].to_list()]
        rep_data_df['BP_HL_time'+str(j+1)+'_rep1_log+1'] = [np.log10(x+1) for x in rep_data_df['BP_HL_time'+str(j+1)+'_rep1'].to_list()]
        rep_data_df['BP_HL_time'+str(j+1)+'_rep2_log+1'] = [np.log10(x+1) for x in rep_data_df['BP_HL_time'+str(j+1)+'_rep2'].to_list()]
        rep_data_df['SAAP_HL_time'+str(j+1)+'_rep1_log+1'] = [np.log10(x+1) for x in rep_data_df['SAAP_HL_time'+str(j+1)+'_rep1'].to_list()]
        rep_data_df['SAAP_HL_time'+str(j+1)+'_rep2_log+1'] = [np.log10(x+1) for x in rep_data_df['SAAP_HL_time'+str(j+1)+'_rep2'].to_list()]

        rep_data_df['BP_HL_time'+str(j+1)+'_mean_log'] = [np.log10(np.nanmean([row['BP_HL_time'+str(j+1)+'_rep1'], row['BP_HL_time'+str(j+1)+'_rep2']])) for i,row in rep_data_df.iterrows()]
        rep_data_df['SAAP_HL_time'+str(j+1)+'_mean_log'] = [np.log10(np.nanmean([row['SAAP_HL_time'+str(j+1)+'_rep1'], row['SAAP_HL_time'+str(j+1)+'_rep2']])) for i,row in rep_data_df.iterrows()]
        rep_data_df['BP_HL_time'+str(j+1)+'_mean_log+1'] = [np.log10(np.nanmean([row['BP_HL_time'+str(j+1)+'_rep1'], row['BP_HL_time'+str(j+1)+'_rep2']])+1) for i,row in rep_data_df.iterrows()]
        rep_data_df['SAAP_HL_time'+str(j+1)+'_mean_log+1'] = [np.log10(np.nanmean([row['SAAP_HL_time'+str(j+1)+'_rep1'], row['SAAP_HL_time'+str(j+1)+'_rep2']])+1) for i,row in rep_data_df.iterrows()]
    new_rep_data_df_list.append(rep_data_df)
    
rep_data_df_all = pd.concat(rep_data_df_list)
rep_data_df.replace(np.inf, np.nan, inplace=True)
rep_data_df.replace(-np.inf, np.nan, inplace=True)
rep_data_df_all.to_csv(aa_subs_dir+'Median_normalized_Replicate_data_all_celltypes.csv')

In [1]:
# compute degradation rates 

def get_model_params(x,y):
    """ 
    Generate linear regression model to fit peptide abundance data to time points
    input: x - timepoints; y - abundance data (or H/L to compute degradation rates)
    output: slope of regression (this is the degradation rate when fitting H/L), R^2 of the fit, and predicted y values
    """
    x = np.array(x).reshape(-1,1)
    y = np.array(y)
    model = LinearRegression(fit_intercept=False)
    model.fit(x, y)
    y_pred = model.predict(x)
    slope = model.coef_[0]
    score = model.score(x,y)
    return(slope, score, y_pred)

In [ ]:
# compute degradation rates of heavy and light and H/L ratio 

bp_deg_rates = []
saap_deg_rates = []
bp_deg_r2_list = []
saap_deg_r2_list = []
bp_deg_pred_list = []
saap_deg_pred_list = []
n_bp_times_in_deg = []
n_saap_times_in_deg = []

bp_light_slope_list = []
saap_light_slope_list = []
bp_light_r2_list = []
saap_light_r2_list = []
bp_light_pred_list = []
saap_light_pred_list = []

bp_heavy_slope_list = []
saap_heavy_slope_list = []
bp_heavy_r2_list = []
saap_heavy_r2_list = []
bp_heavy_pred_list = []
saap_heavy_pred_list = []

i=0
row = rep_data_df_all.loc[i]

for i,row in rep_data_df_all.iterrows():
    if i%1000==0:
        print(i)
    celltype = row['Cell type']
    x = ordered_times[celltype]

    bp_hl = [row['BP_HL_time'+str(j)+'_mean_log+1'] for j in range(1,5)]
    x_bp_hl = [x[j] for j in range(len(x)) if ~np.isnan(bp_hl[j])]
    bp_hl = [x for x in bp_hl if ~np.isnan(x)]
    n_bp = len(bp_hl)

    saap_hl = [row['SAAP_HL_time'+str(j)+'_mean_log+1'] for j in range(1,5)]
    x_saap_hl = [x[j] for j in range(len(x)) if ~np.isnan(saap_hl[j])]
    saap_hl = [x for x in saap_hl if ~np.isnan(x)]
    n_saap = len(saap_hl)

    bp_deg_slope = saap_deg_slope = np.nan
    bp_deg_r2 = saap_deg_r2 = np.nan
    bp_deg_pred = saap_deg_pred = np.nan
    if len(bp_hl)>0:
        bp_deg_slope, bp_deg_r2, bp_deg_pred = get_model_params(x_bp_hl, bp_hl)
    if len(saap_hl)>0:
        saap_deg_slope, saap_deg_r2, saap_deg_pred = get_model_params(x_saap_hl, saap_hl)

    bp_deg_rates.append(bp_deg_slope)
    saap_deg_rates.append(saap_deg_slope)
    bp_deg_r2_list.append(bp_deg_r2)
    saap_deg_r2_list.append(saap_deg_r2)
    bp_deg_pred_list.append(bp_deg_pred)
    saap_deg_pred_list.append(saap_deg_pred)
    n_bp_times_in_deg.append(n_bp)
    n_saap_times_in_deg.append(n_saap)


    bp_light = [row['BP_L_time'+str(j)+'_mean_log'] for j in range(1,5)]
    x_bp_light = [x[j] for j in range(len(x)) if ~np.isnan(bp_light[j])]
    bp_light = [x for x in bp_light if ~np.isnan(x)]

    saap_light = [row['SAAP_L_time'+str(j)+'_mean_log'] for j in range(1,5)]
    x_saap_light = [x[j] for j in range(len(x)) if ~np.isnan(saap_light[j])]
    saap_light = [x for x in saap_light if ~np.isnan(x)]

    bp_light_slope = saap_light_slope = np.nan
    bp_light_r2 = saap_light_r2 = np.nan
    bp_light_pred = saap_light_pred = np.nan
    if len(bp_light)>0:
        bp_light_slope, bp_light_r2, bp_light_pred = get_model_params(x_bp_light, bp_light)
    if len(saap_light)>0:
        saap_light_slope, saap_light_r2, saap_light_pred = get_model_params(x_saap_light, saap_light)

    bp_light_slope_list.append(bp_light_slope)
    saap_light_slope_list.append(saap_light_slope)
    bp_light_r2_list.append(bp_light_r2)
    saap_light_r2_list.append(saap_light_r2)
    bp_light_pred_list.append(bp_light_pred)
    saap_light_pred_list.append(saap_light_pred)


    bp_heavy = [row['BP_H_time'+str(j)+'_mean_log'] for j in range(1,5)]
    x_bp_heavy = [x[j] for j in range(len(x)) if ~np.isnan(bp_heavy[j])]
    bp_heavy = [x for x in bp_heavy if ~np.isnan(x)]

    saap_heavy = [row['SAAP_H_time'+str(j)+'_mean_log'] for j in range(1,5)]
    x_saap_heavy = [x[j] for j in range(len(x)) if ~np.isnan(saap_heavy[j])]
    saap_heavy = [x for x in saap_heavy if ~np.isnan(x)]

    bp_heavy_slope = saap_heavy_slope = np.nan
    bp_heavy_r2 = saap_heavy_r2 = np.nan
    bp_heavy_pred = saap_heavy_pred = np.nan
    if len(bp_heavy)>0:
        bp_heavy_slope, bp_heavy_r2, bp_heavy_pred = get_model_params(x_bp_heavy, bp_heavy)
    if len(saap_heavy)>0:
        saap_heavy_slope, saap_heavy_r2, saap_heavy_pred = get_model_params(x_saap_heavy, saap_heavy)

    bp_heavy_slope_list.append(bp_heavy_slope)
    saap_heavy_slope_list.append(saap_heavy_slope)    
    bp_heavy_r2_list.append(bp_heavy_r2)
    saap_heavy_r2_list.append(saap_heavy_r2)
    bp_heavy_pred_list.append(bp_heavy_pred)
    saap_heavy_pred_list.append(saap_heavy_pred)
    
rep_data_df_all['BP_deg_rate'] = bp_deg_rates
rep_data_df_all['SAAP_deg_rate'] = saap_deg_rates
rep_data_df_all['BP_deg_rate_log'] = [np.log10(x) for x in rep_data_df_all['BP_deg_rate'].to_list()]
rep_data_df_all['SAAP_deg_rate_log'] = [np.log10(x) for x in rep_data_df_all['SAAP_deg_rate'].to_list()]
rep_data_df_all['BP_N_timepoints_in_deg'] = n_bp_times_in_deg
rep_data_df_all['SAAP_N_timepoints_in_deg'] = n_saap_times_in_deg

rep_data_df_all['BP_deg_r2'] = bp_deg_r2_list
rep_data_df_all['SAAP_deg_r2'] = saap_deg_r2_list
rep_data_df_all['BP_deg_predicted'] = bp_deg_pred_list
rep_data_df_all['SAAP_deg_predicted'] = saap_deg_pred_list

rep_data_df_all['BP_light_slope'] = bp_light_slope_list
rep_data_df_all['SAAP_light_slope'] = saap_light_slope_list
rep_data_df_all['BP_light_r2'] = bp_light_r2_list
rep_data_df_all['SAAP_light_r2'] = saap_light_r2_list
rep_data_df_all['BP_light_predicted'] = bp_light_pred_list
rep_data_df_all['SAAP_light_predicted'] = saap_light_pred_list

rep_data_df_all['BP_heavy_slope'] = bp_heavy_slope_list
rep_data_df_all['SAAP_heavy_slope'] = saap_heavy_slope_list
rep_data_df_all['BP_heavy_r2'] = bp_heavy_r2_list
rep_data_df_all['SAAP_heavy_r2'] = saap_heavy_r2_list
rep_data_df_all['BP_heavy_predicted'] = bp_heavy_pred_list
rep_data_df_all['SAAP_heavy_predicted'] = saap_heavy_pred_list

# compute ratio of SAAP degradation rate to BP degradation rate 
rep_data_df_all['SAAP_BP_deg_rate'] = [row['SAAP_deg_rate']/row['BP_deg_rate'] for i,row in rep_data_df_all.iterrows()]
rep_data_df_all['SAAP_BP_deg_rate_log'] = [np.log2(x) for x in rep_data_df_all['SAAP_BP_deg_rate'].to_list()]

rep_data_df_all.to_csv(aa_subs_dir+'Median_normalized_Replicate_data_all_celltypes.csv')

In [ ]:
# plot ratio of SAAP degradation to BP degradation vs RAAS for given cell type. Figure 2l, extended data figure 3 m,n.

celltype = 'Hepatocytes'
rep_data_df = rep_data_df_all.loc[rep_data_df_all['Cell type']==celltype]
#rep_data_df = rep_data_df.loc[(rep_data_df['BP_N_timepoints_in_deg']>2) & (rep_data_df['SAAP_N_timepoints_in_deg']>2)]
rep_data_df.dropna(inplace=True, subset=['SAAP_BP_deg_rate_log', 'RAAS_mean'])


sns.set_style('whitegrid')
fig,ax = plt.subplots(figsize=(3,3))

subset1_df = rep_data_df.loc[rep_data_df['CPTAC SAAP']==False]
subset2_df = rep_data_df.loc[rep_data_df['CPTAC SAAP']==True]

sns.scatterplot(data=rep_data_df, x='RAAS_mean', y='SAAP_BP_deg_rate_log', s=30, alpha=0, hue='CPTAC SAAP', palette=['#AAAAAA', 'k'])#, palette='RdBu_r')
sns.scatterplot(data=subset1_df, x='RAAS_mean', y='SAAP_BP_deg_rate_log', s=30, alpha=0.8, color='#AAAAAA')#, palette='RdBu_r')
sns.scatterplot(data=subset2_df, x='RAAS_mean', y='SAAP_BP_deg_rate_log', s=30, alpha=0.8, color='k')#, palette='RdBu_r')

#scatter = plt.scatter(data=hep_data, x='Light+Heavy RAAS', y='RTO', s=30, alpha=0.5, edgecolor='w', linewidth=0.7)
vect2 = [float(x) for x in rep_data_df['SAAP_BP_deg_rate_log'].values]
vect1 = [float(x) for x in rep_data_df['RAAS_mean'].values]
y_vals = get_pred_y(vect1,vect2)
plt.plot(vect1, y_vals, '--r')

vect1_2 = subset1_df['SAAP_BP_deg_rate_log'].values
vect1_1 = subset1_df['RAAS_mean'].values
y1_vals = get_pred_y(vect1_1, vect1_2)
#plt.plot(vect1_1, y1_vals, '--', color='#555555')

vect2_2 = subset2_df['SAAP_BP_deg_rate_log'].values
vect2_1 = subset2_df['RAAS_mean'].values
y2_vals = get_pred_y(vect2_1,vect2_2)
#plt.plot(vect2_1, y2_vals, '--', color='k')

r,p = sp.stats.pearsonr(vect1, vect2)
r1,p1 = sp.stats.pearsonr(vect1_1, vect1_2)
r2,p2 = sp.stats.pearsonr(vect2_1, vect2_2)
print(r,p)
print(r1,p1)
print(r2,p2)

ax.annotate('r$=-$'+str(np.round(np.abs(r),2)), xy=(1.05,0.7), xycoords='axes fraction', fontsize=12, color='r')
ax.annotate('p$<$10$^{-10}$', xy=(1.05,0.6), xycoords='axes fraction', fontsize=12,color='r')
#ax.annotate('p$=$0.95', xy=(1.05,0.6), xycoords='axes fraction', fontsize=12, color='r')

ax.annotate('r$=-$'+str(np.round(np.abs(r2),2)), xy=(1.05,0.45), xycoords='axes fraction', fontsize=12, color='k')
ax.annotate('p$<$10$^{-4}$', xy=(1.05,0.35), xycoords='axes fraction', fontsize=12, color='k')
#ax.annotate('p$=$0.06', xy=(1.05,0.35), xycoords='axes fraction', fontsize=12, color='k')

ax.annotate('r$=-$'+str(np.round(np.abs(r1),2)), xy=(1.05,0.2), xycoords='axes fraction', fontsize=12, color='#555555')
ax.annotate('p$<$10$^{-4}$', xy=(1.05,0.1), xycoords='axes fraction', fontsize=12, color='#555555')
#ax.annotate('p$=$0.73', xy=(1.05,0.1), xycoords='axes fraction', fontsize=12, color='#555555')

handles = [Line2D([0], [0], marker='o', color='w', markerfacecolor='k', markersize=8), Line2D([0], [0], marker='o', color='w', markerfacecolor='#555555', markersize=8, alpha=0.8)]
plt.legend(fontsize=12,handles=handles, labels=['CPTAC+Healthy', 'Savitski, $\it{et\ al.}$'], handletextpad=0.1,
          bbox_to_anchor=(1.75,1.05), frameon=False, loc='upper right')

ax.tick_params('both', labelsize=14)
plt.ylabel('log$_{2}$('+r'$\alpha_{SAAP}$'+'$/$'+r'$\alpha_{BP}$'+')', fontsize=15)
plt.xlabel('log$_{10}$(RAAS)', fontsize=15)

plt.title(celltype, fontsize=14)
#plt.title('B Cells', fontsize=14)

plt.savefig(outdir+'figures/'+celltype+'_log2_SAAP_BP_deg_vs_RAAS.png', dpi=300, bbox_inches='tight')

In [ ]:
# plot example of H/L ratios relative to time point to illustrate computation. Figure 2k


# filter rep_data_df_all for peptides with no missing data, high R2, high RAAS and low SAAP degradation to find example
rep_data_df = rep_data_df_all.loc[rep_data_df_all['Cell type']=='Hepatocytes']
rep_data_df = rep_data_df.loc[(rep_data_df['SAAP_N_timepoints_in_deg']==4) & (rep_data_df['BP_N_timepoints_in_deg']==4)]
rep_data_df = rep_data_df.loc[(rep_data_df['SAAP_deg_r2']>=0.5) & (rep_data_df['BP_deg_r2']>=0.5)]
example_df = rep_data_df.loc[(rep_data_df['SAAP_BP_deg_rate_log']<0) & (rep_data_df['RAAS_mean']>1)]
cell_quant_data = quant_df.loc[quant_df['Cell type']=='Hepatocytes']

example_row = example_df.loc[[i for i,row in example_df.iterrows() if row['SAAP']=='LFSNGQDVSNK']]

# extract data for plot. convert log10 to ln. 
saap = example_row['SAAP'].values[0]
bp = example_row['BP'].values[0]
aas = example_row['AAS'].values[0]
raas = example_row['RAAS_mean'].values[0]
saap_deg = np.log(10**(example_row['SAAP_deg_rate'].values[0]))
bp_deg = np.log(10**(example_row['BP_deg_rate'].values[0]))
saap_bp_deg = saap_deg/bp_deg
saap_hl_pred = [np.log(10**x) for x in example_row['SAAP_deg_predicted'].values[0]]
bp_hl_pred = [np.log(10**x) for x in example_row['BP_deg_predicted'].values[0]]
row_quant = cell_quant_data.loc[(cell_quant_data['SAAP']==saap) & (cell_quant_data['BP']==bp)]
tps = sorted(list(set(row_quant['Time'])))
saap_light_tps = []
saap_heavy_tps = []
bp_light_tps = []
bp_heavy_tps = []
saap_hl_tps = []
bp_hl_tps = []

for tp in tps:
    tp_quant = row_quant.loc[row_quant['Time']==tp]
    rep1_row = tp_quant.loc[tp_quant['Replicate']==1]
    rep2_row = tp_quant.loc[tp_quant['Replicate']==2]
    time = rep1_row['Time'].values[0]
    hl_saap = np.nanmean([np.log(rep1_row['H/L SAAP intensity'].values[0]+1), np.log(rep2_row['H/L SAAP intensity'].values[0]+1)])
    hl_bp = np.nanmean([np.log(rep1_row['H/L BP intensity'].values[0]+1), np.log(rep2_row['H/L BP intensity'].values[0]+1)])
    saap_hl_tps.append(hl_saap)
    bp_hl_tps.append(hl_bp)

    
# plot figure 2k
fig,ax = plt.subplots(figsize=(3,3))
tps = [0]+tps
saap_hl_tps = [0] + saap_hl_tps
bp_hl_tps = [0] + bp_hl_tps
bp_hl_pred = [0] + bp_hl_pred

sns.regplot(x=tps, y=saap_hl_tps, label='SAAP_H/L', ci=None, scatter_kws={'s':50})
sns.regplot(x=tps, y=bp_hl_tps, label='BP_H/L', ci=None, scatter_kws={'s':50}, line_kws={'linewidth':0})
sns.lineplot(x=tps, y=bp_hl_pred, color=colors[1], linewidth=2)
aas_idx = [i for i,x in enumerate(saap) if bp[i]!=x][0]
plt.title(saap[:aas_idx]+r'$\bf(N$'+r'$\rightarrow$'+r'$\bf M)$'+saap[aas_idx+1:], fontsize=14)
ax.tick_params('both', labelsize=13)
plt.ylabel('ln(1+Heavy $/$ Light)', fontsize=14)
plt.xlabel('Time point (hours)', fontsize=14)
handles, labels = ax.get_legend_handles_labels()
plt.legend(handles=handles, labels =['SAAP', 'BP'], bbox_to_anchor=(0.96,0.85), frameon=False, loc='upper left', fontsize=12)
plt.annotate('RAAS$=$'+str(np.round(10**raas,2)), xy=(1.05,0.48), xycoords='axes fraction', fontsize=12)
plt.annotate(r'$\dfrac{\alpha_{BP}}{\alpha_{SAAP}}$'+'$=$'+str(np.round(np.abs(1/saap_bp_deg),2)), xy=(1.05,0.3), xycoords='axes fraction', fontsize=12)
plt.annotate(r'$\alpha_{SAAP}=$'+'\n'+str(np.round(saap_deg_day,2))+' days$^{-1}$',xy=(0.35,0.15), xycoords='axes fraction', color=colors[0], fontsize=12)
plt.annotate(r'$\alpha_{BP}=$'+'\n'+str(np.round(bp_deg_day,2))+' days$^{-1}$',xy=(0.35,0.8), xycoords='axes fraction', color=colors[1], fontsize=12)
plt.savefig(outdir+'saap_bp_hl_example4fig2.pdf', bbox_inches='tight')


In [ ]:
# see which tissues SAAP identified in this dataset were identified in in the healthy human tissue data 
# Supp. Figure 1a

celltype = 'Hepatocytes'
pep_df = pd.read_csv(val_dir+celltype+'/Results/combined_modified_peptide_label_quant.tsv', sep='\t')

fasta = open(savitski_dir+'databases/2024-07-21-decoys-contam-human_MTP_Hepatocytes.fasta.fas', 'r').read()
fasta_entries = [x for x in fasta.split('>')]
all_mtps = [x for x in fasta_entries if re.match('MTP\|', x)] # these come from Savitski DP search 
all_saap = [x for x in fasta_entries if re.match('SAAP_', x)] # these come from CPTAC/label-free DP searches

# get all peptide sequences that were validated by MSFragger search 
mtp_saap_peps = [x for x in allpeps if x in all_mtp_seqs and x in all_saap_seqs]
mtp_peps = [x for x in allpeps if x in all_mtp_seqs and x not in all_saap_seqs]
saap_peps = [x for x in allpeps if x not in all_mtp_seqs and x in all_saap_seqs]
other_peps = [x for x in allpeps if x not in all_mtp_seqs and x not in all_saap_seqs]

val_saap = mtp_saap_peps + saap_peps 
set_tissues = list(set(filt_prec_quant_df['TMT/Tissue'].to_list()))
set_tissues = [x for x in set_tissues if not re.match('S[1-9]', x)]

# generate dataframe with the number of validated SAAP that were found in each type of healthy tissue
plot_rows = []
plot_cols = ['Tissue', 'N SAAP']
for tissue in set_tissues:
    tissue_ct = 0
    for saap in val_saap:
        tissues = filt_prec_quant_df.loc[filt_prec_quant_df['SAAP']==saap, 'TMT/Tissue'].to_list()
        if tissue in tissues:
            tissue_ct+=1
    plot_rows.append([tissue, tissue_ct])
plot_df = pd.DataFrame(plot_rows, columns=plot_cols)
plot_df.sort_values('N SAAP', ascending=False, inplace=True)

fig,ax = plt.subplots(figsize=(8,4))
sns.barplot(data=plot_df, x='Tissue', y='N SAAP')
plt.xticks(rotation=90);
ax.tick_params('both', labelsize=13)
plt.ylabel('N SAAP', fontsize=14)
plt.xlabel('')

plt.savefig(outdir+'MSFragger_validation_SAAP_in_healthy_tissues.pdf', bbox_inches='tight')

In [ ]:
# generate dataframe for plot of how many candidate SAAP from Savitski DP search were validated by MSFragger 
# supp. figure 1b

plot_rows = []
plot_cols = ['Cell type', 'Sample', '% peptides', '% type']

for celltype in cell_types:
    qmtp_dict = pickle.load(open(aa_subs_dir+'qMTP_dict_'+celltype+'.p','rb'))
    cand_seqs = list(set([x for y in list(qmtp_dict['mistranslated sequence'].values()) for x in y]))
    all_seqs = cand_seqs + cptac_saaps
    cell_df = quant_df.loc[quant_df['Cell type']==celltype]
    cell_times = ordered_times[celltype]
    reps = [1,2]

    all_val_seqs = list(set(cell_df['SAAP'].to_list()))
    for time in cell_times:
        for rep in reps:
            sample = str(time)+'_'+str(rep)
            sample_df = cell_df.loc[(cell_df['Time']==time) & (cell_df['Replicate']==rep)]
            sample_df.dropna(inplace=True, subset=['Light+Heavy SAAP intensity'])
            val_seqs = list(set(sample_df['SAAP'].to_list()))
            pcnt_sav_saap = 100*len([x for x in cand_seqs if x in val_seqs])/len(cand_seqs)
            pcnt_cptac_saap = 100*len([x for x in cptac_saaps if x in val_seqs])/len(cptac_saaps)
            pcnt_all_saap = 100*len([x for x in all_seqs if x in val_seqs])/len(all_seqs)
            plot_rows.append([celltype, sample, pcnt_sav_saap, 'Savitskit SAAP'])
            plot_rows.append([celltype, sample, pcnt_cptac_saap, 'CPTAC SAAP'])
plot_df = pd.DataFrame(plot_rows, columns=plot_cols)

sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize=(3,3))

plot_df = plot_df.loc[plot_df['% type']=='Savitskit SAAP']
sns.boxplot(data=plot_df, x='Cell type', y='% peptides', linewidth=0.8, fliersize=0, dodge=True, saturation=0.8, color='#AAAAAA')
sns.stripplot(data=plot_df, x='Cell type', y='% peptides', linewidth=1, dodge=True, color='#555555')
plt.yticks(fontsize=14)
plt.xticks(rotation=45, fontsize=14)
plt.xlabel('')
plt.ylabel('% validated SAAP', fontsize=15)
plt.savefig(outdir+'figures/percent_validated_boxplot.pdf', bbox_inches='tight')

### Extended data figure 2d,e,f: ionization efficiency normalization

In [ ]:
# ionization efficiencies for peptides and normalization of abundances are computed in decode pipeline quant
# here read in data from MTP_quant_dict.p and plot 

all_ds_mtp_ie = []
all_ds_bp_ie = []

for ds in datasets:
    print(ds)
    data_dir = data_dir_list[datasets.index(ds)]
    quant_dict = pickle.load(open(data_dir+'MTP_quant_dict.p', 'rb'))

    ds_mtp_ie = []
    ds_bp_ie = []
    for q,qdict in quant_dict.items():
        ds_mtp_ie.append(qdict['IE_dict']['MTP_IE'])
        ds_bp_ie.append(qdict['IE_dict']['BP_IE'])
    all_ds_mtp_ie = all_ds_mtp_ie + ds_mtp_ie
    all_ds_bp_ie = all_ds_bp_ie + ds_bp_ie
    
# plot histogram with SAAP and BP ionization efficiencies (Ext. Data Fig. 2d)
fig,ax = plt.subplots(figsize=(4,3))
sns.histplot(all_ds_mtp_ie, alpha=0.8, label='SAAP')
sns.histplot(all_ds_bp_ie, alpha=0.5,color=colors[1], label='BP')
handles, labels = ax.get_legend_handles_labels()
plt.legend(handles=handles[1:], labels=labels[1:])
plt.xlabel('Ionization efficiency', fontsize=14)
plt.ylabel('Count', fontsize=14)
ax.tick_params('both', labelsize=13)
plt.savefig(outdir+'All_peptide_IE_hist.pdf', bbox_inches='tight')
plt.close()

# plot a correlation plot of SAAP to BP ionization efficiency (Ext. Data Fig. 2e)
fig,ax = plt.subplots(figsize=(4,3))
sns.scatterplot(y=all_ds_mtp_ie, x=all_ds_bp_ie, alpha=0.8, s=15, linewidth=0.5)
plt.xlabel('BP ionization efficiency', fontsize=14)
plt.ylabel('SAAP ionization efficiency', fontsize=14)
ax.tick_params('both', labelsize=13)
plt.savefig(outdir+'All_peptide_IE_corr.png', dpi=300, bbox_inches='tight')
plt.close()

# plot a distribution of the fold changes between SAAP and BP ionization efficiencies (Ext. Data Fig. 2f)
ie_fc = [all_ds_mtp_ie[i]/all_ds_bp_ie[i] for i in range(len(ds_mtp_ie))]
fig,ax=plt.subplots(figsize=(4,3))
sns.histplot(ie_fc, bins=20)
plt.xlabel('Ionization fold change', fontsize=14)
plt.ylabel('Count', fontsize=14)
ax.tick_params('both', labelsize=13)
plt.savefig(outdir+'All_saap_bp_IE_FC.pdf', bbox_inches='tight')
plt.close()

### Extended data figure 2a,b,c: N frags, PEP, mass error of high vs low RAAS

In [ ]:
# Ext. Data Fig 2a,b
# generate a dictionary with PEP values and N fragments for all SAAP in all datasets 

paas_pep_dict = {}
for ds in datasets:
    print(ds)
    data_dir = data_dir_list[datasets.index(ds)]
    samples = samples_list[datasets.index(ds)]
    mtp_dict = pickle.load(open(data_dir + 'Ion_validated_MTP_dict.p', 'rb'))
    mtp_quant_dict = pickle.load(open(data_dir + 'MTP_quant_dict.p', 'rb'))

    plot_rows = []
    plot_cols = ['Dataset', 'Sample', 'PAAS', 'PAAS_val_idx', 'DP PEP', 'PAAS PEP', 'PAAS q', 'PAAS Precursor Intensity','BP Precursor Intensity', 'Precursor RAAS', 'N fragments']
    for s in samples:
        s_dict = mtp_dict[s]
        for k, paas in s_dict['mistranslated sequence'].items():
            paas_pep = s_dict['Posterior subs probability'][k]
            paas_q = s_dict['q-value'][k]
            dp_pep = s_dict['DP PEP'][k]
            n_frags = s_dict['fragment_evidence'][k]
            q_dict = [i for i,v in mtp_quant_dict.items() if v['MTP_seq']==paas]
            if len(q_dict)>0:
                q_dict = mtp_quant_dict[q_dict[0]]
                paas_prec = q_dict['MTP_PrecInt'][s]
                bp_prec = q_dict['BP_PrecInt'][s]
                raas = q_dict['Prec_ratio'][s]
                if (~np.isnan(raas)) and (~np.isinf(raas)):
                    plot_rows.append([ds, s, paas, k, dp_pep, paas_pep, paas_q, paas_prec, bp_prec, raas, n_frags])
    plot_df = pd.DataFrame(plot_rows, columns=plot_cols)   
    paas_pep_dict[ds] = plot_df

pickle.dump(paas_pep_dict, open(outdir+'SAAP_PEP_dfs.p', 'wb'))

# create data frame for plot 
all_ds_plot_df = pd.concat([v for k,v in paas_pep_dict.items()])
all_ds_plot_df['RAAS group'] = ['$\geq$0' if raas>=0 else '<0' for i,raas in list(enumerate(all_ds_plot_df['Precursor RAAS'].values))]
all_ds_plot_df = all_ds_plot_df.loc[all_ds_plot_df['PAAS q']<=0.01]

# plot histogram of confidence values stratified by RAAS (Ext. Data Fig. 2a)
fig,ax = plt.subplots(figsize=(4,3))
all_ds_plot_df['-log q'] = [-np.log10(x) for x in all_ds_plot_df['PAAS q']]
high_raas_df = all_ds_plot_df.loc[all_ds_plot_df['Precursor RAAS']>=0]
low_raas_df = all_ds_plot_df.loc[all_ds_plot_df['Precursor RAAS']<0]
sns.kdeplot(data = high_raas_df.loc[~np.isinf(high_raas_df['-log q'])], x='-log q', fill=False, color=colors[1], label='log$_{10}$(RAAS)$\geq$0')
sns.kdeplot(data = low_raas_df.loc[~np.isinf(low_raas_df['-log q'])], x='-log q', fill=False, label='log$_{10}$(RAAS) < 0')
ax.tick_params('both', labelsize=13)
plt.ylabel('Density', fontsize=14)
plt.xlabel('-log q-value', fontsize=14)
plt.legend(fontsize=12)
plt.savefig(outdir+'RAASgroup_logq_hist.pdf', bbox_inches='tight')

# plot histogram of N fragments on substitution site stratified by RAAS (Ext. Data Fig. 2b)
fig,ax = plt.subplots(figsize=(4,3))
sns.kdeplot(data = high_raas_df.loc[~np.isinf(high_raas_df['-log q'])], x='N fragments', fill=False, color=colors[1], label='log$_{10}$(RAAS)$\geq$0')
sns.kdeplot(data = low_raas_df.loc[~np.isinf(low_raas_df['-log q'])], x='N fragments', fill=False, label='log$_{10}$(RAAS) < 0')
ax.tick_params('both', labelsize=13)
plt.ylabel('Density', fontsize=14)
plt.xlabel('# fragments supporting substitution', fontsize=14)
plt.yticks([0,0.03,0.06,0.09,0.12])
plt.legend(fontsize=12)
plt.savefig(outdir+'RAASgroup_Nfrags_hist.pdf', bbox_inches='tight')

In [ ]:
# Ext. Data Fig. 2c

def get_saap_mass_err(saap, s_saap_dict):
    saap_idx = [k for k,v in s_saap_dict['mistranslated sequence'].items() if v==saap]
    err = [s_saap_dict['Mass error (ppm)'][k] for k in saap_idx]
    return(np.median(err))

# create a dictionary with mass errors of SAAP with high and low RAAS
mass_err_dict = {ds:{'High RAAS mass error':[], 'Low RAAS mass error':[]} for ds in datasets}
for ds in datasets:
    data_dir = data_dir_list[datasets.index(ds)]
    ds_prec_df = filt_prec_quant_df.loc[filt_prec_quant_df['Dataset']==ds]   
    saap_dict = pickle.load(open(data_dir+'Ion_validated_MTP_dict.p','rb'))
    samples = samples_list[datasets.index(ds)]
    for s in samples:
        s_saap_dict = saap_dict[s]
        if 'Mass error (ppm)' in s_saap_dict.keys():
            s_prec_df = ds_prec_df.loc[ds_prec_df['TMT/Tissue']==s]
            high_saap = s_prec_df.loc[s_prec_df['RAAS']>=0, 'SAAP'].to_list()
            low_saap = s_prec_df.loc[s_prec_df['RAAS']<0, 'SAAP'].to_list()
            high_errs = [get_saap_mass_err(saap, s_saap_dict) for saap in high_saap]
            low_errs = [get_saap_mass_err(saap, s_saap_dict) for saap in low_saap]
            mass_err_dict[ds]['High RAAS mass error']  = mass_err_dict[ds]['High RAAS mass error'] + high_errs
            mass_err_dict[ds]['Low RAAS mass error']  = mass_err_dict[ds]['Low RAAS mass error'] + low_errs 
pickle.dump(mass_err_dict, open(outdir+'High_v_low_RAAS_mass_error_ppm.p','wb'))

# create dataframe for plot
plot_df_list = []
for ds in datasets:
    print(ds)
    ds_me_dict = mass_err_dict[ds]
    high_errs = ds_me_dict['High RAAS mass error']
    low_errs = ds_me_dict['Low RAAS mass error']
    ds_list = [ds]*len(high_errs+low_errs)
    high_str_list = ['RAAS$\geq$0']*len(high_errs)
    low_str_list = ['RAAS < 0']*len(low_errs)
    ds_plot_df = pd.DataFrame(zip(ds_list, high_errs+low_errs, high_str_list+low_str_list), columns=['Dataset', 'Mass error (ppm)', 'Peptide type'])
    plot_df_list.append(ds_plot_df)
plot_df = pd.concat(plot_df_list)

# plot distributions of mass errors for SAAP with high and low RAAS 
fig,ax = plt.subplots(figsize=(6,3))
sns.boxplot(data=plot_df, x='Dataset', hue='Peptide type', y='Mass error (ppm)', linewidth=1, fliersize=0.5, palette='Greys')
plt.ylabel('Mass error (ppm)', fontsize=14)
ax.tick_params('both', labelsize=13)
plt.xlabel('')
plt.ylim(-4,4)
plt.legend(loc='lower left', fontsize=12, frameon=False, ncol=1)
plt.savefig(outdir+'HighvLow_RAAS_Mass_error_boxplots.png', dpi=300, bbox_inches='tight')

### Figure 2c, Extended Data Figure 2g: Consistency of RAAS values for same substitution detected in different peptides/enzymatic digests

uses tonsil_saap_dict.p generated above in for figure 1h

In [ ]:
other_enz_list = ['Chymotrypsin', 'LysC', 'ArgC', 'GluC']

# get list of SAAPs across digests with the same substitution (using the index in the protein sequence)
prot_aas_idx_pairs = {}
for s, sdict in tonsil_saap_dict.items():
    prot_aas_idx_pairs[s] = []
    pep_ids = sdict['Peptide IDs']
    for p,pid in pep_ids.items():
        prot_aas_idx_pairs[s].append([pid['saap_prot_fasta_idx'], pid['saap_prot_idx']])
et_prot_aas_idx_pairs = [x for y in list(prot_aas_idx_pairs.values()) for x in y if len(x)>0]
saaps_each_pair = {str(pair):[x for x,v in prot_aas_idx_pairs.items() if pair in v] for pair in set_prot_aas_idx_pairs}
saaps_each_pair = {k:v for k,v in saaps_each_pair.items() if len(v)>1}
saap_lists_w_same_aas = []
for k,v in saaps_each_pair.items():
    if v not in saap_lists_w_same_aas:
        saap_lists_w_same_aas.append(v)

# generate dataframe with RAAS values for substitutions found in multiple digests 
plot_rows = []
plot_cols = ['Trypsin RAAS', 'Other RAAS', 'Digest', 'Same sequence', 'SAAP abundance']
for s, sdict in tonsil_saap_dict.items():
    if 'Trypsin' in sdict['Digests']:
        raas_dict = sdict['RAAS']
        trypsin_raas = np.log10(2**raas_dict['Trypsin'])
        saap_abund = sdict['SAAP abundance']['Trypsin']
        # include abundance values to stratify plot by SAAP abundance 
        if saap_abund>=1e10:
            saap_abund=10
        elif saap_abund>=1e9:
            saap_abund=9
        elif saap_abund>=1e8:
            saap_abund=8
        else:
            saap_abund=7
        for enz in other_enz_list:
            if enz in sdict['Digests'] :
                same_seq = True
            if enz in raas_dict:
                raas = np.log10(2**raas_dict[enz])
                plot_rows.append([trypsin_raas, raas, enz, same_seq, saap_abund])

for saap_pair in saap_lists_w_same_aas:
    sdict1 = tonsil_saap_dict[saap_pair[0]]
    sdict2 = tonsil_saap_dict[saap_pair[1]]
    if 'Trypsin' in sdict1['Digests']:
        trypsin_raas = np.log10(2**sdict1['RAAS']['Trypsin'])
        saap_abund = sdict1['SAAP abundance']['Trypsin']
        if saap_abund>=1e10:
            saap_abund=10
        elif saap_abund>=1e9:
            saap_abund=9
        elif saap_abund>=1e8:
            saap_abund=8
        else:
            saap_abund=7
        other_dig = sdict2['Digests']
        for dig in other_dig:
            if dig!='Trypsin':
                other_raas = np.log10(2**sdict2['RAAS'][dig])
                plot_rows.append([trypsin_raas, other_raas, dig, False, saap_abund])
    elif 'Trypsin' in sdict2['Digests']:
        trypsin_raas = np.log10(2**sdict2['RAAS']['Trypsin'])        
        saap_abund = sdict2['SAAP abundance']['Trypsin']
        if saap_abund>=1e10:
            saap_abund=10
        elif saap_abund>=1e9:
            saap_abund=9
        elif saap_abund>=1e8:
            saap_abund=8
        else:
            saap_abund=7
        other_dig = sdict1['Digests']
        for dig in other_dig:
            if dig!='Trypsin':
                other_raas = np.log10(2**sdict1['RAAS'][dig])       
                plot_rows.append([trypsin_raas, other_raas, dig, False, saap_abund])

plot_df = pd.DataFrame(plot_rows, columns=plot_cols)
plot_df = plot_df.replace(np.inf, np.nan)
plot_df = plot_df.replace(-np.inf, np.nan)
plot_df = plot_df.dropna()
plot_df.to_excel(outdir+'Trypsin_vs_other_digest_plot_df.xlsx')

In [ ]:
# plot RAAS of substitution in another digest vs RAAS of substitution in trypsin, colored by SAAP abundance
# Ext. Data Fig. 2g

sns.set_style('whitegrid')
fig,ax=plt.subplots(figsize=(3,3))
sns.scatterplot(data=plot_df, x='Trypsin RAAS', y='Other RAAS', hue='SAAP abundance')
plt.legend(loc='lower left', handletextpad=0, ncol=2, bbox_to_anchor=(0,1), frameon=False, fontsize=11, title='log$_{10}$(SAAP abundance)', title_fontsize=11)

r,p = sp.stats.pearsonr(plot_df['Trypsin RAAS'].to_list(), plot_df['Other RAAS'].to_list())
ax.text(-4.8,0.6, 'r = '+str(np.round(r, 2)), fontsize=11)
ax.text(-4.8,0.1, 'p < 10$^{-11}$', fontsize=11)

plt.ylabel('Other protease log$_{10}$(RAAS)', fontsize=13)
plt.xlabel('Trypsin log$_{10}$(RAAS)', fontsize=13)
ax.tick_params('both', labelsize=12)
plt.savefig(outdir+'tonsil_trypsinRAAS_vs_otherRAAS_scatter_saapabundcolor_newvalsearch.pdf', bbox_inches='tight')

In [ ]:
# plot RAAS of substitution in another digest vs RAAS of substitution in trypsin, colored by digest
# Fig. 2c

sns.set_style('whitegrid')
fig,ax=plt.subplots(figsize=(3,3))
sns.scatterplot(data=plot_df, x='Trypsin RAAS', y='Other RAAS', hue='Digest')
handles, labels = ax.get_legend_handles_labels()  
plt.legend(handles=handles, labels=labels, loc='center', bbox_to_anchor=(0.5,1.08),
           handletextpad=-0.3, ncol=2, frameon=False, fontsize=11, columnspacing=0.1)#, title='log$_{10}$(SAAP abundance)')
ax.annotate(r'X$:$', xy=(0.25,1.01), xycoords='figure fraction', fontsize=15)

r,p = sp.stats.pearsonr(plot_df['Trypsin RAAS'].to_list(), plot_df['Other RAAS'].to_list())
ax.text(-4.8,1, 'r = '+str(np.round(r, 2)), fontsize=11)
ax.text(-4.8,0.5, 'p < 10$^{-11}$', fontsize=11)
plt.yticks([-4,-2,0,2])

plt.ylabel('Protease X, log$_{10}$(RAAS)', fontsize=14)
plt.xlabel('Trypsin, log$_{10}$(RAAS)', fontsize=14)
ax.tick_params('both', labelsize=13)
plt.savefig(outdir+'tonsil_trypsinRAAS_vs_otherRAAS_scatter_digestseqcolor_newvalsearch.pdf', bbox_inches='tight')

### Extended Data FIgure 2h: Compare abundance of missed cleaved BPs to BPs without missed cleavages


In [ ]:
# get data for plot from Supplemental_Data_2.SAAP_precursor_quant.xlsx

ds_bp_mc_dict = {ds:{} for ds in datasets}
for ds in datasets:
    ds_saap_df = filt_saap_df.loc[filt_saap_df['Dataset']==ds]
    bp_list_all = ds_saap_df['BP'].to_list()
    for i,row in ds_saap_df.iterrows():
        bp = row['BP']
        bp_list = [x for x in bp_list_all if x!=bp]
        mc = list(set([x for x in bp_list if bp in x]))
        if len(mc)>0:
            ds_bp_mc_dict[ds][bp] = mc
            
plot_rows = []
plot_cols = ['Dataset', 'BP', 'Missed cleavage BP', 'BP abundance', 'Missed cleavage BP abundance', 'AAS', 'AAS index', 'Color']
for ds in datasets:
    print(ds)
    ds_bp_mc = ds_bp_mc_dict[ds]
    ds_prec_df = filt_prec_quant_df.loc[filt_prec_quant_df['Dataset']==ds]
    samples = samples_list[datasets.index(ds)]
    for bp, mc_list in ds_bp_mc.items():
        for s in samples:
            s_df = ds_prec_df.loc[ds_prec_df['TMT/Tissue']==s]
            for mc in mc_list:
                if bp in s_df['BP'].to_list() and mc in s_df['BP'].to_list():
                    bp_df = s_df.loc[s_df['BP']==bp]
                    bp_abund = bp_df['BP abundance'].values[0]
                    aas = bp_df['AAS'].values[0]
                    saap = bp_df['SAAP'].values[0]
                    raas = bp_df['RAAS'].values[0]
                    aas_idx = [i for i,x in enumerate(saap) if bp[i]!=x][0]
                    mc_abund = s_df.loc[s_df['BP']==mc, 'BP abundance'].values[0]
                    if raas>0:
                        c = 'RAAS>0'
                    else:
                        c = 'RAAS<0'
                    plot_rows.append([ds, bp, mc, bp_abund, mc_abund, aas, aas_idx, c])
                    
plot_df = pd.DataFrame(plot_rows, columns=plot_cols)

In [ ]:
# plot scatterplot of abundance of BPs with missed cleavages to abundance of BPs

sns.set_style('whitegrid')
fig,ax = plt.subplots(figsize=(3,3))
sns.scatterplot(data=plot_df, x='BP abundance', y='Missed cleavage BP abundance', s=40, alpha=0.9, color='#AAAAAA')
ax.tick_params('both', labelsize=13)
plt.xlabel('BP abundance', fontsize=14)
plt.ylabel('Missed cleavage\nBP abundance', fontsize=14)
plt.plot((0,8e7), (0,8e7), '--k', linewidth=1)
plt.tight_layout()
plt.xticks([0,2e8,4e8])
plt.savefig(outdir+'All_missed_cleavage_BP_vs_BP_scatter.pdf')

### Extended Data Figure 3e: Upset plot of SAAP identified across datasets

Uses Supplemental_Data_2.SAAP_proteins.xlsx (filt_saap_df)

In [ ]:
import upsetplot as up
import itertools

In [ ]:
# get dataframe of the number of datasets each SAAP-BP pair is found in 
saap_bp = [row['SAAP']+':'+row['BP'] for i,row in filt_saap_df.iterrows()]
saap_bp_counter = Counter(saap_bp)
saap_bp_count_df = pd.DataFrame.from_dict(saap_bp_counter, columns=['N datasets'], orient='index')

def get_ds(saap, bp, filt_saap_df):
    """ function to get the list of datasets a given SAAP-BP pair is found in"""
    saap_bp_df = filt_saap_df.loc[(filt_saap_df['SAAP']==saap) & (filt_saap_df['BP']==bp)]
    ds_list = saap_bp_df['Dataset'].tolist()
    return(ds_list)

# addd column with the datasets each SAAP-BP pair is found in 
saap_bp_count_df['Datasets'] = ''
for i in saap_bp_count_df.index:
    saap = i.split(':')[0]
    bp = i.split(':')[1]
    ds_str = str(get_ds(saap, bp, filt_saap_df))
    saap_bp_count_df.loc[i,'Datasets'] = ds_str
    
# initiate index for upset plot dataframe 
l = [False, True]
arrays = [list(i) for i in itertools.product(l, repeat=7)]
arrays = np.transpose(arrays)
tuples = list(zip(*arrays))
multiindex = pd.MultiIndex.from_tuples(tuples, names=datasets)

# Create dataframe for upset plot 
saap_counts = []
idx = multiindex[1]
keepidx = []
for idx in multiindex:
    if True in idx:
        saap_ds = [datasets[i] for i,x in enumerate(idx) if x==True]
        n_saap_ds = len(saap_bp_count_df.loc[saap_bp_count_df['Datasets']==str(saap_ds)])
        if n_saap_ds>0:
            saap_counts.append(n_saap_ds)
            keepidx.append(idx)
keepidx = np.transpose(keepidx)
tuples = list(zip(*keepidx))
multiindex = pd.MultiIndex.from_tuples(tuples, names=datasets)
upset_plot_data = pd.Series(saap_counts, index=multiindex)

# plot upset plot
up.plot(upset_plot_data, show_counts=True, min_subset_size=10)
plt.savefig(shared_saap_outdir+'Upsetplot_n10_max_Healthy.pdf', bbox_inches='tight')

### Extended Data Figure 3f: Percentage of samples of each dataset that SAAP identified in 6+ datasets ("shared SAAP") are found in

In [ ]:
# add number of datasets each SAAP-BP pair is found in to data from supplemental_data_2,3,4

filt_saap_df['N datasets'] = filt_saap_df.apply(lambda x: len(get_ds(x['SAAP'], x['BP'], filt_saap_df)), axis=1)
filt_saap_df['Datasets'] = filt_saap_df.apply(lambda x: str(get_ds(x['SAAP'], x['BP'], filt_saap_df)), axis=1)
filt_reporter_quant_df['N datasets'] = filt_reporter_df.apply(lambda x: len(get_ds(x['SAAP'], x['BP'], filt_saap_df)), axis=1)
filt_prec_quant_df['N datasets'] = filt_prec_df.apply(lambda x: len(get_ds(x['SAAP'], x['BP'], filt_saap_df)), axis=1)

# get subsets of the dataframes for which the SAAP-BP pair is found in 6+ datasets
n6_saap_df = filt_saap_df.loc[filt_saap_df['N datasets']>=6]
n6_reporter_df = filt_reporter_quant_df.loc[filt_reporter_quant_df['N datasets']>=6]
n6_prec_df = filt_prec_quant_df.loc[filt_prec_quant_df['N datasets']>=6]

#  generate dataframe for heatmap showing in what percentage of samples does each shared saap come up in each dataset
shared_saap_bp_n6 = [i for i,row in saap_bp_count_df.iterrows() if row['N datasets']>=6]
n_samples_heatmap_df = pd.DataFrame(index=datasets, columns=shared_saap_bp_n6)
pcnt_samples_heatmap_df = pd.DataFrame(index=datasets, columns=shared_saap_bp_n6)

# dictionary with total number of samples in each dataset for computing percentages
n_total_samples_dict = {ds:0 for ds in datasets}
for ds in datasets:
    if ds != 'Healthy':
        sample_map = sample_map_list[datasets.index(ds)]
        n_total_samples = len(sample_map['sample_name'].values)
    else:
        n_total_samples = len(samples_list[datasets.index(ds)])
    n_total_samples_dict[ds] = n_total_samples
        
for saap_bp in shared_saap_bp_n6:
    saap = saap_bp.split(':')[0]
    bp = saap_bp.split(':')[1]
    for ds in datasets:
        if ds!='Healthy':
            ds_reporter_df = n6_reporter_df.loc[n6_reporter_df['Dataset']==ds]
            saap_bp_df = ds_reporter_df.loc[(ds_reporter_df['SAAP']==saap) & (ds_reporter_df['BP']==bp)]
        else:
            ds_prec_df = n6_prec_df.loc[n6_prec_df['Dataset']==ds]
            saap_bp_df = ds_prec_df.loc[(ds_prec_df['SAAP']==saap) & (ds_prec_df['BP']==bp)]
        n_samples = len(saap_bp_df)
        pcnt_samples = 100*n_samples/n_total_samples_dict[ds]
        n_samples_heatmap_df.loc[ds, saap_bp] = n_samples
        pcnt_samples_heatmap_df.loc[ds, saap_bp] = pcnt_samples
        
# plot as a heatmap
pcnt_samples_heatmap_df = pcnt_samples_heatmap_df.astype(float)
c = sns.clustermap(pcnt_samples_heatmap_df, figsize=(5,5), method='ward', xticklabels=False, vmin=0, vmax=80)
c.ax_row_dendrogram.set_visible(False)
c.ax_heatmap.set_xlabel('SAAP in $\geq$6 datasets', fontsize=15)
c.ax_heatmap.set_yticklabels(c.ax_heatmap.get_ymajorticklabels(), fontsize = 14)
c.ax_heatmap.set_facecolor('#F3F3F3')
c.ax_heatmap.yaxis.set_ticks_position('left')
c.ax_cbar.set_ylabel(r'% samples with SAAP', labelpad=3)
c.ax_cbar.set_position([0.87,0.06,0.03,0.7])
c.ax_cbar.yaxis.label.set_size(15)
c.ax_cbar.tick_params(labelsize=13)
c.savefig(outdir+'%_samples_all_ds_sharedSAAPn6_vmax80.pdf', bbox_inches='tight')

### Extended Data Figure 3g: RAAS distributions for shared SAAP (found in 6+ datasets)

In [ ]:
fig,axes = plt.subplots(1,len(datasets),figsize=(len(datasets),3), sharey=True)
plt.subplots_adjust(wspace=0.05)

sns.set_style('whitegrid')
for ax in axes:
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.tick_params('x', bottom=False,labelbottom=False)
    if ax!=axes[0]:    
        ax.spines['left'].set_visible(False)
        ax.tick_params(left=False)
        
# compute medians to sort datasets by
medians = [np.nanmedian(n6_reporter_df.loc[n6_reporter_df['Dataset']==ds,'RAAS'].values) for ds in datasets[:-1]]
med_sort_idx = list(np.argsort(medians))
med_sort_idx.append(max(med_sort_idx)+1)

# plot violin plots
for i,ds in enumerate(datasets):
    ax_idx = med_sort_idx.index(i)
    axes[ax_idx].set_xlabel(ds, fontsize=14)
    if ds!='Healthy':
        ratio_data = [x for x in n6_reporter_df.loc[rn6_reporter_df['Dataset']==ds,'RAAS'].values if ~np.isnan(x) and ~np.isinf(x)]
    else:
        ratio_data = [x for x in n6_prec_df.loc[n6_prec_df['Dataset']==ds,'RAAS'].values if ~np.isnan(x) and ~np.isinf(x)]
    bihist(ratio_data, ratio_data, nbins=10,h=axes[ax_idx])
    if ax_idx==0:
        axes[ax_idx].annotate('N= ', (-0.2,1.01), xycoords='axes fraction', fontsize=13)

    axes[ax_idx].annotate(str(len(ratio_data)), (0.21,1.01), xycoords='axes fraction', fontsize=13)
        
    axes[ax_idx].plot(axes[ax_idx].get_xlim(), (np.median(ratio_data), np.median(ratio_data)), '--r',linewidth=0.7)

axes[0].set_ylabel(r'log$_{10}$ RAAS', fontsize=15)
axes[0].tick_params('y',labelsize=13)
axes[0].set_yticks([-6,-4,-2,0,2,4]);
plt.savefig(outdir+'Sample_level_RAAS_allDS_SharedSAAPin6.pdf', bbox_inches='tight')

### Extended Data Figure 3h: Boxplots of mean RAAS relative to LUAD

In [ ]:
# initate dataframe for plot

plot_rows = []
plot_cols = ['LUAD RAAS', 'Other RAAS', 'SAAP','BP', 'LUAD-Other', 'Dataset']
for ds in datasets:
    print(ds)
    if ds != 'LUAD' and ds!='Healthy':
        luad_ds_rows = [i for i,row in filt_saap_df.iterrows() if 'LUAD' in row['Datasets'] and ds in row['Datasets']]
        luad_ds_df = filt_saap_df.loc[luad_ds_rows]
        saap_list = luad_ds_df['SAAP'].values
        bp_list = luad_ds_df['BP'].values
        
        saap_bp_list = [[saap_list[i], bp_list[i]] for i in range(len(saap_list))]
        for sb in saap_bp_list:
            saap = sb[0]
            bp = sb[1]
            sb_df = luad_ds_df.loc[(luad_ds_df['SAAP']==saap) & (luad_ds_df['BP']==bp)]
            ds_raas = sb_df.loc[sb_df['Dataset']==ds, 'Mean reporter RAAS'].values[0]
            luad_raas = sb_df.loc[sb_df['Dataset']=='LUAD', 'Mean reporter RAAS'].values[0]
            ds_luad_raas = ds_raas - luad_raas 
            plot_rows.append([luad_raas, ds_raas, saap, bp, ds_luad_raas, ds])
plot_df = pd.DataFrame(plot_rows, columns=plot_cols)

# plot data 
fig,ax = plt.subplots(figsize=(6,3))
sns.boxplot(data=plot_df, x='Dataset', y='LUAD-Other', color='#CACACA', order=['LSCC','UCEC','BRCA','PDAC','CCRCC'], fliersize=0.8)
plt.tick_params('y', labelsize=13)
plt.tick_params('x', labelsize=14)
plt.xlabel('')
plt.ylabel('Other - LUAD log$_{10}$(RAAS)', fontsize=14)
plt.ylim([-2,2.5])
plt.savefig(outdir+'Shared_SAAP_Rel2LUAD_boxplots.pdf', bbox_inches='tight')

### Figure 2e: Relative RAAS computed for shared SAAP between 2 patient samples
Analysis limited to CPTAC datasets

In [ ]:
# Create a matrix with median  of relative RAAS computed for shared peptides between each pair of patients 

all_ds_matrices = {}
ds_report_df = filt_reporter_quant_df.loc[filt_quant_reporter_df['Sample type']=='Tumor']
ds_report_df.sort_values('RAAS', inplace=True)
set_samples = list(set(ds_report_df['Sample name'].values))
ds_matrix_rowcol = pd.DataFrame(index=set_samples, columns=set_samples)

for sample1 in set_samples:
    sample1_df = ds_report_df.loc[ds_report_df['Sample name']==sample1]
    sample1_saap_bp = [sample1_df['SAAP'].to_list()[i]+':'+sample1_df['BP'].to_list()[i] for i in range(len(sample1_df))]

    other_samples = set_samples[set_samples.index(sample1):]
    for sample2 in other_samples:
        sample2_df = ds_report_df.loc[ds_report_df['Sample name']==sample2]
        sample2_saap_bp = [sample2_df['SAAP'].to_list()[i]+':'+sample2_df['BP'].to_list()[i] for i in range(len(sample2_df))]
        shared_saap_bp = [x for x in sample1_saap_bp if x in sample2_saap_bp]
        row_shared_raas_ratios = []
        col_shared_raas_ratios = []
        
        # for each SAAP shared between 2 patients, compute relative RAAS on log2 scale
        for sb, saap_bp in enumerate(shared_saap_bp):
            saap = saap_bp.split(':')[0]
            bp = saap_bp.split(':')[1]
            raas1 = sample1_df.loc[(sample1_df['SAAP']==saap) & (sample1_df['BP']==bp), 'RAAS'].values[0]
            raas2 = sample2_df.loc[(sample2_df['SAAP']==saap) & (sample2_df['BP']==bp), 'RAAS'].values[0]
            if ~np.isnan(raas1) and ~np.isnan(raas2) and ~np.isinf(raas1) and ~np.isinf(raas2):
                raas_ratio = raas1 - raas2
                raas_ratio = np.log2(10**raas_ratio)
                col_raas_ratio = raas2 - raas1
                col_raas_ratio = np.log2(10**col_raas_ratio)
                row_shared_raas_ratios.append(raas_ratio)
                col_shared_raas_ratios.append(col_raas_ratio)
        ds_matrix_rowcol.loc[sample1, sample2] = np.nanmedian(row_shared_raas_ratios)
        ds_matrix_rowcol.loc[sample2, sample1] = np.nanmedian(col_shared_raas_ratios)

all_ds_matrices['All_data'] = {'Row_col':ds_matrix_rowcol}

# Add N SAAP per comparison to all_data matrix
ds_report_df = filt_reporter_quant_df.loc[filt_reporter_quant_df['Sample type']=='Tumor']
ds_report_df.sort_values('RAAS', inplace=True)
set_samples = list(set(ds_report_df['Sample name'].values))

ds_matrix_rowcol = pd.DataFrame(index=set_samples, columns=set_samples)
for i,sample1 in enumerate(set_samples):
    if i%100==0:
        print(i)
    sample1_df = ds_report_df.loc[ds_report_df['Sample name']==sample1]
    sample1_saap_bp = [sample1_df['SAAP'].to_list()[i]+':'+sample1_df['BP'].to_list()[i] for i in range(len(sample1_df))]
    other_samples = set_samples[set_samples.index(sample1):]
    for sample2 in other_samples:
        sample2_df = ds_report_df.loc[ds_report_df['Sample name']==sample2]
        sample2_saap_bp = [sample2_df['SAAP'].to_list()[i]+':'+sample2_df['BP'].to_list()[i] for i in range(len(sample2_df))]
        shared_saap_bp = [x for x in sample1_saap_bp if x in sample2_saap_bp]
        ds_matrix_rowcol.loc[sample1, sample2] = len(shared_saap_bp)
        ds_matrix_rowcol.loc[sample2, sample1] = len(shared_saap_bp)

all_ds_matrices['All_data']['N_shared_saap'] = ds_matrix_rowcol

pickle.dump(all_ds_matrices, open(outdir+'All_pairwise_sample_RAAS_ratio_matrices_log2.p', 'wb'))

In [ ]:
# plot heatmnap sorted by row/col median 
ds = 'All_data'

ds_matrix = all_ds_matrices[ds]['Row_col'].astype(float)
medians = [np.median([x for x in row.values if ~np.isnan(x)]) for i,row in ds_matrix.iterrows()]
srt_idx = np.argsort(medians)
col_medians = [np.median([x for x in ds_matrix[col].to_list() if ~np.isnan(x)]) for col in ds_matrix.columns]
col_srt_idx = np.argsort(col_medians)
plot_matrix = ds_matrix.iloc[srt_idx, col_srt_idx]

c = sns.clustermap(plot_matrix, row_cluster=False, col_cluster=False, cmap='RdBu_r', center=0, robust=True, 
                   yticklabels=False, xticklabels=False, figsize=(3,3))#, cbar_kws={'ticklocation':'left'})
c.ax_cbar.set_visible(False)
c.ax_heatmap.set_xlabel('Patient samples', fontsize=14)
c.ax_heatmap.set_ylabel('Patient samples', fontsize=14)
c.ax_heatmap.yaxis.set_label_position('left')
c.ax_heatmap.set_title('1068 cancer patients', fontsize=14)

c.savefig(outdir+ds+'_Paired_RAAS_ratios_btwn_samples_median_sorted_rowcol_log2_nocbar.pdf', bbox_inches='tight')
c.savefig(outdir+ds+'_Paired_RAAS_ratios_btwn_samples_median_sorted_rowcol_log2_nocbar.png', dpi=300, bbox_inches='tight')

medians = [np.median([x for x in row.values if ~np.isnan(x)]) for i,row in ds_matrix.iterrows()]

# barplot of patient medians
srt_medians = [medians[i] for i in srt_idx]
str_srt_idx = [str(x) for x in srt_idx]
plot_df = pd.DataFrame(zip(str_srt_idx, srt_medians), columns=['Index','Median log$_{10}$(RAAS) ratio'])

sns.set_style('ticks')
fig,ax = plt.subplots(figsize=(1,3))
sns.barplot(data=plot_df, x='Median log$_{10}$(RAAS) ratio', y='Index', palette='RdBu_r', saturation=1,linewidth=0)
ax.set_ylabel(None)
ax.set_yticks([]);
ax.tick_params('x', labelsize=13)
plt.ylabel('Median log$_{2}$(relative RAAS)', fontsize=14)
plt.xlabel('')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.set_xticks([-2,0,2]);
ax.yaxis.set_label_position('right')
plt.savefig(outdir+ds+'_Paired_RAAS_ratios_btwn_samples_median_barplot_log2.pdf', bbox_inches='tight')

### Figure 2f: weighted means of relative RAAS for each patient in each dataset


In [ ]:
# plot weighted mean 

def weighted_mean(vals, ns):
    """ function to get mean weighted by the number of shared SAAP """
    wmean = np.sum([vals[i]*ns[i] for i in range(len(vals))])/np.sum(ns)
    return(wmean)

all_ds_matrix = all_ds_matrices['All_data']['Row_col'].astype(float)
all_ds_n_matrix = all_ds_matrices['All_data']['N_shared_saap'].astype(float)

sns.set_style('whitegrid')
fig,axes = plt.subplots(1, 6, figsize=(4.5,3), sharey=True)
plt.subplots_adjust(wspace=0)

# get medians for sorting datasets in plot
medians = []
for i,ds in enumerate(datasets[:-1]):
    ds_sample_map = sample_map_list[datasets.index(ds)]
    ds_samples = list(set(ds_sample_map['sample_name'].to_list()))
    ds_matrix = all_ds_matrix.loc[[r for r,row in all_ds_matrix.iterrows() if r in ds_samples]]#, [c for c in all_ds_matrix.columns if c in ds_samples]]
    n_matrix = all_ds_n_matrix.loc[[r for r,row in all_ds_n_matrix.iterrows() if r in ds_samples]]
    weighted_means = [weighted_mean(ds_matrix.loc[r], row) for r,row in n_matrix.iterrows()]
    vals = [x for x in weighted_means  if ~np.isnan(x)]
    medians.append(np.median(vals))
srt_idx = np.argsort(medians)
ds_sorted = [datasets[:-1][i] for i in srt_idx]

# plot weighted means 
for i,ds in enumerate(ds_sorted):
    ax = axes[i]
    ds_sample_map = sample_map_list[datasets.index(ds)]
    ds_samples = list(set(ds_sample_map['sample_name'].to_list()))
    ds_matrix = all_ds_matrix.loc[[r for r,row in all_ds_matrix.iterrows() if r in ds_samples]]#, [c for c in all_ds_matrix.columns if c in ds_samples]]
    n_matrix = all_ds_n_matrix.loc[[r for r,row in all_ds_n_matrix.iterrows() if r in ds_samples]]
    
    weighted_means = [weighted_mean(ds_matrix.loc[r], row) for r,row in n_matrix.iterrows()]
    vals = weighted_means
    srt_idx = np.argsort(vals)
    srt_vals = [vals[s] for s in srt_idx]
    plot_df = pd.DataFrame(zip(list(range(len(vals))), srt_vals), columns=['Rank', 'RAAS ratio'])

    sns.scatterplot(data=plot_df, y='RAAS ratio', x='Rank', ax=ax, linewidth=0, s=15, color='#AAAAAA')
    ax.set_xlabel(ds, fontsize=12)
    ax.set_xticks([])
    n = len(srt_vals)
    if i == 0:
        ax.annotate('N = '+str(int(np.round(n, 0))), xy=(0.8,1.01), xycoords='axes fraction', ha='right', fontsize=11)
    else:
        ax.annotate(str(int(np.round(n, 0))), xy=(0.5,1.01), xycoords='axes fraction', ha='center', fontsize=11)
    median = np.median([x for x in vals if ~np.isnan(x)])
    ax.plot(ax.get_xlim(), (median,median), '--r', linewidth=0.8)
    
axes[0].set_yticks([-1, 0, 1])
axes[0].tick_params('y', labelsize=13)
axes[0].set_ylabel('log$_{2}$(relative RAAS)', fontsize=14, labelpad=-0.5)

plt.savefig(outdir+'All_Paired_RAAS_ratios_btwn_samples_ranksort_allVals_log2_ds_numerator_allotherds_denom_weighted_mean.pdf', bbox_inches='tight')

### Figure 2h. SAAP  copy number estimates 

In [ ]:
# compute the abundance of core histone proteins using precursor ion intensity summed over isoforms mapping to each core histone

histone_prot_abund_dict = {}
for ds in datasets:
    print(ds)
    data_dir = data_dir_list[datasets.index(ds)]
    
    # prot_dict and blast_map are computed as downstream part of decode pipeline. Provided in Google Drive. 
    prot_dict = pickle.load(open(data_dir+'Allprot_normalized_abundance_dict.p', 'rb'))
    blast_map = pd.read_excel(data_dir+'blast_map_w_gene.xlsx', index_col=0)
    samples = samples_list[datasets.index(ds)]

    histone1_protein_rows = [i for i,row in blast_map.iterrows() if re.search('histone H1', row['Blast protein'])]
    histone1_proteins = blast_map.loc[histone1_protein_rows, 'Protein name'].to_list()
    histone2_protein_rows = [i for i,row in blast_map.iterrows() if re.search('histone H2', row['Blast protein'])]
    histone2_proteins = blast_map.loc[histone2_protein_rows, 'Protein name'].to_list()
    histone3_protein_rows = [i for i,row in blast_map.iterrows() if re.search('histone H3', row['Blast protein'])]
    histone3_proteins = blast_map.loc[histone3_protein_rows, 'Protein name'].to_list()
    histone4_protein_rows = [i for i,row in blast_map.iterrows() if re.search('histone H4', row['Blast protein'])]
    histone4_proteins = blast_map.loc[histone4_protein_rows, 'Protein name'].to_list()

    # dict of total histone protein abundance in each patient sample
    histone_prot_abund_dict[ds] = {}
    for s in samples:
        prot_df = prot_dict[s]
        histone1_prots = prot_df.loc[[x for x in histone1_proteins if x in prot_df.index], 'Precursor intensity'].sum()
        histone2_prots = prot_df.loc[[x for x in histone2_proteins if x in prot_df.index], 'Precursor intensity'].sum()
        histone3_prots = prot_df.loc[[x for x in histone3_proteins if x in prot_df.index], 'Precursor intensity'].sum()
        histone4_prots = prot_df.loc[[x for x in histone4_proteins if x in prot_df.index], 'Precursor intensity'].sum()
        histone_prot_abund_dict[ds][s] = np.median([histone1_prots, histone2_prots, histone3_prots, histone4_prots])

pickle.dump(histone_prot_abund_dict, open(outdir+'Histone_prot_median_precursor_abundance_dict_sumsubtypes.p','wb'))

In [ ]:
# compute copy number estimates for SAAP based on their protein abundances and histone abundances
# uses "histone ruler" method assuming 30e6 copy number for histone proteins

plot_rows = []
plot_cols = ['Dataset', 'SAAP', 'Sample name', 'SAAP abundance', 'Histone abundance', 'SAAP/Histone']
for ds in datasets:
    print(ds)
    ds_quant_df = filt_prec_quant_df.loc[filt_prec_quant_df['Dataset']==ds]
    saaps = list(set(ds_quant_df['SAAP'].values))
    samples = list(set(ds_quant_df['TMT/Tissue'].values))
    samples = [x for x in samples if x!='tonsil']

    for sample in samples:
        histone_abund = histone_prot_abund_dict[ds][sample]
        for saap in saaps:
            saap_abund = ds_quant_df.loc[(ds_quant_df['SAAP']==saap) & (ds_quant_df['TMT/Tissue']==sample), 'SAAP abundance'].values
            if len(saap_abund)>0: # not all SAAP in every sample
                saap_abund = saap_abund[0]
                saap_histone = saap_abund/histone_abund
                plot_rows.append([ds, saap, sample, saap_abund, histone_abund, saap_histone])

plot_df = pd.DataFrame(plot_rows, columns=plot_cols)
plot_df['SAAP copies'] = plot_df.apply(lambda x: np.round(30e6*x['SAAP/Histone']), axis=1)
plot_df.to_excel(outdir+'SAAP_copy_estimate_by_median_histone_sumsubtypes_precursors.xlsx')

In [ ]:
# plot median and max SAAP copy numbers in rank sorted order, highlighting substituted proteins of interest
prec_saap_copy_number_df = plot_df

# combine peptides across all datasets with median and max
rows = []
cols = ['SAAP', 'Median SAAP copies', 'Genes','Pfam', 'RefProt', 'Max SAAP copies']
ds_df = prec_saap_copy_number_df.loc[prec_saap_copy_number_df['Dataset']!='Healthy']
ds_saap = list(set(ds_df['SAAP'].to_list()))
for saap in ds_saap:
    saap_df = ds_df.loc[ds_df['SAAP']==saap]
    median_copies = saap_df['SAAP copies'].median()
    max_copies = saap_df['SAAP copies'].max()
    genes = saap_df['Genes'].to_list()[0]
    pfam = saap_df['Pfam'].to_list()[0]
    refprot = saap_df['RefProt'].to_list()[0]
    rows.append([saap, median_copies, genes, pfam, refprot, max_copies]) 
median_prec_saap_copy_number_df = pd.DataFrame(rows, columns=cols)

# generate plot 
rank_df = median_prec_saap_copy_number_df.loc[median_prec_saap_copy_number_df['Median SAAP copies']>=10, ['SAAP','Median SAAP copies', 'Max SAAP copies']]

sns.set_style('ticks')
fig,ax = plt.subplots(figsize=(3,3))
rank_df.sort_values('Max SAAP copies', ascending=False, inplace=True)
rank_df.reset_index(inplace=True, drop=True)
rank_df['Rank'] = rank_df.index
sns.scatterplot(data=rank_df, x='Rank', y='Max SAAP copies', color='#AAAAAA', linewidth=0, s=80, marker='.', alpha=0.5)

rank_df.sort_values('Median SAAP copies', ascending=False, inplace=True)
rank_df.reset_index(inplace=True, drop=True)
rank_df['Rank'] = rank_df.index

sns.scatterplot(data=rank_df, x='Rank', y='Median SAAP copies', color='#555555', linewidth=0, s=80, marker='.', alpha=0.5)
sns.scatterplot(data=rank_df.loc[rank_df['SAAP']=='ADNTLVAYK'], x='Rank', y='Median SAAP copies', color=colors[2], linewidth=0.5, s=100)#, label='FES')
sns.scatterplot(data=rank_df.loc[rank_df['SAAP']=='LIDLLESGK'], x='Rank', y='Median SAAP copies', color=colors[1], linewidth=0.5, s=100)#, label='DDX17')
sns.scatterplot(data=rank_df.loc[rank_df['SAAP']=='VLAVNQEHEQLMEDYEK'], x='Rank', y='Median SAAP copies', color=colors[4], linewidth=0.5, s=100)#, label='DDX17')
sns.scatterplot(data=rank_df.loc[rank_df['SAAP']=='IIHLGGK'], x='Rank', y='Median SAAP copies', color='k', linewidth=0.5, s=100)#, label='DDX17')
sns.scatterplot(data=rank_df.loc[rank_df['SAAP']=='AGSELAAHQK'], x='Rank', y='Median SAAP copies', color=colors[0], linewidth=0.5, s=100)#, label='PSMA1')

saap2plot = ['ADNTLVAYK', 'LIDLLESGK', 'VLAVNQEHEQLMEDYEK', 'IIHLGGK', 'AGSELAAHQK']
ranks2plot = [rank_df.loc[rank_df['SAAP']==saap, 'Rank'].values[0] for saap in saap2plot]
texts2plot = ['FES','DDX17', 'ACTN1', 'ZNF845', 'PSMA1']
colors2plot = [colors[2], colors[1], colors[4], 'black', colors[0]]
for i,rank in enumerate(ranks2plot):
    txt = texts2plot[i]
    y = rank_df.loc[rank_df['Rank']==rank,'Median SAAP copies']
    if txt=='DDX17' or txt=='PSMA1':
        ax.annotate(txt, xy=(rank,y), xytext=(rank+250,y+10), xycoords='data', textcoords='data', va='center',
                arrowprops=dict(facecolor='k', headwidth=5,width=1, linewidth=0), color=colors2plot[i], fontsize=11)
    elif txt=='ZNF845':
        ax.annotate(txt, xy=(rank,y), xytext=(rank+500,y+2000), xycoords='data', textcoords='data', va='center',
            arrowprops=dict(facecolor='k', headwidth=5,width=1, linewidth=0), color=colors2plot[i], fontsize=11)
    elif txt=='ACTN1':
        ax.annotate(txt, xy=(rank,y), xytext=(rank+1000,y+600), xycoords='data', textcoords='data', va='center',
                arrowprops=dict(facecolor='k', headwidth=5,width=1, linewidth=0), color=colors2plot[i], fontsize=11)
    elif txt=='FES':
        ax.annotate(txt, xy=(rank,y), xytext=(rank+300,y-300), xycoords='data', textcoords='data', va='center',
                arrowprops=dict(facecolor='k', headwidth=5,width=1, linewidth=0), color=colors2plot[i], fontsize=11)
        
plt.ylim([10,1e5])
plt.yscale('log')
ax.tick_params('both', labelsize=13)
plt.ylabel('SAAP copies/cell', fontsize=14)
plt.xlabel('Protein rank', fontsize=14)
handles = [Line2D([0],[0], marker='o', color='w', label='Median', markerfacecolor='#555555'),
          Line2D([0],[0], marker='o', color='w', label='Maximum', markerfacecolor='#AAAAAA')]
plt.legend(handles=handles, fontsize=12, handletextpad=0, markerscale=1.2, frameon=True)
plt.savefig(outdir+'SAAP_copies_median_histone_sumsubtypes_rankplot_ticks_gr10copies_noHealthy_PSMA1_w_max.pdf', bbox_inches='tight')

# Figure 3: Ratios by AAS

#### Incl. Extended Data Figure 5

In [3]:
# custom colormap
# this colormap is used for all RAAS dotplots 

import matplotlib
from matplotlib.cm import get_cmap
inferno_cmap = get_cmap('inferno')
viridis_cmap = get_cmap('viridis')

inferno_colors = inferno_cmap.colors
viridis_colors = viridis_cmap.colors

inferno_colors[-1] = viridis_colors[-1]
custom_cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", inferno_colors)

### Extended Data Figure 5b: Distributions of median RAAS for each substitution type

In [ ]:
# list of all substitution types found in data
AASs = list(set(filt_prec_quant_df['AAS'].values))

# generate dictionary with RAAS values organized by substitution type
ratios_by_aas = {}
for i,ds in enumerate(datasets):
    print(ds)
    ds_prec_df = filt_prec_quant_df.loc[filt_prec_quant_df['Dataset']==ds]
    ds_report_df = filt_reporter_quant_df.loc[filt_reporter_quant_df['Dataset']==ds]
    aas_prec_ratio_df = ds_prec_df.loc[:, ['AAS', 'RAAS']]
    aas_prec_ratio_df.replace(np.inf, np.nan, inplace=True)
    aas_prec_ratio_df.replace(-np.inf, np.nan, inplace=True)
    aas_prec_ratio_df.dropna(how='any', axis=0, inplace=True)
    
    prec_subs_in_plot = [x for x in AASs if x in set(aas_prec_ratio_df['AAS'])]
    prec_medians = [np.nanmedian([x for x in aas_prec_ratio_df.loc[aas_prec_ratio_df['AAS']==sub,'RAAS'].values]) for sub in prec_subs_in_plot]
    prec_sorted_idx = np.argsort(prec_medians)
    prec_sorted_medians = [prec_medians[i] for i in prec_sorted_idx]
    prec_sorted_subs = [prec_subs_in_plot[i] for i in prec_sorted_idx]
    
    if ds !='Healthy':
        aas_ratio_df = ds_report_df.loc[:, ['AAS', 'RAAS']]
        aas_ratio_df.replace(np.inf, np.nan, inplace=True)
        aas_ratio_df.replace(-np.inf, np.nan, inplace=True)
        aas_ratio_df.dropna(how='any', axis=0, inplace=True)
        subs_in_plot = [x for x in AASs if x in set(aas_ratio_df['AAS'])]
        #medians = [np.nanmedian([x for x in aas_ratio_df.loc[aas_ratio_df['AAS']==sub,'Substitution ratio'].values]) for sub in subs_in_plot]
        medians = [np.nanmedian([x for x in aas_ratio_df.loc[aas_ratio_df['AAS']==sub,'RAAS'].values]) for sub in subs_in_plot]
        sorted_idx = np.argsort(medians)
        sorted_medians = [medians[i] for i in sorted_idx]
        sorted_subs = [subs_in_plot[i] for i in sorted_idx]
        ratios_by_aas[ds] = {'Precursor_ratio_df': aas_prec_ratio_df, 'prec_AAS_sorted':prec_sorted_subs, 'prec_medians_sorted':prec_sorted_medians, 
                             'Reporter_ratio_df': aas_ratio_df, 'AAS_sorted':sorted_subs, 'medians_sorted':sorted_medians}
    else:
        ratios_by_aas[ds] = {'Precursor_ratio_df': aas_prec_ratio_df, 'prec_AAS_sorted':prec_sorted_subs, 'prec_medians_sorted':prec_sorted_medians}
pickle.dump(ratios_by_aas, open(outdir+'Ratios_by_AAS.p', 'wb'))

# get median RAAS for each AAS type
median_aas_ratios = {ds:{} for ds in datasets}
for ds in datasets:
    ds_aas_df = ratios_by_aas[ds]['Precursor_ratio_df']
    aas = list(set(ds_aas_df['AAS'].values))
    aas_medians = {aa:np.median([x for x in ds_aas_df.loc[ds_aas_df['AAS']==aa,'RAAS'].values if ~np.isnan(x) and ~np.isinf(x)]) for aa in aas}
    median_aas_ratios[ds] = aas_medians


# plot distributions of median RAAS across AAS types
fig,axes = plt.subplots(1,len(datasets),figsize=(len(datasets),3), sharey=True)
sns.set_style('whitegrid')
for ax in axes:
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.tick_params('x', bottom=False,labelbottom=False)
    if ax!=axes[0]:    
        ax.spines['left'].set_visible(False)
        ax.tick_params(left=False)

# get medians of each dataset to sort datasets for visualization
medians = [np.nanmedian(list(median_aas_ratios[ds].values())) for ds in datasets if ds!='Healthy']
med_sort_idx = list(np.argsort(medians))
med_sort_idx.append(max(med_sort_idx)+1)

for i,ds in enumerate(datasets):
    ax_idx = med_sort_idx.index(i)
    axes[ax_idx].set_xlabel(ds, fontsize=14)
    ratio_data = list(median_aas_ratios[ds].values())
    bihist(ratio_data, ratio_data, nbins=30,h=axes[ax_idx])
    if ax_idx==0:
        axes[ax_idx].annotate('N = ', (-0.2,1), xycoords='axes fraction', fontsize=12)
    axes[ax_idx].annotate('{:,}'.format((len(ratio_data))), (0.3,1), xycoords='axes fraction', fontsize=12)
    axes[ax_idx].plot(axes[ax_idx].get_xlim(), (np.nanmedian(ratio_data), np.nanmedian(ratio_data)), '--r',linewidth=1)
axes[0].set_ylabel(r'log$_{10}$(RAAS)', fontsize=14)
axes[0].tick_params('y', labelsize=12)
plt.suptitle('Distribution of median RAAS across substitution types', fontsize=14)
plt.savefig(outdir+'MedianRAAS_byAAStype_hist.pdf', bbox_inches='tight')

### Figure 3d: weighted correlation of median RAAS for AAS types across datasets

In [ ]:
def weighted_corr(x, y, w):
    """
    Calculates the weighted Pearson correlation coefficient between two vectors.

    Args:
      x (np.ndarray): First vector.
      y (np.ndarray): Second vector.
      w (np.ndarray): Weights vector (same length as x and y).

    Returns:
      float: Weighted Pearson correlation coefficient between x and y.
    """
    cov = weighted_cov(x, y, w)
    std_x = np.sqrt(weighted_cov(x, x, w))
    std_y = np.sqrt(weighted_cov(y, y, w))
    if std_x * std_y == 0:
        return 0.0  # Avoid division by zero
    return(cov / (std_x * std_y))

In [ ]:
# generate a heatmap with the median precursor RAAS for each AAS type in each dataset 
prec_heatmap_data = pd.DataFrame(index=AASs, columns=datasets)
for i, row in prec_heatmap_data.iterrows():
    for ds in datasets:
        ds_ratios = ratios_by_aas[ds]
        medians = ds_ratios['prec_medians_sorted']
        aas = ds_ratios['prec_AAS_sorted']
        for i,aa in enumerate(aas):
            prec_heatmap_data.loc[aa,ds] = medians[i]
prec_heatmap_data.dropna(how='all', inplace=True)
prec_heatmap_data.to_excel(outdir+'Precursor_RAAS_byAAS_alldatasets.xlsx')

# compute correlations of median RAAS by AAS vectors across datasets
prec_heatmap_data.replace(np.inf, np.nan, inplace=True)
heatmap_df_4corr = prec_heatmap_data.dropna(how='any')
corr_heatmap = pd.DataFrame(index=datasets, columns=datasets)
for ds1 in datasets:
    for ds2 in datasets:
        x = heatmap_df_4corr[ds1].to_list()
        y = heatmap_df_4corr[ds2].to_list()
        w = [raas_by_aas_stats[ds1][aas]['N_peptides']+raas_by_aas_stats[ds2][aas]['N_peptides'] for aas in heatmap_df_4corr.index]
        corr = weighted_corr(x, y, w)
        corr_heatmap.loc[ds1,ds2] = corr

# plot correlation heatmap
corr_heatmap = corr_heatmap.astype(float)
s= sns.clustermap(corr_heatmap, figsize=(5,5), annot=np.round(corr_heatmap, 2), cmap='Reds', cbar_kws={'label':'Pearson correlation', 'pad':1}, annot_kws={"fontsize":12},
                 vmax=1, vmin=0)

s.ax_heatmap.tick_params(right=False, left=True, labelright=False, labelleft=True, labelbottom=True, bottom=True)
plt.setp(s.ax_heatmap.xaxis.get_majorticklabels(), rotation=90, fontsize=13)
plt.setp(s.ax_heatmap.yaxis.get_majorticklabels(), fontsize=13)
x0, y0, w, h = s.cbar_pos
s.ax_cbar.set_position([0.87,0.26,0.05,0.5])
s.ax_cbar.yaxis.label.set_size(14)
s.ax_cbar.set_yticklabels([0,0.2,0.4,0.6,0.8, 1], fontsize=13)
s.ax_row_dendrogram.set_visible(False)
s.ax_col_dendrogram.set_visible(False)
s.ax_heatmap.set_title('AAS type median RAAS correlation', position=(0.5,1), fontsize=14)
plt.savefig(outdir+'Correlation_heatmap_AAStype_PrecRatios_weighted_clustered_noNan.pdf', bbox_inches='tight')

### Extended Data Figure 5d,e: AAS types with high and low variance in RAAS across datasets

In [ ]:
# generate a heatmap with the median reporter RAAS (precursor for healthy data) for each AAS type in each dataset 
heatmap_data = pd.DataFrame(index=AASs, columns=datasets)
for i, row in heatmap_data.iterrows():
    for ds in datasets:
        ds_ratios = ratios_by_aas[ds]
        if ds!='Healthy':
            medians = ds_ratios['medians_sorted']
            aas = ds_ratios['AAS_sorted']
        else:
            medians = ds_ratios['prec_medians_sorted']
            aas = ds_ratios['prec_AAS_sorted']
        for i,aa in enumerate(aas):
            heatmap_data.loc[aa,ds] = medians[i]
heatmap_df = heatmap_data.loc[:,['PDAC', 'BRCA', 'CCRCC', 'UCEC','LUAD', 'LSCC', 'Healthy']]
heatmap_df.dropna(how='any', axis=0, inplace=True)
heatmap_df = heatmap_df.astype(float)
heatmap_df.replace(np.inf, np.nan, inplace=True)
heatmap_data.to_excel(outdir+'Reporter_RAAS_byAAS_alldatasets.xlsx')

# compute variance across datasets for each AAS type
var = []
for i,row in heatmap_df.iterrows():
    var.append(np.var(row.values))
# these thresholds were determined by plotting histogram of var values
high_var_idx = [i for i,x in enumerate(var) if x>=0.5]
low_var_idx = [i for i,x in enumerate(var) if x<=0.1]

In [ ]:
# plot heatmap of median RAAS for AAS types with high variance across datasets (Ext. Data Fig. 5e)

high_var_heatmap_df = heatmap_df.iloc[high_var_idx]
c = sns.clustermap(data=high_var_heatmap_df, figsize=(3,4), yticklabels=True, method='ward', xticklabels=True,
                   col_cluster=False, cmap=custom_cmap, vmax=v_max, vmin=v_min)#, robust=True)
c.ax_heatmap.set_xticklabels(c.ax_heatmap.get_xmajorticklabels(), fontsize = 12)
c.ax_heatmap.set_yticklabels(c.ax_heatmap.get_ymajorticklabels(), fontsize = 12)
c.ax_heatmap.set_facecolor('#DDDDDD')
c.ax_cbar.set_ylabel(r'Median log$_{10}$(RAAS)')
c.ax_cbar.set_position([1,.25,0.05,0.5])
c.ax_cbar.yaxis.label.set_size(14)
c.ax_cbar.tick_params(labelsize=13)
c.ax_heatmap.set_title('AAS types with high variance', position=(0.5,0.5), fontsize=13)
c.savefig(outdir+'Reporter_highvar_heatmap_df_by_AAS_heatmap.pdf', bbox_inches='tight')

# plot heatmap of median RAAS for AAS types with low variance across datasets (Ext. Data Fig. 5d)

low_var_heatmap_df = heatmap_df.iloc[low_var_idx]
c = sns.clustermap(data=low_var_heatmap_df, figsize=(3,4), yticklabels=True, method='ward', xticklabels=True,
                   col_cluster=False, cmap=custom_cmap, vmax=v_max, vmin=v_min)#, robust=True)
c.ax_heatmap.set_xticklabels(c.ax_heatmap.get_xmajorticklabels(), fontsize = 12)
c.ax_heatmap.set_yticklabels(c.ax_heatmap.get_ymajorticklabels(), fontsize = 12)
c.ax_heatmap.set_facecolor('#DDDDDD')
c.ax_cbar.set_ylabel(r'Median log$_{10}$(RAAS)')
c.ax_cbar.set_position([1,0.25,0.05,0.5])
c.ax_cbar.yaxis.label.set_size(14)
c.ax_cbar.tick_params(labelsize=13)
c.ax_heatmap.set_title('AAS types with low variance', position=(0.5,0.5), fontsize=13)
c.savefig(outdir+'Reporter_lowvar_heatmap_df_by_AAS_heatmap.pdf', bbox_inches='tight')

### Figure 3a: Ranked bar plot of median RAAS for AAS types, highlighting specific AAS types in violin plot, with barplot showing # SAAP detected for each AAS type

In [ ]:
# add percentile stats for RAAS of each AAS type to Ratios_by_AAS dictionary 

pcntile_list = [90, 75, 25, 10, 'max', 'min']

for ds in datasets:
    print(ds)
    aas_ratio_df = ratios_by_aas[ds]['Precursor_ratio_df']
    aas_in_plot = list(set(aas_ratio_df['AAS'].values))
    
    for pcntile in pcntile_list:
        title_str = pcntile if isinstance(pcntile,str) else str(pcntile)+'percentile'
        if isinstance(pcntile,int):
            prec_percentiles = [np.percentile([x for x in aas_ratio_df.loc[aas_ratio_df['AAS']==aas,'RAAS'].values], pcntile) for aas in aas_in_plot]
        elif pcntile=='max':
            prec_percentiles = [np.max([x for x in aas_ratio_df.loc[aas_ratio_df['AAS']==aas,'RAAS'].values]) for aas in aas_in_plot]
        else:
            prec_percentiles = [np.min([x for x in aas_ratio_df.loc[aas_ratio_df['AAS']==aas,'RAAS'].values]) for aas in aas_in_plot]
        prec_sorted_idx = np.argsort(prec_percentiles)
        prec_sorted_percentiles = [prec_percentiles[i] for i in prec_sorted_idx]
        prec_sorted_subs = [aas_in_plot[i] for i in prec_sorted_idx]

        if ds !='Healthy':
            aas_ratio_df = ratios_by_aas[ds]['Reporter_ratio_df']
            if isinstance(pcntile,int):
                reporter_percentiles = [np.percentile([x for x in aas_ratio_df.loc[aas_ratio_df['AAS']==aas,'RAAS'].values], pcntile) for aas in aas_in_plot]
            elif pcntile=='max':
                reporter_percentiles = [np.max([x for x in aas_ratio_df.loc[aas_ratio_df['AAS']==aas,'RAAS'].values]) for aas in aas_in_plot]
            else:
                reporter_percentiles = [np.min([x for x in aas_ratio_df.loc[aas_ratio_df['AAS']==aas,'RAAS'].values]) for aas in aas_in_plot]
            reporter_sorted_idx = np.argsort(reporter_percentiles)
            reporter_sorted_percentiles = [reporter_percentiles[i] for i in reporter_sorted_idx]
            reporter_sorted_subs = [aas_in_plot[i] for i in reporter_sorted_idx]

            ratios_by_aas[ds]['Prec_AAS_'+str(title_str)+'_sorted']=prec_sorted_subs
            ratios_by_aas[ds]['Prec_'+str(title_str)+'_sorted']=prec_sorted_percentiles
            ratios_by_aas[ds]['Reporter_AAS_'+str(title_str)+'_sorted']=reporter_sorted_subs
            ratios_by_aas[ds]['Reporter_'+str(title_str)+'_sorted']=reporter_sorted_percentiles
        else:
            ratios_by_aas[ds]['Prec_AAS_'+str(title_str)+'_sorted']=prec_sorted_subs
            ratios_by_aas[ds]['Prec_'+str(title_str)+'_sorted']=prec_sorted_percentiles
            ratios_by_aas[ds]['Reporter_AAS_'+str(title_str)+'_sorted']=prec_sorted_subs
            ratios_by_aas[ds]['Reporter_'+str(title_str)+'_sorted']=prec_sorted_percentiles

# add the number of SAAP with each AAS type quantified across all datasets          
for ds in datasets:
    print(ds)
    ds_ratios = ratios_by_aas[ds]
    prec_aas = ds_ratios['Precursor_ratio_df']
    n_prec_aas = {aas:len(prec_aas.loc[prec_aas['AAS']==aas]) for aas in set(prec_aas['AAS'].values)}
    ds_ratios['Precursor_N_AAS'] = n_prec_aas
    if ds!='Healthy':
        reporter_aas = ds_ratios['Reporter_ratio_df']
        n_reporter_aas = {aas:len(reporter_aas.loc[reporter_aas['AAS']==aas]) for aas in set(reporter_aas['AAS'].values)}
        ds_ratios['Reporter_N_AAS'] = n_reporter_aas

pickle.dump(ratios_by_aas, open(outdir+'Ratios_by_AAS.p','wb'))

In [ ]:
# create dataframe with values needed for plot (e.g. median RAAS, 90th percentile RAAS, for each AAS type)

plot_rows = []
plot_cols = ['Dataset','AAS type','Median Log$_{10}$ RAAS', '90th percentile Log$_{10}$ RAAS', 'Max Log$_{10}$ RAAS', 'RAAS type', 'N']

for ds in datasets:
    print(ds)
    ds_ratios = ratios_by_aas[ds]
    prec_aas = ds_ratios['prec_AAS_sorted']
    prec_medians = ds_ratios['prec_medians_sorted']
    
    prec_90th_aas_pcntile = ds_ratios['Prec_AAS_90percentile_sorted']
    prec_90th_pcntile = ds_ratios['Prec_90percentile_sorted']
    prec_10th_aas_pcntile = ds_ratios['Prec_AAS_10percentile_sorted']
    prec_10th_pcntile = ds_ratios['Prec_10percentile_sorted']
    prec_90th_pcntile = [prec_10th_pcntile[prec_10th_aas_pcntile.index(aas)] if prec_medians[i]<0 else prec_90th_pcntile[prec_90th_aas_pcntile.index(aas)] for i,aas in enumerate(prec_aas)]
    
    prec_aas_max= ds_ratios['Prec_AAS_max_sorted']
    prec_max = ds_ratios['Prec_max_sorted']
    prec_aas_min= ds_ratios['Prec_AAS_min_sorted']
    prec_min = ds_ratios['Prec_min_sorted']
    prec_max = [prec_min[prec_aas_min.index(aas)] if prec_medians[i]<0 else prec_max[prec_aas_max.index(aas)] for i,aas in enumerate(prec_aas)]
    prec_n = [ds_ratios['Precursor_N_AAS'][aas] for aas in prec_aas]
    for i in range(len(prec_aas)):
        plot_rows.append([ds,prec_aas[i],prec_medians[i], prec_90th_pcntile[i],prec_max[i], 'Precursor', prec_n[i]])
    
    if ds!='Healthy':
        reporter_aas = ds_ratios['AAS_sorted']
        reporter_medians = ds_ratios['medians_sorted']
        reporter_90th_aas_pcntile = ds_ratios['Reporter_AAS_90percentile_sorted']
        reporter_90th_pcntile = ds_ratios['Reporter_90percentile_sorted']   
        reporter_10th_aas_pcntile = ds_ratios['Reporter_AAS_10percentile_sorted']
        reporter_10th_pcntile = ds_ratios['Reporter_10percentile_sorted']
        reporter_medians = [reporter_medians[i] for i,x in enumerate(reporter_aas) if x in reporter_90th_aas_pcntile] 
        reporter_aas = [x for x in reporter_aas if x in reporter_90th_aas_pcntile]
        reporter_90th_pcntile = [reporter_10th_pcntile[reporter_10th_aas_pcntile.index(aas)] if reporter_medians[i]<0 else reporter_90th_pcntile[reporter_90th_aas_pcntile.index(aas)] for i,aas in enumerate(reporter_aas)]
        reporter_aas_max= ds_ratios['Reporter_AAS_max_sorted']
        reporter_max = ds_ratios['Reporter_max_sorted']
        reporter_aas_min= ds_ratios['Reporter_AAS_min_sorted']
        reporter_min = ds_ratios['Reporter_min_sorted']
        reporter_max = [reporter_min[reporter_aas_min.index(aas)] if reporter_medians[i]<0 else reporter_max[reporter_aas_max.index(aas)] for i,aas in enumerate(reporter_aas)]
        reporter_n = [ds_ratios['Reporter_N_AAS'][aas] for aas in reporter_aas]
        for i in range(len(reporter_aas)):
            plot_rows.append([ds,reporter_aas[i],reporter_medians[i], reporter_90th_pcntile[i],reporter_max[i], 'Reporter', reporter_n[i]])

all_plot_df = pd.DataFrame(plot_rows,columns=plot_cols)
all_plot_df.to_excel(outdir+'RAAS_by_AAS_summary_stats_allDS.xlsx')


# collapse to get data for all datasets together

set_aas = list(set(all_plot_df['AAS type'].to_list()))
all_ds_plot_rows = []
plot_cols = ['AAS type','Median Log$_{10}$ RAAS', '90th percentile Log$_{10}$ RAAS','Max Log$_{10}$ RAAS', 'N']+['N '+ds for ds in datasets]
for aas in set_aas:
    aas_df = all_plot_df.loc[all_plot_df['AAS type']==aas]
    aas_df = aas_df.loc[((aas_df['Dataset']!='Healthy') & (aas_df['RAAS type']=='Reporter')) | (aas_df['Dataset']=='Healthy')]
    med = aas_df['Median Log$_{10}$ RAAS'].median()
    if med < 0:
        pcnt90 = min(aas_df['90th percentile Log$_{10}$ RAAS'])
    else:
        pcnt90 = np.max(aas_df['90th percentile Log$_{10}$ RAAS'])
    if med < 0:
        raas_max = min(aas_df['Max Log$_{10}$ RAAS'])
    else:
        raas_max = np.max(aas_df['Max Log$_{10}$ RAAS'])
    n = aas_df['N'].sum()
    ccrcc_n = aas_df.loc[aas_df['Dataset']=='CCRCC']
    if len(ccrcc_n)>0:
        ccrcc_n = ccrcc_n['N'].values[0]
    else:
        ccrcc_n = 0
    brca_n = aas_df.loc[aas_df['Dataset']=='BRCA']
    if len(brca_n)>0:
        brca_n = brca_n['N'].values[0]
    else:
        brca_n = 0
    ucec_n = aas_df.loc[aas_df['Dataset']=='UCEC']
    if len(ucec_n)>0:
        ucec_n = ucec_n['N'].values[0]
    else:
        ucec_n = 0
    luad_n = aas_df.loc[aas_df['Dataset']=='LUAD']
    if len(luad_n)>0:
        luad_n = luad_n['N'].values[0]
    else:
        luad_n = 0
    pdac_n = aas_df.loc[aas_df['Dataset']=='PDAC']
    if len(pdac_n)>0:
        pdac_n = pdac_n['N'].values[0]
    else:
        pdac_n = 0
    lscc_n = aas_df.loc[aas_df['Dataset']=='LSCC']
    if len(lscc_n)>0:
        lscc_n = lscc_n['N'].values[0]
    else:
        lscc_n = 0
    healthy_n = aas_df.loc[aas_df['Dataset']=='Healthy']
    if len(healthy_n)>0:
        healthy_n = healthy_n['N'].values[0]
    else:
        healthy_n = 0
    all_ds_plot_rows.append([aas, med, pcnt90, raas_max, n, ccrcc_n, ucec_n, brca_n, luad_n, pdac_n, lscc_n, healthy_n])

all_ds_plot_df = pd.DataFrame(all_ds_plot_rows, columns=plot_cols)
all_ds_plot_df.to_excel(outdir+'Reporter_RAAS_by_AAS_data_for_allDS_plot.xlsx')

In [ ]:
# plot barplots with # SAAP with each AAS and median/90th percentile RAAS for each AAS type 

# these AAS are highlighted with red lines and in the volcano plot
aas2plot= ['H to Q', 'H to D', 'E to D', 'T to S', 'Q to G', 'S to G']

plot_df = all_ds_plot_df
subs_in_plot = list(set(plot_df['AAS type']))
medians = [np.mean([x for x in plot_df.loc[plot_df['AAS type']==sub,'Median Log$_{10}$ RAAS'].values]) for sub in subs_in_plot]
sorted_idx = np.argsort(medians)
sorted_medians = [medians[i] for i in sorted_idx]
sorted_subs = [subs_in_plot[i] for i in sorted_idx]

sns.set_style('whitegrid')
f = plt.figure()
gs = gridspec.GridSpec(2,1,height_ratios=[1,2], hspace=0.05)
ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1])

sns.barplot(data=plot_df,x='AAS type', y='90th percentile Log$_{10}$ RAAS', color='#aaaaaa', order=sorted_subs, linewidth=0.05, errorbar=None, ax=ax1, label='90th percentile')#, errwidth=0.1)
sns.barplot(data=plot_df,x='AAS type', y='Median Log$_{10}$ RAAS', color='#555555', order=sorted_subs, linewidth=0.05, errorbar=None, ax=ax1, label='Median')#, errwidth=0.1)
ax1.set_xticks([]);
ax1.set_ylabel(r'log$_{10}$(RAAS)', fontsize=14)
ax1.set_xlabel('Substitution type', fontsize=14)
ax1.tick_params('y', labelsize=13)

sns.barplot(data=plot_df,x='AAS type', y='N', color='black', order=sorted_subs, linewidth=0.0, errorbar=None, ax=ax0)#, errwidth=0.1)
for aas in aas2plot:
    aas_df = plot_df.loc[plot_df['AAS type']==aas]
    max_raas = aas_df['90th percentile Log$_{10}$ RAAS'].values[0]
    n = aas_df['N'].values[0]
    idx = sorted_subs.index(aas)
    plt.plot((idx,idx), (max_raas, 0), '-r', linewidth=2)
    ax0.plot((idx,idx), (1, n), '-r', linewidth=2)

ax0.set_xticks([]);
ax0.set_xlabel('')
ax0.set_yscale('log')#, basey=2)
ax0.grid(which='major', visible=True)
#ax0.set_title(ds, fontsize=14)
ax0.set_ylabel('log$_{10}$(# SAAP)', fontsize=14, labelpad=12)
ax0.set_yticks([10,100, 1000])
ax0.set_yticklabels(['1','2','3'])
ax1.set_yticks([-4,-2,0, 2])
ax0.tick_params('y',labelsize=13)
ax1.legend(bbox_to_anchor=(0.38,0.95), fontsize=11, frameon=False, labelspacing=0.3)#, loc='upper left')
ax1.yaxis.set_major_locator(FixedLocator([-6,-4,-2,0, 2]))
ax0.margins(x=0)
ax1.margins(x=0)
plt.savefig(outdir+'All_data_reporterRAAS_by_AAS_mean_90th_barplot.pdf', bbox_inches='tight')

In [ ]:
# plot the volcano plot highlighting AAS with large N across range of RAAS distributions

fig,axes = plt.subplots(1, len(aas2plot), figsize=(7,2), sharey=True)
plt.subplots_adjust(wspace=0)
sns.set_style('whitegrid')

quant_df = filt_reporter_quant_df
for i,aas in enumerate(aas2plot):
    print(aas)
    ax = axes[i]
    ax.set_xlabel(aas, fontsize=15)
    ax.set_xticks([])    
    aas_raas_data = quant_df.loc[quant_df['AAS']==aas, 'RAAS'].values
    aas_raas_data = [x for x in aas_raas_data if ~np.isnan(x) and ~np.isinf(x)]
    
    bihist(aas_raas_data, aas_raas_data, nbins=30, h=ax)
    if i>0:
        ax.spines['left'].set_visible(False)
    ax.spines['top'].set_visible(False)
    if ax==axes[-1]:
        ax.spines['right'].set_visible(False)
    if ax==axes[0]:
        ax.annotate('N= ', (0,1.01), xycoords='axes fraction', fontsize=13)
    ax.annotate('{:,}'.format((len(aas_raas_data))), (0.3,1.01), xycoords='axes fraction', fontsize=13)
axes[0].set_yticks([-4,-2,0,2])
axes[0].set_ylabel(r'log$_{10}$(RAAS)', fontsize=16)
axes[0].tick_params('y', labelsize=15)
plt.savefig(outdir+'All_DS_example_AAStype_RAASdistributions_bihist.pdf', bbox_inches='tight')    

### Extended Data Figure 5g: Median RAAS for Encoded and Incoporated AAS types correlations across datasets

In [5]:
# add median precursor and reporter ion RAAS across all peptides identified with each base (encoded) or incorporated AA to ratios_by_aas dictionary

aa_list = sorted(list(set(x[0] for x in AASs)))

# add median precursor RAAS across all peptides with each base AA
for ds in datasets: 
    print(ds)
    aa_ratio_rows = []
    aa_ratio_cols = ['Base AA', 'RAAS']
    ds_ratio_df = ratios_by_aas[ds]['Precursor_ratio_df']
    for aa in aa_list:
        aa_idx = [i for i,row in ds_ratio_df.iterrows() if row['AAS'][0]==aa]
        if len(aa_idx)>0:
            aa_df = ds_ratio_df.loc[aa_idx]
            for j, row in aa_df.iterrows():
                aa_ratio_rows.append([aa, row['RAAS']])
    ds_aa_df = pd.DataFrame(aa_ratio_rows, columns = aa_ratio_cols)  
    aa_medians = [np.nanmedian([x for x in ds_aa_df.loc[ds_aa_df['Base AA']==aa,'RAAS'].values]) for aa in aa_list]
    aa_sorted_idx = np.argsort(aa_medians)
    aa_sorted_medians = [aa_medians[i] for i in aa_sorted_idx]
    aa_sorted_subs = [aa_list[i] for i in aa_sorted_idx]
    ratios_by_aas[ds]['Base_prec_AA_RAAS_df'] = ds_aa_df
    ratios_by_aas[ds]['Base_prec_AAS_sorted'] = aa_sorted_subs
    ratios_by_aas[ds]['Base_prec_medians_sorted'] = aa_sorted_medians

# add median reporter RAAS across all peptides with each base AA
for ds in datasets: 
    print(ds)
    if ds!='Healthy':
        aa_ratio_rows = []
        aa_ratio_cols = ['Base AA', 'RAAS']
        ds_ratio_df = ratios_by_aas[ds]['Reporter_ratio_df']
        for aa in aa_list:
            aa_idx = [i for i,row in ds_ratio_df.iterrows() if row['AAS'][0]==aa]
            if len(aa_idx)>0:
                aa_df = ds_ratio_df.loc[aa_idx]
                for j, row in aa_df.iterrows():
                    aa_ratio_rows.append([aa, row['RAAS']])
        ds_aa_df = pd.DataFrame(aa_ratio_rows, columns = aa_ratio_cols)  
        aa_medians = [np.nanmedian([x for x in ds_aa_df.loc[ds_aa_df['Base AA']==aa,'RAAS'].values]) for aa in aa_list]
        aa_sorted_idx = np.argsort(aa_medians)
        aa_sorted_medians = [aa_medians[i] for i in aa_sorted_idx]
        aa_sorted_subs = [aa_list[i] for i in aa_sorted_idx]
        ratios_by_aas[ds]['Base_reporter_AA_RAAS_df'] = ds_aa_df
        ratios_by_aas[ds]['Base_reporter_AAS_sorted'] = aa_sorted_subs
        ratios_by_aas[ds]['Base_reporter_medians_sorted'] = aa_sorted_medians
        
# add median precursor RAAS across all peptides with each susbtituted/destination AA  
for ds in datasets: 
    print(ds)
    aa_ratio_rows = []
    aa_ratio_cols = ['Sub AA', 'RAAS']
    ds_ratio_df = ratios_by_aas[ds]['Precursor_ratio_df']
    for aa in aa_list:
        aa_idx = [i for i,row in ds_ratio_df.iterrows() if row['AAS'][-1]==aa]
        if len(aa_idx)>0:
            aa_df = ds_ratio_df.loc[aa_idx]
            for j, row in aa_df.iterrows():
                aa_ratio_rows.append([aa, row['RAAS']])
    ds_aa_df = pd.DataFrame(aa_ratio_rows, columns = aa_ratio_cols)  
    aa_medians = [np.nanmedian([x for x in ds_aa_df.loc[ds_aa_df['Sub AA']==aa,'RAAS'].values]) for aa in aa_list]
    aa_sorted_idx = np.argsort(aa_medians)
    aa_sorted_medians = [aa_medians[i] for i in aa_sorted_idx]
    aa_sorted_subs = [aa_list[i] for i in aa_sorted_idx]
    ratios_by_aas[ds]['Sub_AA_RAAS_df'] = ds_aa_df
    ratios_by_aas[ds]['Sub_AAS_sorted'] = aa_sorted_subs
    ratios_by_aas[ds]['Sub_medians_sorted'] = aa_sorted_medians

# add median reporter RAAS across all peptides with each susbtituted/destination AA  
for ds in datasets: 
    print(ds)
    if ds!='Healthy':
        aa_ratio_rows = []
        aa_ratio_cols = ['Sub AA', 'RAAS']
        ds_ratio_df = ratios_by_aas[ds]['Reporter_ratio_df']
        for aa in aa_list:
            aa_idx = [i for i,row in ds_ratio_df.iterrows() if row['AAS'][-1]==aa]
            if len(aa_idx)>0:
                aa_df = ds_ratio_df.loc[aa_idx]
                for j, row in aa_df.iterrows():
                    aa_ratio_rows.append([aa, row['RAAS']])
        ds_aa_df = pd.DataFrame(aa_ratio_rows, columns = aa_ratio_cols)  
        aa_medians = [np.nanmedian([x for x in ds_aa_df.loc[ds_aa_df['Sub AA']==aa,'RAAS'].values]) for aa in aa_list]
        aa_sorted_idx = np.argsort(aa_medians)
        aa_sorted_medians = [aa_medians[i] for i in aa_sorted_idx]
        aa_sorted_subs = [aa_list[i] for i in aa_sorted_idx]
        ratios_by_aas[ds]['Sub_reporter_AA_RAAS_df'] = ds_aa_df
        ratios_by_aas[ds]['Sub_reporter_AAS_sorted'] = aa_sorted_subs
        ratios_by_aas[ds]['Sub_reporter_medians_sorted'] = aa_sorted_medians
    
pickle.dump(ratios_by_aas, open(outdir+'Ratios_by_AAS.p', 'wb'))

In [ ]:
# generate heatmap dataframe that will be used to compute correlations of Base AA RAAS across datasets
heatmap_data = pd.DataFrame(index=aa_list, columns=datasets)
for i, row in heatmap_data.iterrows():
    for ds in datasets:
        if ds!='Healthy':
            medians = ratios_by_aas[ds]['Base_reporter_medians_sorted']
            aas = ratios_by_aas[ds]['Base_reporter_AAS_sorted']
        else:
        medians = ratios_by_aas[ds]['Base_prec_medians_sorted']
        aas = ratios_by_aas[ds]['Base_prec_AAS_sorted']
        for i,aa in enumerate(aa_list):
            heatmap_data.loc[aa,ds] = medians[aas.index(aa)]

# compute correlations of vectors of Base AA median RAAS across datasets 
heatmap_df_4corr = heatmap_df.dropna(how='any')
corr_heatmap = pd.DataFrame(index=datasets, columns=datasets)
for ds1 in datasets:
    for ds2 in datasets:
        corr = sp.stats.pearsonr(heatmap_df_4corr[ds1].values, heatmap_df_4corr[ds2].values)[0]
        corr_heatmap.loc[ds1,ds2] = corr
corr_heatmap = corr_heatmap.astype(float)

# plot the correlation heatmap for base AA RAAS across datasets 
s= sns.clustermap(corr_heatmap, figsize=(5,5), annot=np.round(corr_heatmap, 2), cmap='Reds', cbar_kws={'label':'Pearson correlation', 'pad':1}, annot_kws={"fontsize":12},
                 vmax=1, vmin=0, row_cluster=False, col_cluster=False)

s.ax_heatmap.tick_params(right=False, left=True, labelright=False, labelleft=True, labelbottom=True, bottom=True)
plt.setp(s.ax_heatmap.xaxis.get_majorticklabels(), rotation=90, fontsize=13)
plt.setp(s.ax_heatmap.yaxis.get_majorticklabels(), fontsize=13)
s.ax_cbar.set_position([0.8,0.26,0.05,0.5])
s.ax_cbar.yaxis.label.set_size(14)
s.ax_cbar.set_yticklabels([0,0.2,0.4,0.6,0.8, 1], fontsize=13)
s.ax_row_dendrogram.set_visible(False)
s.ax_col_dendrogram.set_visible(False)
plt.savefig(outdir+'Correlation_heatmap_BaseAA_reporterRatios_unclustered.pdf', bbox_inches='tight')

In [ ]:
# generate heatmap dataframe that will be used to compute correlations of substituted AA RAAS across datasets
heatmap_data = pd.DataFrame(index=aa_list, columns=datasets)
for i, row in heatmap_data.iterrows():
    for ds in datasets:
        if ds!='Healthy':
            medians = ratios_by_aas[ds]['Sub_reporter_medians_sorted']
            aas = ratios_by_aas[ds]['Sub_reporter_AAS_sorted']
        else:
        medians = ratios_by_aas[ds]['Sub_prec_medians_sorted']
        aas = ratios_by_aas[ds]['Sub_prec_AAS_sorted']
        for i,aa in enumerate(aa_list):
            heatmap_data.loc[aa,ds] = medians[aas.index(aa)]

# compute correlations of vectors of Substituted AA median RAAS across datasets 
heatmap_df_4corr = heatmap_df.dropna(how='any')
corr_heatmap = pd.DataFrame(index=datasets, columns=datasets)
for ds1 in datasets:
    for ds2 in datasets:
        corr = sp.stats.pearsonr(heatmap_df_4corr[ds1].values, heatmap_df_4corr[ds2].values)[0]
        corr_heatmap.loc[ds1,ds2] = corr
corr_heatmap = corr_heatmap.astype(float)

# plot the correlation heatmap for substituted AA RAAS across datasets 
s= sns.clustermap(corr_heatmap, figsize=(5,5), annot=np.round(corr_heatmap, 2), cmap='Reds', cbar_kws={'label':'Pearson correlation', 'pad':1}, annot_kws={"fontsize":12},
                 vmax=1, vmin=0, row_cluster=False, col_cluster=False)

s.ax_heatmap.tick_params(right=False, left=True, labelright=False, labelleft=True, labelbottom=True, bottom=True)
plt.setp(s.ax_heatmap.xaxis.get_majorticklabels(), rotation=90, fontsize=13)
plt.setp(s.ax_heatmap.yaxis.get_majorticklabels(), fontsize=13)
s.ax_cbar.set_position([0.8,0.26,0.05,0.5])
s.ax_cbar.yaxis.label.set_size(14)
s.ax_cbar.set_yticklabels([0,0.2,0.4,0.6,0.8, 1], fontsize=13)
s.ax_row_dendrogram.set_visible(False)
s.ax_col_dendrogram.set_visible(False)
plt.savefig(outdir+'Correlation_heatmap_SubstitutedAA_reporterRatios_unclustered.pdf', bbox_inches='tight')

### Figure 3f: ANOVA of RAAS variance by data type, AAS type, tissue type

In [ ]:
# get dataframe for ANOVA analysis

# list of tissues each CPTAC dataset represents, in same order as dataset_list 
cptac_tissue_list = ['kidney', 'endometrium', 'breast', 'lung', 'pancreas', 'lung']

anova_rows = []
anova_cols = ['SAAP', 'BP', 'AAS', 'RAAS', 'Tissue', 'Dataset', 'Data_type'] # data type will be TMT or label-free
for i,row in filt_reporter_quant_df.iterrows():
    ds = row['Dataset']
    anova_rows.append([row['SAAP'], row['BP'], row['AAS'], row['RAAS'], cptac_tissue_list[datasets.index(ds)], ds, 'TMT-labeled'])
healthy_df = filt_prec_quant_df.loc[filt_prec_quant_df['Dataset']=='Healthy']
for i,row in healthy_df.iterrows():
    anova_rows.append([row['SAAP'], row['BP'], row['AAS'], row['RAAS'], row['TMT/Tissue'], 'Healthy', 'Label-free'])

    anova_df = pd.DataFrame(anova_rows, columns=anova_cols)
anova_df.replace(np.inf, np.nan, inplace=True)
anova_df.replace(-np.inf, np.nan, inplace=True)
anova_df.dropna(how='any', inplace=True)

anova_df['Encoded'] = [x.split(' to ')[0] for x in anova_df.AAS.to_list()]
anova_df['Incorporated'] = [x.split(' to ')[1] for x in anova_df.AAS.to_list()]
anova_df.to_excel(outdir+'ANOVA_AAS_tissue_df.xlsx')

In [ ]:
# generate ordinary least squares model with variables of interest (tissue type, data type, AAS type, etc)
model = ols('RAAS ~ C(AAS) + C(Encoded) + C(Incorporated) + C(Tissue) + C(Data_type)', data=anova_df).fit()
df = sm.stats.anova_lm(model, typ=3)
df.reset_index(inplace=True)
df.columns = ['Variable']+df.columns.to_list()[1:]

# plot results
plot_df = df.loc[1:5, ['Variable', 'F', 'PR(>F)']]
plot_df = df.loc[1:5, ['Variable', 'F', 'PR(>F)']]
plot_df['PR(>F)'] = ['{:.2e}'.format(x) for x in plot_df['PR(>F)']]
plot_df['Variable'] = ['Substitution\ntype', 'Encoded\namino acid', 'Incorporated\namino acid', 'Tissue', 'Data type']

sns.set_style('whitegrid')
fig,ax = plt.subplots(figsize=(5,4))
sns.barplot(data=plot_df, x='Variable', y='F', hue='PR(>F)', dodge=False, palette='Greys',
           order=['Data type', 'Incorporated\namino acid', 'Encoded\namino acid', 'Tissue','Substitution\ntype'])
handles, labels = ax.get_legend_handles_labels()
plt.legend(handles=handles, labels=['p$<10^{-100}$', 'p$<10^{-51}$', 'p$<10^{-23}$', 'p$<10^{-8}$'], loc='upper left', fontsize=13)
plt.xlabel('')
plt.ylabel('ANOVA F statistic', fontsize=15)
ax.tick_params('both', labelsize=14)
ax.tick_params('x', rotation=90)
plt.savefig(outdir+'ANOVA_tissue_AAS_results_extended.pdf', bbox_inches='tight')

### Extended Data Figure 5h,i:  AAS type enrichment in tissue types

In [ ]:
# set of all AAS types
aas_list = list(set(filt_saap_df['AAS'].to_list()))

# get index of rows in precursor quant data (Extended_Data_2) corresponding to tissue types represented by CPTAC and healthy datasets 
kidney_rows = [i for i,row in filt_prec_quant_df.iterrows() if row['Dataset']=='CCRCC' or row['TMT/Tissue']=='kidney']
lung_rows = [i for i,row in filt_prec_quant_df.iterrows() if row['Dataset']=='LUAD' or row['Dataset']=='LSCC' or row['TMT/Tissue']=='lung']
pancreas_rows = [i for i,row in filt_prec_quant_df.iterrows() if row['Dataset']=='PDAC' or row['TMT/Tissue']=='pancreas']
endometrium_rows = [i for i,row in filt_prec_quant_df.iterrows() if row['Dataset']=='UCEC' or row['TMT/Tissue']=='endometrium']


def get_aas_fc_alt(aas, tissue_rows):
    """
    Function to get the log2 fold change and pvalue between RAAS for peptides with an AAS type in a given tissue vs RAAS for peptides with that AAS type in all other tissues
    Input: AAS type, index in precursor quant df of tissues of interest (extracted above)
    Output: Log2FC and Pvalue of RAAS comparison for AAS across tissues
    Requires at least 10 peptides with AAS in tissue type and other tissue types for comparison
    """
    tissue_df = filt_prec_quant_df.loc[tissue_rows]
    tissue_raas = [x for x in tissue_df.loc[tissue_df['AAS']==aas, 'RAAS'].to_list() if ~np.isnan(x) and ~np.isinf(x)]
    other_rows = [i for i in filt_prec_quant_df.index if i not in tissue_rows]
    other_df = filt_prec_quant_df.loc[other_rows]
    other_raas = [x for x in other_df.loc[other_df['AAS']==aas, 'RAAS'].to_list() if ~np.isnan(x) and ~np.isinf(x)]   
    if len(tissue_raas)>10 and len(other_raas)>10:
        l2fc = np.log2(10**(np.mean(tissue_raas)-np.mean(other_raas)))
        pval = sp.stats.ttest_ind(tissue_raas, other_raas)[1]
    else:
        l2fc = np.nan
        pval = np.nan
    return(l2fc, pval)

# get dataframe with fold change and pvalues for AAS type RAAS in given tissue vs all other tissues 
aas_fc_rows = []
aas_fc_cols = ['AAS', 'Tissue', 'RAAS FC', 'pvalue']
for aas in aas_list:
    kidney_fc, kidney_pval = get_aas_fc_alt(aas, kidney_rows)
    lung_fc, lung_pval = get_aas_fc_alt(aas, lung_rows)
    pancreas_fc, pancreas_pval = get_aas_fc_alt(aas, pancreas_rows)
    endometrium_fc, endometrium_pval = get_aas_fc_alt(aas, endometrium_rows)
    aas_fc_rows.append([aas, 'Kidney', kidney_fc, kidney_pval])
    aas_fc_rows.append([aas, 'Lung', lung_fc, lung_pval])
    aas_fc_rows.append([aas, 'Pancreas', pancreas_fc, pancreas_pval])
    aas_fc_rows.append([aas, 'Endometrium', endometrium_fc, endometrium_pval])
aas_fc_df_alt = pd.DataFrame(aas_fc_rows, columns=aas_fc_cols)
aas_fc_df_alt.dropna(how='any', axis=0, inplace=True)

# adjust pvalues for multiple testing
aas_fc_df_alt['padj'] = multipletests(aas_fc_df_alt['pvalue'].to_list(), method='fdr_bh')[1]
aas_fc_df_alt['-log q'] = [-1*np.log10(x) for x in aas_fc_df_alt['padj']]
aas_fc_df_alt['Significant'] = ['Significant' if row['padj']<=0.01 and np.abs(row['RAAS FC']>=1) else 'Not significant' for i,row in aas_fc_df_alt.iterrows()]
aas_fc_df.to_excel(outdir+'AAS_RAAS_tissue_FC_n10.xlsx')

In [ ]:
# plot volcano plot of tissue type RAAS significance for AAS types (Ext. Data Fig. 5h)

sns.set_style('ticks')
fig,ax = plt.subplots(figsize=(3,3))
sns.scatterplot(data=aas_fc_df_alt, y='-log q', x='RAAS FC', hue='Hue', palette=['#aaaaaa']+colors)
handles, labels = ax.get_legend_handles_labels()
plt.legend(handles=handles, labels=labels, title='', fontsize=11, handletextpad=0, bbox_to_anchor=(0.48,1.08), ncol=2, frameon=False, loc='center', columnspacing=0.2)

plt.ylabel('-log$_{10}$(q)', fontsize=14)
plt.xlabel('log$_{2}$(RAAS$_{tissue}/$RAAS$_{other}$)', fontsize=14)
ax.tick_params('both', labelsize=13)
plt.plot((-1,-1), ax.get_ylim(), '--k' , linewidth=0.5)
plt.plot((1,1), ax.get_ylim(), '--k' , linewidth=0.5)
plt.plot(ax.get_xlim(), (2,2), '--k' , linewidth=0.5)

sig_df = aas_fc_df_alt.loc[aas_fc_df_alt['Significant']=='Significant']
texts = []
for i,row in sig_df.iterrows():
    x=row['RAAS FC']
    y=row['-log q']
    s=row['AAS'][0]+r'$\rightarrow$'+row['AAS'][-1]
    if x>0:
        texts.append(ax.text(x,y,s,fontsize=11))
adjust_text(texts, expand_points=(1,1), expand_text=(1,2), ax=fig.axes[0],
            arrowprops=dict(arrowstyle='->', color='#555555', lw=1), force_text=0.3)

plt.savefig(outdir+'Tissue_AAS_highRAAS_volcano.pdf', bbox_inches='tight')

In [ ]:
# create dataframe with RAAS values and number of peptides for AAS types with significantly high RAAS in given tissue

plot_rows = []
plot_cols = ['Tissue', 'RAAS', 'Type', 'N', 'AAS']
sig_df = aas_fc_df_alt.loc[aas_fc_df_alt['Significant']=='Significant']
sig_df = sig_df.loc[sig_df['RAAS FC']>0]
sig_df.sort_values('Tissue', inplace=True)
for i,row in sig_df.iterrows():
    aas = row['AAS']
    aas_str = aas[0]+r'$\rightarrow$'+aas[-1]
    tissue = row['Tissue']
    print(aas, tissue)
    if tissue == 'Kidney':
        tissue_rows = kidney_rows
    elif tissue =='Lung':
        tissue_rows = lung_rows
    elif tissue == 'Pancreas':
        tissue_rows = pancreas_rows
    elif tissue == 'Endometrium':
        tissue_rows = endometrium_rows
    tissue_df = filt_prec_quant_df.loc[tissue_rows]
    tissue_raas = [x for x in tissue_df.loc[tissue_df['AAS']==aas, 'RAAS'].to_list() if ~np.isnan(x) and ~np.isinf(x)]
    n_tissue = len(tissue_raas)
    other_rows = [i for i in filt_prec_quant_df.index if i not in tissue_rows]
    other_df = filt_prec_quant_df.loc[other_rows]
    other_raas = [x for x in other_df.loc[other_df['AAS']==aas, 'RAAS'].to_list() if ~np.isnan(x) and ~np.isinf(x)]   
    n_other = len(other_raas)
    for raas in tissue_raas:
        plot_rows.append([tissue, raas, 'Significant tissue', n_tissue, aas_str])#+'\n'+tissue])
    for raas in other_raas:
        plot_rows.append([tissue, raas, 'Other tissues', n_other, aas_str])#+'\n'+tissue])

plot_df = pd.DataFrame(plot_rows, columns=plot_cols)
order = [r'L$\rightarrow$S', r'G$\rightarrow$S', r'G$\rightarrow$Q', r'T$\rightarrow$V', r'A$\rightarrow$T', r'N$\rightarrow$S',r'S$\rightarrow$Q', r'T$\rightarrow$S',  r'C$\rightarrow$S', r'Q$\rightarrow$G']


# plot boxplots of RAAS distributions for AAS types with significantly high RAAS in a specific tissue (Ext. Data Fig. 5i)
# plot barplot of number of SAAP with AAS type in specific tissue and all other tissues 
sns.set_style('whitegrid')
fig,axes = plt.subplots(1,2,figsize=(3,3), gridspec_kw={'width_ratios':[3,1]}, sharey=True)
plt.subplots_adjust(wspace=0.01)
bx = sns.boxplot(data=plot_df, y='AAS', x='RAAS', hue='Type', palette=[colors[0], '#AAAAAA'], 
                 fliersize=0.5, linewidth=1, ax=axes[0], order=order)
axes[0].tick_params('both', labelsize=13)
axes[0].legend().remove()
axes[0].set_ylabel('')
axes[0].set_xlabel('log$_{10}$(RAAS)', fontsize=14)

br = sns.barplot(data=plot_df, y='AAS', x='N', hue='Type', palette=[colors[0], '#AAAAAA'], ax=axes[1], order=order)
axes[1].legend().remove()
axes[1].tick_params('both', labelsize=13)
axes[1].set_ylabel('')
axes[1].set_xscale('log')
axes[1].set_xlabel('# SAAP', fontsize=14)

g = '#AAAAAA'
custom_colors = [colors[0]]*2+[colors[0],g]+ [colors[1], g]*6+[colors[2], g]*3
for i, patch in enumerate(bx.patches):
    patch.set_facecolor(custom_colors[i % len(custom_colors)])
    patch.set_alpha(0.8)

custom_br_colors = [colors[0]]+[colors[1]]*6+[colors[2]]*3+[g]*10
for i, patch in enumerate(br.patches):
    patch.set_facecolor(custom_br_colors[i % len(custom_colors)])
    patch.set_alpha(1)

plt.savefig(outdir+'Sig_tissue_AAS_highRAAS_boxplots_orderedbyFC.pdf', bbox_inches='tight')

# Figure 4. Tumor vs. Normal analysis

#### Incl. Extended Data Figure 6

### Extended Data Figure 6a: Distribution of RAAS fold changes between tumor and normal samples 

In [ ]:
# create dictionary with fold changes for each SAAP's RAAS computed between normal and tumor samples from same patient
# uses saap_protein_df, i.e. Supplemental_Data_2.SAAP_proteins.xlsx (read in earlier in script)
# uses reporter_quant_df, i.e. Supplemental_Data_4.SAAP_reporter_quant.xlsx (read in earlier in script)

all_saap_protein_df = pd.read_excel(proj_dir+'Supplemental_Data_2.SAAP_proteins.xlsx', index_col=0)
reporter_quant_df = pd.read_excel(proj_dir+'Supplemental_Data_4.SAAP_reporter_quant.xlsx', index_col=0)
tvn_datasets = [x for x in datasets if x!='BRCA' and x!='Healthy'] # BRCA not included because only tumor data
reporter_quant_df['Patient ID'] = [x.split('_')[0] for x in reporter_quant_df['Sample name'].values]


patient_level_tvn_dict = {ds:{} for ds in tvn_datasets}
for ds in tvn_datasets: 
    print(ds)
    ds_saap_df = all_saap_protein_df.loc[all_saap_protein_df['Dataset']==ds]
    ds_quant_df = reporter_quant_df.loc[reporter_quant_df['Dataset']==ds]
    set_patients = list(set(reporter_quant_df['Patient ID'].values))
    patient_level_tvn_rows = []
    patient_level_tvn_cols = ['Dataset', 'SAAP', 'BP','Patient ID', 'Tumor TMT set', 'Normal TMT set', 'Tumor SAAP', 'Normal SAAP','Log$_2$Tumor/Normal SAAP', 'Tumor BP', 'Normal BP','Log$_2$Tumor/Normal BP', 'Tumor RAAS', 'Normal RAAS', 'Log$_2$Tumor/Normal RAAS']

    for i,row in ds_saap_df.iterrows():
        saap = row['SAAP']
        bp = row['BP']
        saap_df = ds_quant_df.loc[(ds_quant_df['SAAP']==saap) & (ds_quant_df['BP']==bp)]
        for patient in set_patients:
            patient_df = saap_df.loc[saap_df['Patient ID']==patient]
            if 'Tumor' in patient_df['Sample type'].values and 'Normal' in patient_df['Sample type'].values:
                t_row = patient_df.loc[patient_df['Sample type']=='Tumor']
                n_row = patient_df.loc[patient_df['Sample type']=='Normal']

                t_saap = t_row['SAAP abundance'].values[0]
                t_bp = t_row['BP abundance'].values[0]
                t_raas = t_row['RAAS'].values[0]
                t_tmt = t_row['TMT set'].values[0]

                n_saap = n_row['SAAP abundance'].values[0]
                n_bp = n_row['BP abundance'].values[0]
                n_raas = n_row['RAAS'].values[0]
                n_tmt = n_row['TMT set'].values[0]    

                saap_fc = np.log2(t_saap/n_saap)
                bp_fc = np.log2(t_bp/n_bp)
                raas_fc = np.log2((10**t_raas)/(10**n_raas))

                patient_level_tvn_rows.append([ds, saap, bp, patient, t_tmt, n_tmt,
                                              t_saap, n_saap, saap_fc, t_bp, n_bp, bp_fc, t_raas, n_raas, raas_fc])
    patient_level_tvn_df = pd.DataFrame(patient_level_tvn_rows, columns=patient_level_tvn_cols)
    patient_level_tvn_dict[ds] = patient_level_tvn_df
pickle.dump(patient_level_tvn_dict, open(outdir+'Patient_level_Tumor_v_Normal_data.p','wb'))

In [ ]:
# plot overall distribution of tumor vs normal RAAS value for each SAAP. Computed with matched patient samples

tvn_dict = patient_level_tvn_dict

col2plot = 'Log$_2$Tumor/Normal RAAS'
fig,axes = plt.subplots(1,len(tvn_datasets),figsize=(5,2.5), sharey=True)
sns.set_style('whitegrid')
plt.subplots_adjust(wspace=0)
for ax in axes:
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.tick_params('x', bottom=False,labelbottom=False)
    if ax!=axes[0]:    
        ax.spines['left'].set_visible(False)
        ax.tick_params(left=False)

# sort datasets in plot by median of distribution 
medians = []
for ds in tvn_datasets:
    ds_df = tvn_dict[ds]
    data = [x for x in ds_df[col2plot].values if ~np.isnan(x) and ~np.isinf(x)]
    medians.append(np.median(data))
med_sort_idx = list(np.argsort(medians))

for i,ds in enumerate(tvn_datasets):
    print(ds)
    ax_idx = med_sort_idx.index(i)
    axes[ax_idx].set_xlabel(ds, fontsize=13)
    ds_df = tvn_dict[ds]
    data = [x for x in ds_df[col2plot].values if ~np.isnan(x) and ~np.isinf(x)]
    bihist(data, data, nbins=70,h=axes[ax_idx])
    if ax_idx==0:
        axes[ax_idx].annotate('N = '+str(len(data)),(0.05,1), xycoords='axes fraction', fontsize=12)
    else:
        axes[ax_idx].annotate(str(len(data)), (0.25,1), xycoords='axes fraction', fontsize=12)
    axes[ax_idx].plot((axes[ax_idx].get_xlim()), (np.median(data), np.median(data)), '--r',linewidth=0.7)
axes[0].set_ylabel('log$_2$(Tumor/Normal RAAS)', fontsize=14)
axes[0].tick_params('both', labelsize=13)
plt.ylim([-5,5])
plt.savefig(outdir+'patient_level_Tv_RAAS_bihist.pdf', bbox_inches='tight')

### Extended Data Figure 6c: Tumor vs Normal RAAS in patients for PSMA1 SAAP 

In [ ]:
saap = 'AGSELAAHQK'

# get data for plot from tvn_dict
saap_df_list = []
for ds in tvn_datasets:
    ds_df = tvn_dict[ds]
    saap_df = ds_df.loc[ds_df['SAAP']==saap]
    saap_df_list.append(saap_df)
all_ds_saap_df = pd.concat(saap_df_list)
all_ds_saap_df = all_ds_saap_df.loc[:,['Dataset','Patient ID','Tumor RAAS','Normal RAAS']]
all_ds_saap_df.replace(np.inf, np.nan, inplace=True)
all_ds_saap_df.replace(-np.inf, np.nan, inplace=True)
all_ds_saap_df.dropna(inplace=True, how='any')

# plot scatterplot of tumor vs normal RAAS for datasets 
plot_ds_saap_df = all_ds_saap_df.loc[all_ds_saap_df['Dataset'].isin(['LUAD','PDAC',])]
sns.set_style('whitegrid')
fig,ax = plt.subplots(figsize=(2.5,2.5))
sns.scatterplot(data = plot_ds_saap_df, x='Normal RAAS', y='Tumor RAAS',hue='Dataset', s=35, alpha=0.5)
plt.title(saap[0]+r'$\bf(Q$'+r'$\rightarrow$'+r'$\bfG)$'+saap[2:]+'\nProteasome subunit A, type 1', fontsize=14)
ax.tick_params('both', labelsize=13)
plt.ylabel('Tumor log$_{10}$(RAAS)', fontsize=14)
plt.xlabel('Normal log$_{10}$(RAAS)', fontsize=14)
plt.xlim([-1.5, 1.5])
plt.ylim(plt.xlim())
plt.xticks([-1,0,1]);
plt.plot(plt.ylim(), plt.ylim(), '--k', linewidth=0.8)
handles, labels = ax.get_legend_handles_labels()

# pvalues used in legend are computed below 
plt.legend(handles = handles, labels = ['LUAD, q$<$10$^{-14}$', 'PDAC, q$<$10$^{-3}$'], 
           title='', loc='center', handletextpad=0, labelspacing=0, bbox_to_anchor=(0.8,0.15))
plt.savefig(outdir+saap+'_TRAAS_vs_NRAAS_allDS_PSMA1_PDAC_LUAD_trim.pdf', bbox_inches='tight')

In [ ]:
# showing pvalues between tumor and normal RAAS in patients for PSMA1 SAAP in each dataset
# used in plot annotation 

ps = []
for ds in tvn_datasets:
    print(ds)
    ds_df = all_ds_saap_df.loc[all_ds_saap_df['Dataset']==ds]
    print(sp.stats.ttest_rel(ds_df['Normal RAAS'].values, ds_df['Tumor RAAS'].values, alternative='less'))
    ps.append(sp.stats.ttest_rel(ds_df['Normal RAAS'].values, ds_df['Tumor RAAS'].values, alternative='less')[1])
adj_ps = multipletests(ps, method='fdr_bh')[1]
for i,ds in enumerate(tvn_datasets):
    print(ds, adj_ps[i])

### Figure 4b: Distribution of RAAS for PSMA1 SAAP 

In [ ]:
# plot distribution of RAAS for this PSMA1 SAAP in the CPTAC datasets
cptac_datasets = datasets[:-1]

filt_reporter_df = filt_reporter_quant_df
proteasome_saap = 'AGSELAAHQK'
saap_df = filt_reporter_df.loc[filt_reporter_df['SAAP']==proteasome_saap]

sns.set_style('whitegrid')
col2plot = 'RAAS'
fig,axes = plt.subplots(1,len(cptac_datasets),figsize=(4.2,2.5), sharey=True)
sns.set_style('whitegrid')
plt.subplots_adjust(wspace=0.1)
for ax in axes:
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.tick_params('x', bottom=False,labelbottom=False)
    if ax!=axes[0]:    
        ax.spines['left'].set_visible(False)
        ax.tick_params(left=False)

# sort order of datasets in plot by median RAAS of distribution 
medians = []
for ds in cptac_datasets:
    ds_df = saap_df.loc[saap_df['Dataset']==ds]
    data = [x for x in ds_df[col2plot].values if ~np.isnan(x) and ~np.isinf(x)]
    medians.append(np.median(data))
med_sort_idx = list(np.argsort(medians))

for i,ds in enumerate(cptac_datasets):
    print(ds)
    ax_idx = med_sort_idx.index(i)
    axes[ax_idx].set_xlabel(ds, fontsize=12.5)
    ds_df = saap_df.loc[saap_df['Dataset']==ds]
    data = [x for x in ds_df[col2plot].values if ~np.isnan(x) and ~np.isinf(x)]
    bihist(data, data, nbins=20,h=axes[ax_idx])
    if ax_idx==0:
        axes[ax_idx].annotate('N$=$'+str(len(data)),(-0.1,1.01), xycoords='axes fraction', fontsize=12)
    else:
        axes[ax_idx].annotate(str(len(data)), (0.25,1.01), xycoords='axes fraction', fontsize=12)
    axes[ax_idx].plot((axes[ax_idx].get_xlim()), (np.median(data), np.median(data)), '--r',linewidth=0.7)
axes[0].set_ylabel('log$_{10}$(RAAS)', fontsize=14)
axes[0].tick_params('both', labelsize=13)
fig.text(0.5, 1, r'PSMA1: Proteasome subunit $\alpha$ type-1; Q$\rightarrow$G', ha='center', fontsize=13)
plt.ylim([-2,2])
plt.savefig(outdir+'PSMA1_RAAS_bihist.pdf', bbox_inches='tight')

### Figure 4a: Tumor vs Normal RAAS in patients for LaminA SAAP 

In [ ]:
saap = 'AGNTWGCGNSLR'

# get data for plot from tvn_dict
saap_df_list = []
for ds in tvn_datasets:
    ds_df = tvn_dict[ds]
    saap_df = ds_df.loc[ds_df['SAAP']==saap]
    saap_df_list.append(saap_df)
all_ds_saap_df = pd.concat(saap_df_list)
all_ds_saap_df = all_ds_saap_df.loc[:,['Dataset','Patient ID','Tumor RAAS','Normal RAAS']]
all_ds_saap_df.replace(np.inf, np.nan, inplace=True)
all_ds_saap_df.replace(-np.inf, np.nan, inplace=True)
all_ds_saap_df.dropna(inplace=True, how='any')

# plot scatterplot of tumor vs normal RAAS for datasets 
plot_ds_saap_df = all_ds_saap_df.loc[all_ds_saap_df['Dataset'].isin(['LUAD','LSCC', 'UCEC'])]
sns.set_style('whitegrid')
fig,ax = plt.subplots(figsize=(2.5,2.5))
sns.scatterplot(data = plot_ds_saap_df, x='Normal RAAS', y='Tumor RAAS',hue='Dataset', s=40, alpha=0.7)
plt.plot(plt.ylim(), plt.ylim(), '--r')
plt.title(saap[0]+r'$\bf(Q$'+r'$\rightarrow$'+r'$\bfG)$'+saap[2:]+'\nLamin isoform A', fontsize=15)
ax.tick_params('both', labelsize=14)
plt.ylabel('Tumor log$_{10}$(RAAS)', fontsize=15)
plt.xlabel('Normal log$_{10}$(RAAS)', fontsize=15)
plt.xticks([-3, -2.5, -2, -1.5])
handles, labels = ax.get_legend_handles_labels()

# pvalues used in legend are computed below 
plt.legend(handles = handles, labels = ['UCEC, q$<$10$^{-3}$', 'LUAD, q$<$10$^{-7}$', 'LSCC, q$<$10$^{-6}$'],
           title='', loc='center', handletextpad=-0.1, labelspacing=0, bbox_to_anchor=(0.85,0.2))
plt.savefig(outdir+saap+'_TRAAS_vs_NRAAS_allDS.pdf', bbox_inches='tight')

In [ ]:
# showing pvalues between tumor and normal RAAS in patients for LaminA SAAP in each dataset
# used in plot annotation 

ps = []
for ds in tvn_datasets:
    print(ds)
    ds_df = all_ds_saap_df.loc[all_ds_saap_df['Dataset']==ds]
    print(sp.stats.ttest_rel(ds_df['Normal RAAS'].values, ds_df['Tumor RAAS'].values, alternative='less'))
    ps.append(sp.stats.ttest_rel(ds_df['Normal RAAS'].values, ds_df['Tumor RAAS'].values, alternative='less')[1])

ps = [x if ~np.isnan(x) else 1 for x in ps]
adj_ps = multipletests(ps, method='fdr_bh')[1]
for i,ds in enumerate(tvn_datasets):
    print(ds, adj_ps[i])

### Extended Data Figure 6b: RAAS vs. tumor stage

Requires clinical data provided as supplemental files in CPTAC publications - available in the google drive 

In [ ]:
# read in clinical data 
ccrcc_clin_data = pd.read_excel(ccrcc_proj_dir+'Publication/CPTAC_ccRCC_metadata/S044_CPTAC_CCRCC_Discovery_Cohort_Clinical_Data_r2_Jan2019.xlsx', skiprows=2)
lscc_clin_data = pd.read_excel(lscc_proj_dir+'Publication/S063_S058_CPTAC_LSCC_Discovery_Cohort_Clinical_Data_r2_July2021.xlsx', sheet_name='Patient_Clinical_Attributes')
pdac_clin_data = pd.read_excel(pdac_proj_dir+'Publication/S061_CPTAC_PDA_Discovery_Cohort_Clinical_Data_r1_Feb2021.xlsx', sheet_name='Patient_Clinical_Attributes')
ucec_clin_data = pd.read_excel(ucec_proj_dir+'Publication/S053_S043_CPTAC_UCEC_Discovery_Cohort_Clinical_Data_r2_Feb2020.xlsx', sheet_name='Patient_Clinical_Attributes')
luad_clin_data = pd.read_excel(luad_proj_dir+'Publication/mmc1_metadata.xlsx', sheet_name='Annotions_S1A')
brca_clin_data = pd.read_excel(brca_proj_dir+'Publication/Krug_metaData.xlsx', sheet_name='A) Metadata')

# create lists in same order as dataset_list
# list of clinical data for each dataset
clin_data_list = [ccrcc_clin_data, ucec_clin_data, brca_clin_data, luad_clin_data, pdac_clin_data, lscc_clin_data]
# list of column name indicating sample ID in clinical data for each dataset
patient_col_list = ['Case_ID', 'case_id', 'Sample.ID', 'Participant', 'case_id', 'case_id']
# list of column name indicating tumor stage in clinical data for each dataset
tumor_stage_col_list = ['Tumor_Stage_Pathological', 'tumor_stage_pathological', 'Tumor.Stage', 'Stage', 'tumor_stage_pathological', 'tumor_stage_pathological']

In [ ]:
def reduce_stage(stage):
    """ function to get Stage I, II, or III from stage subsets
        Streamlines annotation of tumor stage across datasets
    """
    if (stage=='Stage IA') or (stage=='Stage IB') or (stage=='Stage IA3') or (stage=='1A') or (stage==1) or (stage=='1B'):
        stage = 'Stage I'
    elif (stage=='Stage IIA') or (stage=='Stage IIB') or (stage=='2A') or (stage=='2B'):
        stage = 'Stage II'
    elif (stage=='Stage IIIA') or (stage=='Stage IIIB') or (stage=='Stage IIIC') or (stage=='3A'):
        stage = 'Stage III'
    return(stage)


def get_tumor_stage(ds, patient):
    """ function to get clinical tumor stage for a patient sample in a dataset """
    clin_data = clin_data_list[datasets.index(ds)]
    case_col = patient_col_list[datasets.index(ds)]
    tumor_stage_col = tumor_stage_col_list[datasets.index(ds)]
    if ds!='LUAD':
        stage = clin_data.loc[clin_data[case_col]==patient, tumor_stage_col].values[0]
    else:
        stage = clin_data.loc[(clin_data[case_col]==patient) & (clin_data['Type']=='Tumor'), tumor_stage_col].values[0]
    stage = reduce_stage(stage)
    return(stage)

In [ ]:
# plot boxplots of median RAAS per patient, stratified by patient tumor stage 

plot_df  = filt_reporter_quant_df.loc[filt_reporter_quant_df['Sample type']=='Tumor']

sns.set_style('whitegrid')
fig,ax = plt.subplots(figsize=(5,2.5))
sns.boxplot(data=plot_df, y='RAAS', x='Dataset', hue='Tumor stage', hue_order=stages, fliersize=0.75, linewidth=0.8)#, color='#AAAAAA')
plt.ylabel('log$_{10}$(RAAS)', fontsize=14)
plt.xlabel('')
ax.tick_params('both', labelsize=13)
plt.rcParams['legend.handlelength'] = 1
plt.rcParams['legend.handleheight'] = 1.125
handles, labels = ax.get_legend_handles_labels()
plt.legend(handles = handles, labels=['I', 'II', 'III','IV','unknown'], ncol=5, bbox_to_anchor=(0.5,1.15), fontsize=11, 
plt.savefig(outdir+'RAAS_vs_TumorStage_SAAPupinT_boxplot_byDataset.pdf', bbox_inches='tight')

# Extended Data Figure 9a: Pfam domain enrichment

In [ ]:
# read in Supplemental_Data_2.SAAP_proteins.xlsx
filt_saap_df = pd.read_excel(proj_dir+'Supplemental_Data_2.SAAP_proteins.xlsx', index_col=0)

# convert columns with Pfam domain information to lists 
filt_saap_df['Pfam domains'] = [x.split(';') if isinstance(x,str) else x for x in filt_saap_df['Pfam domains'].values]
filt_saap_df['SAAP in Pfam domains'] = [x.split(';') if isinstance(x,str) else x in filt_saap_df['SAAP in Pfam domains'].values]

In [ ]:
# create a dataframe with p-values for significance of RAAS in Pfam domains
# pvalues are computed by comparing the RAAS for all peptides mapping to a given domain relative to random set of 5000 RAAS using KS test

ds_saap_df = filt_saap_df
domain_list = []
raas_list = []
for i,row in ds_saap_df.iterrows():
    doms = row['Pfam domains']
    if ds =='Healthy':
        raas = row['Mean precursor RAAS']
    else:
        raas = row['Mean reporter RAAS']
    saap_in_doms = row['SAAP in Pfam domains']
    if len(doms)>0:
        keep_doms = []
        for dl, dom_list in enumerate(doms):
            for d, dom in enumerate(dom_list):
                if saap_in_doms[dl][d]==True and dom not in keep_doms:
                    keep_doms.append(dom)
        for dom in keep_doms:
            domain_list.append(dom)
            raas_list.append(raas)
    else:
        domain_list.append('Unidentified domain')
        raas_list.append(raas)
set_pfam_domains = list(set([i for j in [x for y in list(ds_saap_df['Pfam domains'].values) for x in y] for i in j]))
set_pfam_domains.append('Unidentified domain')
pfam_raas_df = pd.DataFrame(zip([ds]*len(domain_list), domain_list, raas_list), columns=['Dataset','Domain', 'Mean RAAS'])
pfam_raas_df.to_excel(data_dir+'Pfam_RAAS_df_allDS.xlsx')

# get random domain RAAS
random_domain_raas_assignments = {dom:{'Random RAAS':[]} for dom in set_pfam_domains}
raas_mean = np.mean([x for x in raas_list if ~np.isnan(x)])
raas_std = np.std([x for x in raas_list if ~np.isnan(x)])
ct=0
while ct<5000:
    for domain in set_pfam_domains:
        random_domain_raas_assignments[domain]['Random RAAS'].append(np.random.normal(raas_mean, raas_std, 1))
    ct+=1
domain_raas_dict = {domain:{'RAAS list':list(pfam_raas_df.loc[pfam_raas_df['Domain']==domain, 'Mean RAAS'].values)} for domain in set_pfam_domains}
if ds == 'Healthy':
    all_raas = [x for x in ds_saap_df['Mean precursor RAAS'] if ~np.isnan(x)]
else:
    all_raas = [x for x in ds_saap_df['Mean reporter RAAS'] if ~np.isnan(x)]

# compute significance of RAAS in each domain using KS test
domain_sig_rows = []
domain_sig_cols = ['Domain', 'N peptides', 'Median RAAS', 'Mean RAAS','p_median_greater','p_median_less', 'p_mean_greater','p_mean_less', 'p_MW', 'p_KS']
for domain, domain_dict in domain_raas_dict.items():
        domain_raas_list = [x for x in domain_dict['RAAS list'] if ~np.isnan(x)]
        n_peps = len(domain_raas_list)
        if n_peps>0:
            domain_median_raas = np.median(domain_raas_list)
            domain_mean_raas = np.mean(domain_raas_list)
            domain_random_raas_list = random_domain_raas_assignments[domain]['Random RAAS']
            ks_p = sp.stats.ks_2samp(domain_raas_list, all_raas)[1]
            domain_sig_rows.append([domain, n_peps, domain_median_raas, domain_mean_raas, p_median_greater, p_median_less, p_mean_greater, p_mean_less, mw_p, ks_p])
            domain_raas_dict[domain]['N peptides'] = n_peps
            domain_raas_dict[domain]['Median RAAS'] =  domain_median_raas
            domain_raas_dict[domain]['Mean RAAS'] = domain_mean_raas
            domain_raas_dict[domain]['p_KS'] = ks_p
domain_sig_df = pd.DataFrame(domain_sig_rows, columns=domain_sig_cols)

# adjust pvalues
ks_pvals = domain_sig_df['p_KS'].values
ks_pvals_adj = multipletests(ks_pvals, method='fdr_bh')[1]
domain_sig_df['p_KS_adj'] = ks_pvals_adj

domain_sig_df.to_excel(data_dir+'Pfam_domain_RAAS_significance_allDS.xlsx')

In [ ]:
# read in file which has pvalues for each dataset
all_ds_dom_dicts = pickle.load(open(proj_dir+'pipeline_output/analysis_dependencies/All_DS_Pfam_dom_dict.p', 'rb'))

# add dataset specific pvalues to Pfam domain dataframe
domain_sig_df['CCRCC_p'] = np.nan
domain_sig_df['UCEC_p'] = np.nan
domain_sig_df['BRCA_p'] = np.nan
domain_sig_df['LUAD_p'] = np.nan
domain_sig_df['PDAC_p'] = np.nan
domain_sig_df['LSCC_p'] = np.nan
domain_sig_df['N sig'] = np.nan

for i,row in domain_sig_df.iterrows():
    dom = row['Domain']
    ps = []
    for ds in datasets[:-1]:
        dom_dict = all_ds_dom_dicts[ds]
        if dom in dom_dict:
            dom_dict = dom_dict[dom]
            p = dom_dict['p-value']
            ps.append(p)
            domain_sig_df.loc[i, ds+'_p'] = p
    n_sig = len([x for x in ps if x<0.05])
    domain_sig_df.loc[i, 'N sig'] = n_sig
domain_sig_df.to_excel(outdir+'Pfam_domain_RAAS_significance_allDS_.xlsx')

In [ ]:
# extract domains to plot - those that have significant pvalues in at least 4 datasets 
doms2plot = domain_sig_df.loc[domain_sig_df['N sig']>=4, 'Domain'].to_list()
raas = domain_sig_df.loc[domain_sig_df['N sig']>=4, 'Median RAAS'].to_list()
sorted_idx = np.argsort(raas)
sorted_domains = [doms2plot[i] for i in sorted_idx]

x_labels = datasets
y_labels = sorted_domains

# get data for plot - color data = RAAS, size data = N peptides with domain 
color_data = pd.DataFrame(index=y_labels, columns=datasets)
size_data = pd.DataFrame(index=y_labels, columns=datasets)
for i in x_labels:
    data_dir = data_dir_list[datasets.index(i)]
    domain_ds_df = pd.read_excel(data_dir+'Pfam_domain_RAAS_significance_26May24.xlsx')
    for j in y_labels:
        dom_ds_row = domain_ds_df.loc[(domain_ds_df['Domain']==j)]
        if len(dom_ds_row)>0:
            color_data.loc[j,i] = dom_ds_row['Median RAAS'].values[0]
            n =  dom_ds_row['N peptides'].values[0]
            size_data.loc[j,i] =n

color_data = color_data.astype(float)
size_data = size_data.astype(float)

color_data.to_excel(outdir+'Pfam_Domains_RAAS_SigIn4.xlsx')
size_data.to_excel(outdir+'Pfam_Domains_Npeps_SigIn4.xlsx')

In [ ]:
# combined repetitive domains manually 

color_data = pd.read_excel(outdir+'Pfam_Domains_RAAS_SigIn4.xlsx', index_col=0)
size_data = pd.read_excel(outdir+'Pfam_Domains_Npeps_SigIn4.xlsx', index_col=0)


# pathway order - high mean raas on top 
doms2plot = list(set(color_data.index))
medians = [np.nanmean(row.values) for i, row in color_data.iterrows()]
sorted_idx = np.argsort(medians)
sorted_domains = [color_data.index.to_list()[i] for i in sorted_idx]
color_data = color_data.loc[sorted_domains]
size_data = size_data.loc[sorted_domains]

# adjust sizes for visualization purposes 
import copy
size_data2plot = copy.deepcopy(size_data)
for i,row in size_data2plot.iterrows():
    for col in size_data2plot.columns:
        if size_data.loc[i,col]<5:
            size_data2plot.loc[i,col] = 0.3
        elif size_data.loc[i,col]<50:
            size_data2plot.loc[i,col] = 0.4
        else:
            size_data2plot.loc[i,col] = 0.5

In [ ]:
# plot dotplot of domains with significantly high RAAS 

M=len(datasets)
N=len(sorted_domains)
x, y = np.meshgrid(np.arange(M), np.arange(N))

x_labels = datasets
y_labels = sorted_domains

sns.set_style('ticks')
fig, ax = plt.subplots(figsize=(13,6))
s = size_data2plot.values
c = color_data.
R = s

circles = [plt.Circle((j,i), radius=r) for r, j, i in zip(R.flat, x.flat, y.flat)]
col = PatchCollection(circles, array=c.flatten(), cmap=custom_cmap)
ax.add_collection(col)
ax.set_aspect('equal')
ax.set(xticks=np.arange(M), yticks=np.arange(N),
       xticklabels=x_labels, yticklabels=y_labels)
ax.tick_params('both', labelsize=12)
ax.set_xticks(np.arange(M+1)-0.5, minor=True)
ax.set_yticks(np.arange(N+1)-0.5, minor=True)
ax.tick_params('x', rotation=90)
cb = fig.colorbar(col, shrink=0.75, anchor=(-0.3,1))
cb.ax.tick_params(labelsize=12)
cb.ax.set_ylabel('Mean log$_{10}$(RAAS)', fontsize=14)
legend_elements = [Line2D([0], [0], marker='o', color='w', label='$<$5',markerfacecolor='#AAAAAA', markersize=8),
                  Line2D([0], [0], marker='o', color='w', label='[5,50)',markerfacecolor='#AAAAAA', markersize=12),
                  Line2D([0], [0], marker='o', color='w', label='$\geq$50',markerfacecolor='#AAAAAA', markersize=15)]

plt.legend(handles=legend_elements, handletextpad=0, title='# SAAP', bbox_to_anchor=(1.85,.25), frameon=False, fontsize=12, title_fontsize=14)
plt.savefig(outdir+'pfamdomains_sigin4_dotplot.pdf', bbox_inches='tight')

# Figure 5: Conservation of SAAP to mouse tissues

#### Incl. Extended Data Figure 10

### Extended Data Figure 10d. Upset plot of intersection of mouse and human SAAP

In [ ]:
mouse_dir = proj_dir+'pipeline_output/Mouse/'
mtp_dict = pickle.load(open(mouse_dir+'AA_subs_pipeline/MTP_dict.p', 'rb'))

# get list of SAAP identified in mouse data 
samples = list(mtp_dict.keys())
all_mouse_saap = [x for y in [list(mtp_dict[s]['mistranslated sequence'].values()) for s in samples] for x in y]
all_mouse_saap = list(set([x for y in all_mouse_saap for x in y]))

# get list of SAAP identified in CPTAC/human label-free data
filt_saap_peptides = list(set(filt_saap_df['SAAP'].to_list()))

# Add list of datasets each saap is found in to filt_saap_df (Supplemental_Data_2). 
# Was also computed above in analysis for Extended 3e.
# get_ds fxn defined above
filt_saap_df['Datasets'] = filt_saap_df.apply(lambda x: str(get_ds(x['SAAP'], x['BP'], filt_saap_df)), axis=1)

# get intersection of mouse and human SAAP
filt_overlap = [x for x in filt_saap_peptides if x in all_mouse_saap]
overlap_saap_df = filt_saap_df.loc[filt_saap_df['SAAP'].isin(filt_overlap)]

# add mouse to list of datasets intersected SAAP is found in
ds_list_mouse = []
for i,row in overlap_saap_df.iterrows():
    ds_list = row['Datasets']
    ds_list.append('Mouse')
    ds_list_mouse.append(str(ds_list))
overlap_saap_df['Datasets_w_mouse'] = ds_list_mouse

In [ ]:
# prepare dataframe for upset plot
data4upset = []
data4upset_cols = ['SAAP','BP','Datasets']
saap_bp = [row['SAAP']+':'+row['BP'] for i,row in overlap_saap_df.iterrows()]
saap_bp = list(set(saap_bp))
for sb in saap_bp:
    saap = sb.split(':')[0]
    bp = sb.split(':')[1]
    overlap_rows = [i for i,row in overlap_saap_df.iterrows() if row['SAAP']==saap and row['BP']==bp]
    datasets = overlap_saap_df.loc[overlap_rows, 'Dataset'].to_list()
    datasets.append('Mouse')
    data4upset.append([saap, bp, str(datasets)])
upset_plot_df = pd.DataFrame(data4upset, columns=data4upset_cols)

# prepare index for upset plot
datasets = ['CCRCC','UCEC','BRCA','LUAD','PDAC','LSCC','Healthy', 'Mouse']
l = [False, True]
arrays = [list(i) for i in itertools.product(l, repeat=8)]
arrays = np.transpose(arrays)
tuples = list(zip(*arrays))
multiindex = pd.MultiIndex.from_tuples(tuples, names=datasets)

# format data and index into multiindex dataframe for upset plot
saap_counts = []
idx = multiindex[1]
keepidx = []
for idx in multiindex:
    if True in idx:
        saap_ds = [datasets[i] for i,x in enumerate(idx) if x==True]
        n_saap_ds = len(upset_plot_df.loc[upset_plot_df['Datasets']==str(saap_ds)])
        if n_saap_ds>0:
            saap_counts.append(n_saap_ds)
            keepidx.append(idx)  
keepidx = np.transpose(keepidx)
tuples = list(zip(*keepidx))
multiindex = pd.MultiIndex.from_tuples(tuples, names=datasets)
upset_plot_data = pd.Series(saap_counts, index=multiindex)

# plot upset plot
fig = plt.figure(figsize=(6,3))
up.plot(upset_plot_data, show_counts=True, fig=fig, element_size=20)
plt.savefig(outdir + 'Upsetplot_only_overlap.pdf', bbox_inches='tight')

### Figure 5e: Correlation of RAAS values for SAAP intersected between human and mouse data

In [ ]:
dp_ev_dict = pickle.load(open(mouse_dir+'AA_subs_pipeline/DP_search_evidence_dict.p','rb'))

# extract RAAS values for intersected SAAP into dataframe
plot_rows = []
plot_cols = ['Mouse tissue', 'SAAP', 'BP', 'SAAP intensity', 'BP intensity', 'Mouse RAAS', 'Human RAAS','Dataset']
for s in mtp_dict.keys():
    s_mtp = mtp_dict[s]
    s_ev = dp_ev_dict[s]
    for saap in filt_overlap:
        saap_keys = [k for k,v in s_mtp['mistranslated sequence'].items() if saap in v]
        saap_int = np.sum([s_mtp['Intensity'][k] for k in saap_keys])
        bps = [s_mtp['DP Base Sequence'][k] for k in saap_keys]
        for bp in bps:
            bp_int = s_ev.loc[s_ev['Sequence']==bp, 'Intensity'].sum()
            mouse_raas = np.log10(saap_int/bp_int)
            human_data = overlap_saap_df.loc[(overlap_saap_df['SAAP']==saap) & (overlap_saap_df['BP']==bp)]
            for i,row in human_data.iterrows():
                ds = row['Dataset']
                human_raas = row['Mean precursor RAAS']
                plot_rows.append([s, saap, bp, saap_int, bp_int, mouse_raas, human_raas, ds])

plot_df = pd.DataFrame(plot_rows, columns=plot_cols)
plot_df.drop_duplicates(inplace=True)
plot_df.dropna(how='any', inplace=True)

# plot scatterplot of mouse vs human RAAS colored by tissue
sns.set_style('whitegrid')
fig,ax = plt.subplots(figsize=(2.5,2.5))
sns.scatterplot(data=plot_df, x='Human RAAS', y='Mouse RAAS', hue='Mouse tissue', alpha=0.6, s=40)
plt.legend(bbox_to_anchor=(0.5,1.1), fontsize=12, title='Mouse tissue', loc='center', 
           ncol=3, frameon=False, title_fontsize=12, handletextpad=-0.5, columnspacing=0.3)
ax.tick_params('both', labelsize=13)
plt.ylabel('Mouse log$_{10}$(RAAS)', fontsize=14)
plt.xlabel('Human log$_{10}$(RAAS)', fontsize=14)

r,p = sp.stats.pearsonr(plot_df['Human RAAS'].values, plot_df['Mouse RAAS'].values)
ax.text(0.85, -3.2, 'r='+str(np.round(r,2)), fontsize=12)
ax.text(0.85,-4.1, 'p$<$10$^{-10}$', fontsize=12)
plt.plot((-4.8,4), (-4.8,4), '--k', linewidth=0.8)
plt.xlim(plt.ylim())
plt.xticks([-4,-2,0,2,4]);
plt.savefig('Mouse_human_RAAS_correlation_allPepquant.pdf', bbox_inches='tight')

### Figure 5f: ANOVA to attribute variance in RAAS to species, tissue type, and AAS type

In [ ]:
filt_prec_df = pd.read_excel(proj_dir + 'Supplemental_Data_3.SAAP_precursor_quant.xlsx', index_col=0)

# tissue type that each CPTAC dataset represents
cptac_tissue_dict = {'CCRCC':'kidney', 'PDAC':'pancreas', 'BRCA':'breast','UCEC':'endometrium','LSCC':'lung','LUAD':'lung'}

# prepare dataframe for ANOVA 
plot_rows = []
plot_cols = ['SAAP', 'BP', 'AAS', 'RAAS', 'Dataset', 'Tissue', 'Species','Dataset_type']
for s in mtp_dict.keys():
    s_mtp = mtp_dict[s]
    s_ev = dp_ev_dict[s]
    for saap in filt_overlap:
        saap_keys = [k for k,v in s_mtp['mistranslated sequence'].items() if saap in v]
        saap_int = np.sum([s_mtp['Intensity'][k] for k in saap_keys])
        bps = [s_mtp['DP Base Sequence'][k] for k in saap_keys]
        for bp in bps:
            bp_int = s_ev.loc[s_ev['Sequence']==bp, 'Intensity'].sum()
            mouse_raas = np.log10(saap_int/bp_int)
            mouse_tissue = s
            human_data = overlap_saap_df.loc[(overlap_saap_df['SAAP']==saap) & (overlap_saap_df['BP']==bp)]
            for i,row in human_data.iterrows():
                ds = row['Dataset']
                prec_df = filt_prec_df.loc[(filt_prec_df['Dataset']==ds) & (filt_prec_df['SAAP']==saap) & (filt_prec_df['BP']==bp)]
                for i,row in prec_df.iterrows():
                    raas = row['RAAS']
                    aas = row['AAS']
                    if ds=='Healthy':
                        tissue = row['TMT/Tissue']
                        ds_type = 'Label-free'
                    else:
                        tissue = cptac_tissue_dict[ds]
                        ds_type = 'TMT'
                    plot_rows.append([saap, bp, aas, raas, ds, tissue, 'Human', ds_type])
            plot_rows.append([saap, bp, aas, mouse_raas, 'Kuster_mouse', mouse_tissue, 'Mouse', 'Label-free'])
plot_df = pd.DataFrame(plot_rows, columns=plot_cols)

anova_df = plot_df
anova_df.replace(np.inf, np.nan, inplace=True)
anova_df.replace(-np.inf, np.nan, inplace=True)
anova_df.dropna(how='any', inplace=True)

In [ ]:
# generate ANOVA model with independent variables of interest and RAAS dependent variable
model = ols('RAAS ~ C(Tissue) + C(AAS) + C(Species)', data=anova_df).fit()
df = sm.stats.anova_lm(model, typ=3)
df.reset_index(inplace=True)
df.columns = ['Variable']+df.columns.to_list()[1:]

# generate dataframe with ANOVA results to plot 
plot_df = df.loc[1:3, ['Variable', 'F', 'PR(>F)']]
for i in plot_df.index:
    plot_df.loc[i,'PR(>F)'] = '{:0.0e}'.format(plot_df.loc[i,'PR(>F)'])
plot_df['Variable'] = ['Tissue','AAS','Species']
plot_df = plot_df.loc[[1,3,2],:]

# plot ANOVA results 
sns.set_style('whitegrid')
fig,ax = plt.subplots(figsize=(4,4))
sns.barplot(data=plot_df, x='Variable', y='F', hue='PR(>F)', dodge=False, palette='Greys',
           order=['Tissue', 'AAS', 'Species'])#, hue_order=['Species','Data type','Tissue','AAS'])
handles, labels = ax.get_legend_handles_labels()
plt.legend(handles = handles, labels = ['p<10$^{-24}$', 'p<10$^{-38}$', 'p<10$^{-248}$'],
           loc='upper left', fontsize=12, title_fontsize=13, labelspacing=0.1)
plt.xlabel('')
plt.ylabel('ANOVA F statistic', fontsize=14)
ax.tick_params('both', labelsize=13)
plt.savefig('mouse_ANOVA_tissue_AAS_result.pdf', bbox_inches='tight')